# Library

In [1]:
import pandas as pd
import numpy as np
import re, os
from tqdm import tqdm

## [로드] 파일

In [4]:
ods_fold = 'D:\과업\[한국환경공단]자동차_환경_빅데이터_구축_2차\data\ODS'

In [6]:
# about 45s
df = pd.read_csv(os.path.join(ods_fold, '[ODS][G4]등록정보&제원정보(말소제거)(한글컬럼)(2022.12.29).csv'), low_memory=False, encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511251 entries, 0 to 1511250
Data columns (total 59 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   차량관리번호        1511251 non-null  object 
 1   차량등록번호        1511251 non-null  object 
 2   본거지법정동코드      1511251 non-null  int64  
 3   소유자구분         1511251 non-null  object 
 4   차종_등록정보       1511249 non-null  object 
 5   용도            1511251 non-null  object 
 6   미정1_x         1511251 non-null  object 
 7   현재회원번호        1510412 non-null  object 
 8   미정2_x         1511251 non-null  object 
 9   미정3_x         1511251 non-null  object 
 10  차대번호          1511251 non-null  object 
 11  최초등록일자        1511251 non-null  int64  
 12  차량연식          1511251 non-null  int64  
 13  차량제작일자        1342461 non-null  float64
 14  배출가스검사유효일자    1511239 non-null  float64
 15  제원관리번호        1511251 non-null  object 
 16  배출가스인증번호      1387542 non-null  object 
 17  배출가스등급        1511251 non-n

In [4]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류'],
      dtype='object')

In [5]:
df['차종_등록정보'].unique()

array(['승용', '화물', '승합', nan, '특수'], dtype=object)

In [6]:
df['용도'].unique()

array(['자가용', '영업용', '관용'], dtype=object)

In [7]:
df['저공해조치종류'].unique()

array([nan,  4.,  3.,  1.,  5.,  2.])

In [8]:
df['저공해조치구조변경YN'].unique()

array(['N', 'Y'], dtype=object)

In [9]:
df['차종_제원정보'].unique()

array(['승용', '화물', '승합', '특수'], dtype=object)

In [10]:
df['차종분류'].unique()

array(['일반', '일반형-픽업', '일반형-카고', '승용겸화물', '특수', '다목적', '특수용도형-특수용도형',
       '기타', '밴', '덤프', '특수작업형', '견인', '구난', '특수용도형-탱크로리', '특수용도형-피견인'],
      dtype=object)

In [11]:
df['차종유형'].unique()

array(['소형', '대형', '중형', '경형'], dtype=object)

In [12]:
df['자동차형식'].unique()

array([nan, 'YR-4-A-S', 'ACV30', ..., 'F7CKF5', 'HD25-9MT0-LHFbF',
       'HD050-8MT2-LGc-A'], dtype=object)

# 분석시 필요한 컬럼

In [13]:
# es_col = ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급', '차량말소YN', '차명', '차종_제원정보', '차종분류', '차종유형', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '구동형식', '변속기종류']

# [로드] 4등급 제원정보(필수)

In [14]:
# # 14.3s
# df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)
# df.info()

# 전처리

## 차대번호 중복

In [15]:
len(df['차대번호'].unique())

1511211

In [16]:
df.shape

(1511251, 40)

In [17]:
df.loc[df.duplicated('차대번호', keep=False) == True].sort_values('차대번호').shape

(77, 40)

In [18]:
df.loc[df.duplicated('차대번호', keep=False) == True].sort_values('차대번호').head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,승인일자,구동형식,변속기종류
982713,1GBEL19W3XB192382-02,39고6855,1141011800,주민,승용,자가용,1GBEL19W3XB192382,20091013,1999,19991231.0,...,4300.0,6.0,4805.0,1980.0,1915.0,독일,N,20090805,NaN,A
962690,1GBEL19W3XB192382-01,36거2927,1111017500,주민,승용,자가용,1GBEL19W3XB192382,20090917,1999,20081124.0,...,4300.0,6.0,4805.0,1980.0,1915.0,독일,N,20090805,NaN,A
261516,2B7HB21X2TK148416-01,서울32머6063,1117012400,기타,승용,자가용,2B7HB21X2TK148416,19961231,1996,19961231.0,...,5204.0,7.0,5380.0,2110.0,2300.0,미국,N,20020420.0,NaN,A
348629,2B7HB21X2TK148416-02,40버3368,2872036022,주민,승용,자가용,2B7HB21X2TK148416,20020604,1996,19960529.0,...,3903.0,7.0,5130.0,2110.0,1900.0,대한민국,N,20020614,NaN,A
220048,3B3ES67C6ST512047-01,전북31러9026,4513040021,기타,승용,자가용,3B3ES67C6ST512047,19951113,1995,19951113.0,...,1996.0,5.0,4364.0,1715.0,1390.0,미국,N,19960415,NaN,A


In [19]:
# 차대번호 중복 예시
df.loc[df.duplicated('차대번호', keep=False) == True].sort_values('차대번호').to_excel('analysis2/[G4][차대번호]중복_예시.xlsx')

In [20]:
df.sort_values(['최초등록일자', '배출가스검사유효일자', '차대번호'], ascending=False).head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,승인일자,구동형식,변속기종류
1511250,KMFLA18KP8C028585-01,83노5980,2653010500,법인,화물,자가용,KMFLA18KP8C028585,20221125,2008,20071224.0,...,5899.0,3.0,7010.0,2420.0,2660.0,대한민국,N,20071004,NaN,M
1511249,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,5899.0,3.0,7710.0,2420.0,2960.0,대한민국,N,20080529,NaN,A
1511248,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,5899.0,2.0,6675.0,2255.0,2950.0,대한민국,N,20080518,NaN,M
1511247,KMCGK17HP9C112030-01,93루6527,4518010400,주민,화물,자가용,KMCGK17HP9C112030,20220407,2009,20090316.0,...,3933.0,3.0,6225.0,2030.0,2700.0,대한민국,N,20090202,NaN,M
1511245,KLACA26RD8B226753-01,153구6446,2826011300,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,1991.0,7.0,4635.0,1850.0,1720.0,대한민국,N,20070703,NaN,A


In [21]:
df.shape

(1511251, 40)

### 차대번호 중복 처리
- 최초등록일자, 배출가스검사유효일자 최신

In [22]:
# about 6s
df1 = df.sort_values(['최초등록일자', '배출가스검사유효일자', '차대번호'], ascending=False).reset_index(drop=True).drop_duplicates('차대번호', keep='first').reset_index(drop=True)

In [23]:
df1['차대번호'].value_counts()

KMFLA18KP8C028585    1
KNAJC52438A790779    1
KMHSJ81WP8U284243    1
KMHSJ81WP8U284255    1
KMHSJ81WP8U284280    1
                    ..
KMFWBX7JBAU207274    1
KMFZAY7JAAU581614    1
KMFZAZ7JAAU579198    1
KMFZAZ7JBAU581356    1
CN-62-T-2213         1
Name: 차대번호, Length: 1511211, dtype: int64

In [24]:
df1[df1['차대번호'] == 'KNADA2322JS052345']['차명']

1506390    프라이드
Name: 차명, dtype: object

In [25]:
df2 = df1.copy()

## 연료 정보 수정
- 엔진형식, 차명으로 확인

In [26]:
f = df2.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1152697
5,휘발유,332313
1,LGP(액화석유가스),26089
3,기타연료,39
0,CNG(압축천연가스),1
4,알코올,1


In [27]:
g = df2.groupby('연료', as_index=False)['차대번호'].groups
g['기타연료']

Int64Index([   3398, 1204214, 1469511, 1481118, 1482406, 1484113, 1484758,
            1484934, 1485283, 1485766, 1487537, 1489525, 1489663, 1489747,
            1489892, 1490095, 1490261, 1490558, 1490888, 1491267, 1491493,
            1492109, 1493113, 1493863, 1494465, 1496009, 1496852, 1499227,
            1501010, 1501890, 1502043, 1502364, 1502721, 1504515, 1505466,
            1505629, 1505925, 1507140, 1510807],
           dtype='int64')

In [28]:
# B5 : 휘발유
# G16SF : 휘발유
# H20MR : 휘발유
# MSV3 : 휘발유
# 01821300581 : 
# G15CF : 휘발유
# G4AJ : 휘발유
df2.loc[df2['연료'] == '기타연료', '엔진형식'].unique()

array(['-', 'G4AJ', 'MSV3', 'B5', 'H20MR', 'G16SF', nan, 'G15CF',
       '01821300581'], dtype=object)

In [29]:
df2.loc[df2['엔진형식'] == 'B5', '연료'] = '휘발유'
df2.loc[df2['엔진형식'] == 'G16SF', '연료'] = '휘발유'
df2.loc[df2['엔진형식'] == 'H20MR', '연료'] = '휘발유'
df2.loc[df2['엔진형식'] == 'MSV3', '연료'] = '휘발유'
df2.loc[df2['엔진형식'] == 'G15CF', '연료'] = '휘발유'
df2.loc[df2['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
df2[df2['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
3398,에어스트림탐지견수송트레일러,-
1499227,스텔라오토매틱,NaN
1501010,벤즈 230E,NaN
1507140,벤즈230E,01821300581
1510807,도요다크라운로얄싸롱,NaN


In [30]:
df2.loc[df2['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'
df2.loc[df2['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
df2.loc[df2['차명'] == '벤즈 230E', '연료'] = '휘발유'
df2.loc[df2['차명'] == '벤즈230E', '연료'] = '휘발유'
df2[df2['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
3398,에어스트림탐지견수송트레일러,-


In [31]:
f = df2.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1152697
5,휘발유,332353
1,LGP(액화석유가스),26089
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


In [32]:
f.columns = ['연료', '4등급']
f

,연료,4등급
2,경유,1152697
5,휘발유,332353
1,LGP(액화석유가스),26089
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


## [excel] 연료별 통계

In [33]:
f.to_excel('analysis2/[G4]연료별_차대번호_통계.xlsx', index=False)

## 지역별 수량 통계

## [로드] 법정동코드

In [34]:
# about 6s
code = pd.read_excel('data/법정동코드_2022.10.14.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [35]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y


In [36]:
code['code1'] = code['법정동코드'].astype('str').str[:5]

In [37]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y,11000
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y,11110
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y,11110
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y,11110
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y,11110


In [38]:
code.shape

(46340, 10)

In [39]:
code2 = code.drop_duplicates(['시도', 'code1'], keep='first').reset_index(drop=True)
code2.shape

(461, 10)

In [40]:
code2.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y,11000
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y,11110
2,1114000000,서울,중구,NaN,NaN,NaN,존재,수도권,Y,11140
3,1117000000,서울,용산구,NaN,NaN,NaN,존재,수도권,Y,11170
4,1120000000,서울,성동구,NaN,NaN,NaN,존재,수도권,Y,11200


In [41]:
code2[code2['시도'] == '세종']

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
122,3611000000,세종,NaN,NaN,NaN,NaN,존재,중부권,Y,36110


In [42]:
code2['code1'].dtype

dtype('O')

In [43]:
code2 = code2[['시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', 'code1']]

In [44]:
code2.head()

,시도,시구군,폐지여부,대기관리권역,대기관리권역YN,code1
0,서울,NaN,존재,수도권,Y,11000
1,서울,종로구,존재,수도권,Y,11110
2,서울,중구,존재,수도권,Y,11140
3,서울,용산구,존재,수도권,Y,11170
4,서울,성동구,존재,수도권,Y,11200


### 법정동 코드 추가

In [45]:
df2['본거지법정동코드'].head()

0    2653010500
1    4824034000
2    4482533000
3    4518010400
4    2826011300
Name: 본거지법정동코드, dtype: int64

In [46]:
df2['본거지법정동코드'].isnull().sum()

0

In [47]:
df2['code1'] = df2['본거지법정동코드'].astype('str').str[:5]
df2['code1'].head()

0    26530
1    48240
2    44825
3    45180
4    28260
Name: code1, dtype: object

In [48]:
df3 = df2.merge(code2, on='code1', how='left')
df3[['code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN']].head()

,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN
0,26530,부산,사상구,존재,동남권,Y
1,48240,경남,사천시,존재,동남권,N
2,44825,충남,태안군,존재,중부권,Y
3,45180,전북,정읍시,존재,중부권,N
4,28260,인천,서구,존재,수도권,Y


In [49]:
df3.shape

(1511211, 46)

In [50]:
df3['시도'].isnull().sum()

0

In [51]:
df3['시도'].unique()

array(['부산', '경남', '충남', '전북', '인천', '대구', '대전', '전남', '경기', '울산', '광주',
       '강원', '경북', '충북', '서울', '제주', '세종'], dtype=object)

In [52]:
df3['시구군'].isnull().sum()

8075

In [53]:
df3['시구군'].unique()

array(['사상구', '사천시', '태안군', '정읍시', '서구', '달성군', '유성구', '동래구', '여수시',
       '고양시 일산동구', '논산시', '울주군', '화성시', '청양군', '남구', '부여군', '김제시', '완주군',
       '정선군', '구미시', '거제시', '평택시', '영동군', '원주시', '성남시 중원구', '목포시', '완도군',
       '파주시', '동두천시', '남양주시', '청주시 흥덕구', '춘천시', '단양군', '동구', '김천시',
       '전주시 덕진구', '함안군', '영주시', '안성시', '칠곡군', '사하구', '김포시', '서산시', '달서구',
       '군산시', '종로구', '성남시 수정구', '청주시 상당구', '인제군', '장성군', '광산구', '청도군',
       '경산시', '천안시 서북구', '아산시', '동대문구', '창원시 의창구', '부천시', '홍성군', '구리시',
       '포천시', '중구', '철원군', '음성군', '창원시 성산구', '부평구', '양산시', '북구', '보은군',
       '여주시', '용산구', '용인시 기흥구', '광양시', '서천군', '서초구', '당진시', '천안시 동남구',
       '고흥군', '충주시', '양주시', '수원시 장안구', '강릉시', '강화군', '성동구', '노원구',
       '안산시 상록구', '성북구', '영등포구', '구로구', '고령군', '제천시', '강남구', '강동구',
       '미추홀구', '광진구', '시흥시', '영암군', '연제구', '창녕군', '해운대구', '광주시', '영천시',
       '진주시', '고성군', '제주시', '공주시', '마포구', '창원시 진해구', '용인시 처인구', '서대문구',
       '경주시', '화순군', '횡성군', '포항시 남구', '나주시', '금정구', '청주시 청원구', '연수구',
      

In [54]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [55]:
l1 = df3.groupby(['대기관리권역', '대기관리권역YN', '연료'])['차대번호'].count().unstack()
l1

연료               CNG(압축천연가스)  LGP(액화석유가스)        경유  기타연료  알코올       휘발유
대기관리권역 대기관리권역YN                                                         
그 외    N                 NaN       1171.0   68951.0   NaN  NaN   12854.0
남부권    N                 1.0        742.0   37520.0   NaN  NaN    5857.0
       Y                 NaN       1235.0   65038.0   NaN  NaN   15033.0
동남권    N                 NaN        777.0   81561.0   1.0  NaN   12405.0
       Y                 NaN       5138.0  246320.0   NaN  NaN   65825.0
수도권    N                 NaN        131.0    9416.0   NaN  NaN    2222.0
       Y                 NaN      12411.0  428411.0   NaN  1.0  168989.0
중부권    N                 NaN        580.0   40534.0   NaN  NaN    6294.0
       Y                 NaN       3904.0  174946.0   NaN  NaN   42874.0

In [56]:
l2 = df3.groupby(['시도', '대기관리권역YN', '연료'])['차대번호'].count().unstack()
l2

연료           CNG(압축천연가스)  LGP(액화석유가스)        경유  기타연료  알코올      휘발유
시도 대기관리권역YN                                                        
강원 N                 NaN        798.0   47022.0   NaN  NaN   8504.0
경기 N                 NaN        117.0    7970.0   NaN  NaN   2006.0
   Y                 NaN       6772.0  258734.0   NaN  NaN  86799.0
경남 N                 NaN        319.0   35440.0   1.0  NaN   5744.0
   Y                 NaN        713.0   61569.0   NaN  NaN  12983.0
경북 N                 NaN        458.0   46121.0   NaN  NaN   6661.0
   Y                 NaN       1003.0   49696.0   NaN  NaN  12996.0
광주 Y                 NaN        709.0   30327.0   NaN  NaN   8326.0
대구 Y                 NaN       1659.0   50619.0   NaN  NaN  16775.0
대전 Y                 NaN       1236.0   29972.0   NaN  NaN  10430.0
부산 Y                 NaN       1492.0   58089.0   NaN  NaN  17202.0
서울 Y                 NaN       4353.0  110407.0   NaN  NaN  63736.0
세종 Y                 NaN         69.0    6681.0   NaN  NaN   1325.0
울산 Y                 NaN        271.0   26347.0   NaN  NaN   5869.0
인천 N                 NaN         14.0    1446.0   NaN  NaN    216.0
   Y                 NaN       1286.0   59270.0   NaN  1.0  18454.0
전남 N                 1.0        742.0   37520.0   NaN  NaN   5857.0
   Y                 NaN        526.0   34711.0   NaN  NaN   6707.0
전북 N                 NaN        391.0   28879.0   NaN  NaN   4235.0
   Y                 NaN        585.0   31537.0   NaN  NaN   7280.0
제주 N                 NaN        373.0   21929.0   NaN  NaN   4350.0
충남 N                 NaN         32.0    2360.0   NaN  NaN    452.0
   Y                 NaN        951.0   67399.0   NaN  NaN  14300.0
충북 N                 NaN        157.0    9295.0   NaN  NaN   1607.0
   Y                 NaN       1063.0   39357.0   NaN  NaN   9539.0

### [excel] 지역별 수량 통계

In [57]:
local = pd.ExcelWriter('analysis2/[G4][차대번호]지역별_통계.xlsx', engine='openpyxl')
l1.to_excel(local, sheet_name='권역')
l2.to_excel(local, sheet_name='지역')
local.save()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_27496\3012724629.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  local.save()


# [출력] 4등급 제원정보(csv)
- 차량말소 적용, 차대번호 중복제거, 연료정보 수정, 지역명 추가

In [58]:
# 4등급 제원정보 수정(차량말소 적용, 차대번호 중복제거, 연료정보 수정, 지역명 추가)
# about 30s
df3.to_csv('data/[BD1][G4]등록정보&제원정보(차대번호_중복제거,연료수정,지역추가)(2022-12-29).csv', index=False)

# [로드] 4등급 제원정보(csv)

In [59]:
# # 약 24s
# df3 = pd.read_csv('data/220819_22.06월기준_4등급2.csv', encoding='cp949', index_col=0)
# df3.info()

# 분석

## 지역별 연료별 통계

In [60]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [61]:
statics_by_local_fuel = df3.groupby(['시도', '연료'])['차대번호'].count()
statics_by_local_fuel

시도  연료         
강원  LGP(액화석유가스)       798
    경유              47022
    휘발유              8504
경기  LGP(액화석유가스)      6889
    경유             266704
    휘발유             88805
경남  LGP(액화석유가스)      1032
    경유              97009
    기타연료                1
    휘발유             18727
경북  LGP(액화석유가스)      1461
    경유              95817
    휘발유             19657
광주  LGP(액화석유가스)       709
    경유              30327
    휘발유              8326
대구  LGP(액화석유가스)      1659
    경유              50619
    휘발유             16775
대전  LGP(액화석유가스)      1236
    경유              29972
    휘발유             10430
부산  LGP(액화석유가스)      1492
    경유              58089
    휘발유             17202
서울  LGP(액화석유가스)      4353
    경유             110407
    휘발유             63736
세종  LGP(액화석유가스)        69
    경유               6681
    휘발유              1325
울산  LGP(액화석유가스)       271
    경유              26347
    휘발유              5869
인천  LGP(액화석유가스)      1300
    경유              60716
    알코올                 1
    휘발유             18

In [62]:
statics_by_local_fuel2 = statics_by_local_fuel.unstack(['연료'])
statics_by_local_fuel2

연료,CNG(압축천연가스),LGP(액화석유가스),경유,기타연료,알코올,휘발유
시도,,,,,,
강원,NaN,798.0,47022.0,NaN,NaN,8504.0
경기,NaN,6889.0,266704.0,NaN,NaN,88805.0
경남,NaN,1032.0,97009.0,1.0,NaN,18727.0
경북,NaN,1461.0,95817.0,NaN,NaN,19657.0
광주,NaN,709.0,30327.0,NaN,NaN,8326.0
대구,NaN,1659.0,50619.0,NaN,NaN,16775.0
대전,NaN,1236.0,29972.0,NaN,NaN,10430.0
부산,NaN,1492.0,58089.0,NaN,NaN,17202.0
서울,NaN,4353.0,110407.0,NaN,NaN,63736.0


In [63]:
statics_by_local_fuel2.to_excel('analysis2/[G4][법정동코드]지역_연료별_통계.xlsx')

## 배출가스인증번호

### 배출가스인증번호 내역 확인

In [64]:
df3['배출가스인증번호'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1511211 entries, 0 to 1511210
Series name: 배출가스인증번호
Non-Null Count    Dtype 
--------------    ----- 
1387523 non-null  object
dtypes: object(1)
memory usage: 23.1+ MB


In [65]:
df3['배출가스인증번호'].isnull().sum()

123688

In [66]:
df3['배출가스인증번호'].value_counts()

7MY-HD-14-68      266633
8MY-KM-13-19       92265
6MY-KM-14-61       74800
9MY-KM-14-41       66913
7MY-HD-14-46       59606
                   ...  
6MYHD1497              1
AMY-PPE-14-631         1
7MY-SK-24-03           1
NMY-JK-S-107           1
7MY-KL-0M-24           1
Name: 배출가스인증번호, Length: 306, dtype: int64

In [67]:
temp = df3['배출가스인증번호'].value_counts().reset_index()
temp

,index,배출가스인증번호
0,7MY-HD-14-68,266633
1,8MY-KM-13-19,92265
2,6MY-KM-14-61,74800
3,9MY-KM-14-41,66913
4,7MY-HD-14-46,59606
...,...,...
301,6MYHD1497,1
302,AMY-PPE-14-631,1
303,7MY-SK-24-03,1
304,NMY-JK-S-107,1


##### 빈칸 처리

In [68]:
gas_df = df3.copy()
gas_df.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,삭제YN,승인일자,구동형식,변속기종류,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN
0,KMFLA18KP8C028585-01,83노5980,2653010500,법인,화물,자가용,KMFLA18KP8C028585,20221125,2008,20071224.0,...,N,20071004,NaN,M,26530,부산,사상구,존재,동남권,Y
1,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,N,20080529,NaN,A,48240,경남,사천시,존재,동남권,N
2,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,N,20080518,NaN,M,44825,충남,태안군,존재,중부권,Y
3,KMCGK17HP9C112030-01,93루6527,4518010400,주민,화물,자가용,KMCGK17HP9C112030,20220407,2009,20090316.0,...,N,20090202,NaN,M,45180,전북,정읍시,존재,중부권,N
4,KLACA26RD8B226753-01,153구6446,2826011300,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,N,20070703,NaN,A,28260,인천,서구,존재,수도권,Y


In [69]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호']

9164        
13026       
1178694     
1220009     
1319091     
1382158     
1460000     
1474876     
1490530     
1497241     
1501010     
1510807     
Name: 배출가스인증번호, dtype: object

In [70]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].shape

(12,)

In [71]:
blank_index = gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].index
blank_index

Int64Index([   9164,   13026, 1178694, 1220009, 1319091, 1382158, 1460000,
            1474876, 1490530, 1497241, 1501010, 1510807],
           dtype='int64')

In [72]:
gas_df.groupby(['배출가스인증번호'])['차대번호'].count()

배출가스인증번호
                   12
 7MY-DC-24-72       1
 7MY-HD-24-30       8
 8MY-DC-24-12       1
3MY-FK-08        1064
                 ... 
XMY-HD-47         887
XMY-HD-48         354
XMY-HD-55        2469
XMY-KM-47        3098
YKM-KM-12          70
Name: 차대번호, Length: 306, dtype: int64

In [73]:
gas_df[gas_df['배출가스인증번호'] == ' 7MY-DC-24-72'].index

Int64Index([803468], dtype='int64')

In [74]:
# 빈칸을 포함한 예시
gas_df.loc[803468, '배출가스인증번호']

' 7MY-DC-24-72'

In [75]:
gas_df.loc[blank_index, '배출가스인증번호'] = np.nan

In [76]:
gas_df['배출가스인증번호'] = gas_df['배출가스인증번호'].str.replace(' ', '')
gas_df.loc[810574, '배출가스인증번호']

'7MY-DC-23-69'

In [77]:
gas_df['배출가스인증번호'].isnull().sum()

123700

In [78]:
gas_df.groupby(['배출가스인증번호'])['차대번호'].count()

배출가스인증번호
3MY-FK-08       1064
4MY-AD-20         34
4MY-DC-33          7
4MY-PA-06         13
5MY-AD-13-06     378
                ... 
XMY-HD-47        887
XMY-HD-48        354
XMY-HD-55       2469
XMY-KM-47       3098
YKM-KM-12         70
Name: 차대번호, Length: 302, dtype: int64

In [79]:
df3.loc[:, '배출가스인증번호'].isnull().sum()

123688

In [80]:
# 배출가스인증번호 현황
gas_df.groupby(['배출가스인증번호'])['차대번호'].count().to_excel('analysis2/[G4][배출가스인증번호]유니크값_통계.xlsx')

In [81]:
gas_df.shape

(1511211, 46)

#### 12\. [e]배출가스인증번호

In [82]:
# - 정보
#     - 총 : 1511211
#     - null : 123700
#     - 정상
#         - 2005.01.04 이후 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}-[숫자]{2}, 12자리) : 1137958
#         - 2005.01.04 이전 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}, 9자리) : 
#     - 오류(null 값 제외) : 21959
#       - 예시
#           - 163      TY87-배K-02
#           - 173    7MY-KJ-0M-24
#           - 216      TY87-배K-03
#           - 236      TY87-배K-10
#           - 293      TY87-배K-03

In [83]:
# 2005년 이후 표기 방식 중 길이가 12자리가 아닌 값 : 
#     - [max] 차량연식 : 
# 2005년 이전 표기 방식 중 길이가 9자리가 아닌 값 : 

In [84]:
# 2005.01.04 이후 표기 방식 : 1137958
count = 0
p121 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}[-][0-9]{2}')
idx121 = []
for i, one in tqdm(enumerate(gas_df.loc[:, '배출가스인증번호'].to_numpy())):
    if p121.search(str(one)):
        idx121.append(i)
        count += 1
count

1511211it [00:01, 868714.30it/s] 


1137958

In [85]:
# 2005.01.04 이후 표기 방식이 아닌 값 : 373253
count = 0
idx122 = []
for i, one in tqdm(enumerate(gas_df.loc[:, '배출가스인증번호'].to_numpy())):
    if not p121.search(str(one)):
        idx122.append(i)
        count += 1
count

1511211it [00:01, 1025629.64it/s]


373253

In [86]:
gas_df.loc[:, '배출가스인증번호'].reindex(idx122)[:5]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_27496\1946529048.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  gas_df.loc[:, '배출가스인증번호'].reindex(idx122)[:5]


0     NaN
1     NaN
2     NaN
4     NaN
18    NaN
Name: 배출가스인증번호, dtype: object

In [87]:
# 2005.01.04 이전 배출가스 표기 방식 : 227594
count = 0
p123 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}')
idx123 = []
for i, one in tqdm(enumerate(gas_df.loc[idx122, '배출가스인증번호'].to_numpy())):
    if p123.search(str(one)):
        idx123.append(idx122[i])
        count += 1
count

373253it [00:00, 871448.63it/s]


227594

In [88]:
gas_df.loc[idx123, ['최초등록일자', '차량연식', '차량제작일자', '차종유형', '배출가스인증번호']][:5]

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
155,20190312,2001,20010305.0,중형,WMY-HD-32
195,20190114,1999,19990123.0,중형,WMY-HD-32
198,20190110,2000,20001231.0,중형,WMY-HD-32
204,20190103,1998,19980916.0,중형,WMY-HD-32
220,20181220,2002,20020405.0,중형,WMY-HD-32


In [89]:
# null 값 : 123700
idx124 = gas_df[gas_df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].index
len(idx124)

123700

In [90]:
gas_df.loc[:, '배출가스인증번호'].reindex(idx124)[:5]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_27496\105041851.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  gas_df.loc[:, '배출가스인증번호'].reindex(idx124)[:5]


0     NaN
1     NaN
2     NaN
4     NaN
18    NaN
Name: 배출가스인증번호, dtype: object

In [91]:
# 2005.01.04 이후 표기 방식이 아닌 값 중 null 값 제외한 오류 값 : 21959
idx125 = sorted(set(idx122) - set(idx124) - set(idx123))
len(idx125)

21959

In [92]:
gas_df.loc[:, '배출가스인증번호'].reindex(idx125)[:5]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_27496\3501197350.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  gas_df.loc[:, '배출가스인증번호'].reindex(idx125)[:5]


43     6MY-SS-0M-24
142    FMY-VTK-14-1
182    FMY-VTK-14-1
185    6MY-SS-0M-24
186    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [93]:
# 2005년 이상 배출가스인증번호 데이터
gas_temp = gas_df.loc[idx123, ['최초등록일자', '차량연식', '차량제작일자', '차종유형', '배출가스인증번호']]
gas_temp[gas_temp['차량연식'] >= 2005].head()

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
8449,20120409,2011,20111219.0,대형,8MY-PA-14-3
15191,20120111,2011,20111219.0,대형,8MY-PA-14-3
15526,20120109,2011,20110517.0,대형,8MY-PA-14-3
16067,20120106,2011,20110216.0,대형,8MY-PA-14-3
19593,20120102,2011,20110516.0,대형,8MY-PA-14-3


In [94]:
# 2005년 이상년도에서 맞지않는 배출가스인증번호 샘플 : 2971
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12)]

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
8449,20120409,2011,20111219.0,대형,8MY-PA-14-3
15191,20120111,2011,20111219.0,대형,8MY-PA-14-3
15526,20120109,2011,20110517.0,대형,8MY-PA-14-3
16067,20120106,2011,20110216.0,대형,8MY-PA-14-3
19593,20120102,2011,20110516.0,대형,8MY-PA-14-3
...,...,...,...,...,...
1154411,20051104,2005,20051024.0,중형,4MY-PA-06
1154420,20051104,2005,20050905.0,중형,4MY-PA-06
1154428,20051103,2005,20051024.0,중형,4MY-PA-06
1154623,20051014,2005,20050903.0,중형,4MY-PA-06


In [95]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 )].shape

(2971, 5)

In [96]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 ), '차량연식'].max()

2011

In [97]:
# 2005년 미만 년도에서 맞지않는 배출가스인증번호 샘플 : 1
gas_temp.loc[(gas_temp['차량연식'] < 2005) & (gas_temp['배출가스인증번호'].str.len() != 9 )]

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
530992,20091013,1999,19991231.0,대형,9MY-PP-14-W38


##### [출력] 배출가스인증번호에 포함된 빈칸 처리

In [98]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [99]:
len(gas_df.columns)

46

In [100]:
# about 
gas_df.to_csv('data/[G4]등록정보&제원정보(배인번호빈칸처리)(2022-12-29).csv', index=False, encoding='cp949')

##### 오류 배출가스인증번호 유형 통계

In [101]:
# - 총 오류 샘플 수 : 21959
#     - 1번째 부분 3자리가 아닌 경우 : 
#         - 빈 값(' ')인 경우 : 
#         - 바('-') 기호가 없는 경우 : 
#         - 1번째 부분 1자리인 경우(n-XXX-XX-XX) : 
#         - 1번째 부분 2자리인 경우(nn-XXX-XX-XX) : 
#         - 1번째 부분 4자리인 경우(nnnn-XXX-XX-XX) : 
#
#     - 2번째 부분 2자리가 아닌 경우 : 
#         - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 
#         - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 
#
#     - 2번째 부분 2자리이면서 숫자만 있는 경우 : 
#
#     - 3번째 부분 2자리가 아닌 경우 : 
#         - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 
#
#     - 3번째 부분 2자리이면서 문자가 있는 경우 : 
#
#     - 4번째 부분 2자리가 아닌 경우 : 
#         - 중복 오류 : 
#     - 4번째 부분 2자리이면서 문자가 있는 경우 : 
#         - 중복 오류 : 

In [102]:
gas_df.loc[idx125[:5], '배출가스인증번호']

43     6MY-SS-0M-24
142    FMY-VTK-14-1
182    FMY-VTK-14-1
185    6MY-SS-0M-24
186    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [103]:
gas_temp2 = gas_df.loc[idx125, '배출가스인증번호']
gas_temp2.head()

43     6MY-SS-0M-24
142    FMY-VTK-14-1
182    FMY-VTK-14-1
185    6MY-SS-0M-24
186    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

###### 1번째 자리 오류 유형

In [104]:
# 1번째 부분 3자리가 아닌 경우 : 
#     - 빈 값(' ')인 경우 : 
#     - 바('-') 기호가 없는 경우 : 
#     - 1자리인 경우 : 
#     - 2자리인 경우 : 
#     - 4자리인 경우 : 

In [105]:
# 1번째 부분 3자리가 아닌 경우 : 17792
idx126 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) != 3:
            idx126.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 447735.24it/s]


17792

In [106]:
gas_df.loc[idx126, '배출가스인증번호']

1961        6MYHD1497
869698           C135
883223           C135
890260           C135
906819           C135
              ...    
1511202    TY87-배K-03
1511203    TY87-배K-03
1511204    TY87-배K-03
1511206    TY87-배K-03
1511207    TY87-배K-03
Name: 배출가스인증번호, Length: 17792, dtype: object

In [107]:
gas_df.loc[idx126[0], '배출가스인증번호']

'6MYHD1497'

In [108]:
# 빈 값(' ')인 경우 : 0
count = 0
idx126_1 = []
for i, one in enumerate(gas_df.loc[idx126, '배출가스인증번호']):
    if one == ' ':
        idx126_1.append(gas_df.loc[idx126, '배출가스인증번호'].index[i])
        count += 1
count

0

In [109]:
gas_df.loc[idx126_1, '배출가스인증번호']

Series([], Name: 배출가스인증번호, dtype: object)

In [110]:
# 바('-') 기호가 없는 경우 : 7
count = 0
idx126_2 = []
for i, one in enumerate(gas_df.loc[idx126, '배출가스인증번호']):
    if '-' not in one:
        idx126_2.append(gas_df.loc[idx126, '배출가스인증번호'].index[i])
        count += 1
count

7

In [111]:
gas_df.loc[list(set(idx126_2) - set(idx126_1)), '배출가스인증번호']

951360         C135
869698         C135
906819         C135
890260         C135
883223         C135
1961      6MYHD1497
963660         C135
Name: 배출가스인증번호, dtype: object

In [112]:
len(list(set(idx126_2) - set(idx126_1)))

7

In [113]:
# 1번째 부분 1자리인 경우(n-XXX-XX-XX) : 0
idx126_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 1:
            idx126_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 930105.04it/s]


0

In [114]:
gas_df.loc[idx126_3[:5], '배출가스인증번호']

Series([], Name: 배출가스인증번호, dtype: object)

In [115]:
set(idx126_1) == set(idx126_3)

True

In [116]:
# 1번째 부분 2자리인 경우(nn-XXX-XX-XX) : 0
idx126_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 2:
            idx126_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 799322.39it/s]


0

In [117]:
# 1번째 부분 4자리인 경우(nnnn-XXX-XX-XX) : 17791(17785)
idx126_5 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 4:
            idx126_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 451439.67it/s]


17791

In [118]:
# 4자리인 경우(앞에서 중복된 샘플 제거) : 17785
len(set(idx126_5) - set(idx126_1) - set(idx126_2))

17785

###### 2번째 자리 오류 유형

In [119]:
# 2번째 부분 2자리가 아닌 경우 : 2825
#   - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 
#   - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 

In [120]:
# 2번째 부분 2자리가 아닌 경우 : 2825
idx127 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) != 2:
            idx127.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 694009.70it/s]


2825

In [121]:
len(set(idx127) - set(idx126))

2825

In [122]:
gas_df.loc[idx127[:5], '배출가스인증번호']

142    FMY-VTK-14-1
182    FMY-VTK-14-1
186    FMY-VTK-14-1
188    FMY-VTK-14-1
191    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [123]:
# 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
idx127_1 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 1:
            idx127_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 969563.57it/s]


0

In [124]:
gas_df.iloc[idx127_1[:5], 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [125]:
# 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2825
idx127_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 3:
            idx127_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 784326.88it/s]


2825

In [126]:
gas_df.loc[idx127_3[:5], '배출가스인증번호']

142    FMY-VTK-14-1
182    FMY-VTK-14-1
186    FMY-VTK-14-1
188    FMY-VTK-14-1
191    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [127]:
# 2번째 부분 4자리인 경우(XXX-nnnn-XX-XX) : 0
idx127_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 4:
            idx127_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 898703.42it/s]


0

In [128]:
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
idx127_5 = []
p127_5 = re.compile('[0-9]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p127_5.search(str(one).split('-')[1]):
            idx127_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 474401.72it/s]


0

###### 3번째 자리 오류 유형

In [129]:
# 3번째 부분 2자리가 아닌 경우 : 
#   - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 
# 3번째 부분 2자리이면서 문자가 있는 경우 : 

In [130]:
# 3번째 부분 2자리가 아닌 경우 : 1
idx128 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[2]) != 2:
            idx128.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 936393.43it/s]


1

In [131]:
gas_df.loc[idx128, '배출가스인증번호']

1172443    NMY-JK-S-107
Name: 배출가스인증번호, dtype: object

In [132]:
len(set(idx128) - set(idx126) - set(idx127))

1

In [133]:
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1341(21951)
idx128_1 = []
p128_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[2]) == 2) and p128_1.search(str(one).split('-')[1]):
            idx128_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 301869.24it/s]


21951

In [134]:
# 3번째 부분 2자리이면서 문자가 있는 경우(앞에서 중복된 샘플 제거) : 1341
len(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))

1341

In [135]:
gas_df.loc[list(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))[:5], '배출가스인증번호']

884741    8MY-BK-0M-24
811014    8MY-HS-0M-24
765960    7MY-HC-0M-24
950283    7MY-CH-0M-24
823309    7MY-DS-0M-24
Name: 배출가스인증번호, dtype: object

###### 4번째 자리 오류 유형

In [136]:
# 4번째 부분 2자리가 아닌 경우 : 
#   - 중복 오류 : 
# 4번째 부분 2자리이면서 문자가 있는 경우 : 

In [137]:
# 4번째 부분 2자리가 아닌 경우 : 0(1999)
idx129 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[3]) != 2:
            idx129.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 707932.46it/s]


1999

In [138]:
gas_df.loc[idx129[:5], '배출가스인증번호']

142    FMY-VTK-14-1
182    FMY-VTK-14-1
186    FMY-VTK-14-1
188    FMY-VTK-14-1
191    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [139]:
# 중복 추출된 샘플 제거
len(set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

In [140]:
# 4번째 부분 2자리이면서 문자가 있는 경우 : 1(19127)
idx129_1 = []
p129_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p129_1.search(str(one).split('-')[1]):
            idx129_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 322831.02it/s]


19127

In [141]:
len(set(idx129_1) - set(idx126) - set(idx127) - set(idx128_1))

1

###### 한글이 포함된 경우

In [142]:
idx1210 = []
p1210 = re.compile('[가-힣]')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if p1210.search(one):
            idx1210.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

21959it [00:00, 474996.24it/s]


17785

In [143]:
len(set(idx1210) - set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

#### 배인번호, 제원관리번호 별 통계

In [144]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [145]:
gas_df['차종_등록정보'].unique()

array(['화물', '승용', '승합', '특수', nan], dtype=object)

In [146]:
gas_df['차종유형'].unique()

array(['대형', '중형', '소형', '경형'], dtype=object)

In [147]:
gas_df.groupby(['배출가스인증번호', '제원관리번호'])['차대번호'].count()

배출가스인증번호   제원관리번호           
3MY-FK-08  00031123015731309      1
           20018700000001         5
           20029400000000        12
           20042500000000       231
           20042500000001       202
                               ... 
XMY-KM-47  10124800030001       220
YKM-KM-12  10016400440000        54
           10016400450000         2
           10150600000000        13
           99934125020813405      1
Name: 차대번호, Length: 8420, dtype: int64

In [148]:
gas_df.shape

(1511211, 46)

In [149]:
# [출력] 배인번호별, 제원관리번호별 차량 수 통계
gas_df.groupby(['배출가스인증번호', '제원관리번호'])['차대번호'].count().to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_차량수_통계.xlsx')

##### 배인번호 미기입 차량 중 제원관리번호 매칭 유무 차량 통계 추가

In [150]:
temp1 = gas_df.groupby(['배출가스인증번호', '제원관리번호'])['차대번호'].count().reset_index()
temp1

,배출가스인증번호,제원관리번호,차대번호
0,3MY-FK-08,00031123015731309,1
1,3MY-FK-08,20018700000001,5
2,3MY-FK-08,20029400000000,12
3,3MY-FK-08,20042500000000,231
4,3MY-FK-08,20042500000001,202
...,...,...,...
8415,XMY-KM-47,10124800030001,220
8416,YKM-KM-12,10016400440000,54
8417,YKM-KM-12,10016400450000,2
8418,YKM-KM-12,10150600000000,13


##### 배인번호는 다르고, 제원관리번호는 같은 경우 확인

In [151]:
len(temp1['제원관리번호'].unique())

8196

In [152]:
tes = temp1['제원관리번호'].to_list()
no_list = []
for one in tes:
    no_list.append(tes.count(one))

In [153]:
temp1['중복제원관리번호_수'] = no_list
temp1_1 = temp1.loc[temp1['중복제원관리번호_수'] != 1]
temp1_1

,배출가스인증번호,제원관리번호,차대번호,중복제원관리번호_수
210,6MY-BK-13-08,00120002200031307,380,2
420,6MY-HD-23-55,A0810005208163308,1,2
450,6MY-HD-23-55,B0W10009300003308,14,2
451,6MY-HD-23-55,B0W10009600003308,5,2
456,6MY-HD-23-55,B1010009200003307,3,3
...,...,...,...,...
7230,PMY-HD-17,10021000360001,2,2
7235,PMY-HD-17,10021000370001,36,2
7362,RMY-HD-20,10021000300003,1,3
8000,TY87-배K-02,10016000000000,18,2


In [154]:
temp1_1['제원관리번호'] = temp1_1['제원관리번호'].astype('str')
temp1_1.sort_values('제원관리번호')

C:\Users\kbjung\AppData\Local\Temp\ipykernel_27496\194387575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1_1['제원관리번호'] = temp1_1['제원관리번호'].astype('str')


,배출가스인증번호,제원관리번호,차대번호,중복제원관리번호_수
210,6MY-BK-13-08,00120002200031307,380,2
4310,7MY-KM-14-63,00120002200031307,1,2
7145,PMY-HD-17,10014700070001,1,2
6348,KMY-HD-03,10014700070001,1,2
8000,TY87-배K-02,10016000000000,18,2
...,...,...,...,...
822,6MY-HD-23-55,BAM10000300003308,4,2
1195,7MY-CH-0M-24,BAY10000100014207,1,2
4870,7MY-SJ-0M-24,BAY10000100014207,4,2
3878,7MY-HD-24-30,BBE10000700003307,2,2


In [155]:
temp1_1.sort_values('제원관리번호').to_excel('analysis2/[G4][배출가스인증번호]중복_제원관리번호_예시.xlsx', index=False)

In [156]:
temp1_1.drop_duplicates(['배출가스인증번호', '제원관리번호']).sort_values('제원관리번호')

,배출가스인증번호,제원관리번호,차대번호,중복제원관리번호_수
210,6MY-BK-13-08,00120002200031307,380,2
4310,7MY-KM-14-63,00120002200031307,1,2
7145,PMY-HD-17,10014700070001,1,2
6348,KMY-HD-03,10014700070001,1,2
8000,TY87-배K-02,10016000000000,18,2
...,...,...,...,...
822,6MY-HD-23-55,BAM10000300003308,4,2
1195,7MY-CH-0M-24,BAY10000100014207,1,2
4870,7MY-SJ-0M-24,BAY10000100014207,4,2
3878,7MY-HD-24-30,BBE10000700003307,2,2


In [157]:
temp1_1.drop_duplicates(['배출가스인증번호', '제원관리번호']).sort_values('제원관리번호').to_excel('analysis2/[G4][배출가스인증번호]중복_제원관리번호(배인,제원 중복제거)_예시.xlsx', index=False)

##### 배인번호 없는 샘플로 통계

In [158]:
gas_df[gas_df['배출가스인증번호'].isnull() == True].shape

(123700, 46)

In [159]:
temp2 = gas_df[gas_df['배출가스인증번호'].isnull() == True].groupby(['제원관리번호'])['차대번호'].count().reset_index()
temp2

,제원관리번호,차대번호
0,00030110800000304,1
1,00030112400001628,1
2,00031105000011116,1
3,00031105000011203,1
4,00031105000011412,1
...,...,...
9195,BD410001600003309,1
9196,BD410001700003309,1
9197,BDV10000100003308,1
9198,BEU10001700013310,1


In [160]:
ttemp1_2 = temp1.merge(temp2, on='제원관리번호', how='left')
ttemp1_2

,배출가스인증번호,제원관리번호,차대번호_x,중복제원관리번호_수,차대번호_y
0,3MY-FK-08,00031123015731309,1,1,NaN
1,3MY-FK-08,20018700000001,5,1,NaN
2,3MY-FK-08,20029400000000,12,1,NaN
3,3MY-FK-08,20042500000000,231,1,NaN
4,3MY-FK-08,20042500000001,202,1,NaN
...,...,...,...,...,...
8415,XMY-KM-47,10124800030001,220,1,NaN
8416,YKM-KM-12,10016400440000,54,1,NaN
8417,YKM-KM-12,10016400450000,2,1,NaN
8418,YKM-KM-12,10150600000000,13,1,NaN


In [161]:
ttemp1_2.columns = ['배출가스인증번호', '제원관리번호', '차량_수', '중복제원관리번호_수', '배출가스인증번호_미기입_차량_수']
ttemp1_2

,배출가스인증번호,제원관리번호,차량_수,중복제원관리번호_수,배출가스인증번호_미기입_차량_수
0,3MY-FK-08,00031123015731309,1,1,NaN
1,3MY-FK-08,20018700000001,5,1,NaN
2,3MY-FK-08,20029400000000,12,1,NaN
3,3MY-FK-08,20042500000000,231,1,NaN
4,3MY-FK-08,20042500000001,202,1,NaN
...,...,...,...,...,...
8415,XMY-KM-47,10124800030001,220,1,NaN
8416,YKM-KM-12,10016400440000,54,1,NaN
8417,YKM-KM-12,10016400450000,2,1,NaN
8418,YKM-KM-12,10150600000000,13,1,NaN


In [162]:
array1 = gas_df[gas_df['배출가스인증번호'].isnull() == False]['제원관리번호'].unique()
len(array1)

8196

In [163]:
array2 = gas_df[gas_df['배출가스인증번호'].isnull() == True]['제원관리번호'].unique()
len(array2)

9200

In [164]:
set(array1) & set(array2)

{'A0410000500051207',
 'A0710001100361207',
 'A0810005207533307',
 'A0810007000311307',
 'B1R10005100003308',
 'B3C10016600003307'}

In [165]:
gas_df.loc[gas_df['제원관리번호'] == 'A0410000500051207', ['엔진형식', '차명']].shape

(2135, 2)

In [166]:
gas_df.loc[gas_df['제원관리번호'] == 'A0710001100361207', ['엔진형식', '차명']].shape

(1553, 2)

In [167]:
gas_df.loc[gas_df['제원관리번호'] == 'A0810005207533307', ['엔진형식', '차명']].shape

(31, 2)

In [168]:
gas_df.loc[gas_df['제원관리번호'] == 'A0810007000311307', ['엔진형식', '차명']].shape

(2146, 2)

In [169]:
gas_df.loc[gas_df['제원관리번호'] == 'B1R10005100003308', ['엔진형식', '차명']].shape

(2, 2)

In [170]:
gas_df.loc[gas_df['제원관리번호'] == 'B3C10016600003307', ['엔진형식', '차명']].shape

(56, 2)

###### [출력] [배출가스인증번호]제원관리번호별_차량수_통계(미기입 추가)

In [171]:
ttemp1_2.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_차량수_통계(미기입 추가).xlsx', index=False)

In [172]:
list(set(array1) & set(array2))

['A0810007000311307',
 'B3C10016600003307',
 'A0810005207533307',
 'A0710001100361207',
 'A0410000500051207',
 'B1R10005100003308']

In [173]:
# 배인번호 없는 샘플에서 제원관리번호로 매칭 가능한 샘플 제거
len( set(temp2['제원관리번호']) - (set(array1) & set(array2)) )

9194

In [174]:
# about 9s
temp2_1 = pd.DataFrame()
for one in list(set( temp2['제원관리번호']) - (set(array1) & set(array2)) ):
    temp2_1 = pd.concat([temp2_1, temp2[temp2['제원관리번호'] == one]], ignore_index=True)

In [175]:
temp2_1.shape

(9194, 2)

In [176]:
temp2_1.head()

,제원관리번호,차대번호
0,10D20002100001304,1
1,99932601002761303,1
2,10031700050000,2
3,20016200040002,1
4,99931107070611103,1


In [177]:
temp2_1.drop_duplicates(['제원관리번호']).shape

(9194, 2)

###### [출력] [배출가스인증번호]제원관리번호_미스매칭_통계
- 배인번호 없는 샘플 중 제원관리번호로도 매칭 안되는 샘플

In [178]:
temp2_1.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호_미스매칭_통계.xlsx', index=False)

##### 배인번호 통계(배인번호, 제원, 연료별)

In [179]:
gas_df.groupby(['배출가스인증번호', '제원관리번호', '연료'])['차대번호'].count()

배출가스인증번호   제원관리번호             연료         
3MY-FK-08  00031123015731309  휘발유              1
           20018700000001     휘발유              5
           20029400000000     휘발유             12
           20042500000000     휘발유            231
           20042500000001     휘발유            202
                                            ... 
XMY-KM-47  10124800030001     LGP(액화석유가스)    220
YKM-KM-12  10016400440000     LGP(액화석유가스)     54
           10016400450000     LGP(액화석유가스)      2
           10150600000000     LGP(액화석유가스)     13
           99934125020813405  LGP(액화석유가스)      1
Name: 차대번호, Length: 8420, dtype: int64

In [180]:
gas_df.groupby(['배출가스인증번호', '제원관리번호', '연료'])['차대번호'].count().to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_연료별_차량수_통계.xlsx')

In [181]:
temp3 = gas_df.groupby(['배출가스인증번호', '제원관리번호', '연료'])['차대번호'].count().reset_index()
temp3

,배출가스인증번호,제원관리번호,연료,차대번호
0,3MY-FK-08,00031123015731309,휘발유,1
1,3MY-FK-08,20018700000001,휘발유,5
2,3MY-FK-08,20029400000000,휘발유,12
3,3MY-FK-08,20042500000000,휘발유,231
4,3MY-FK-08,20042500000001,휘발유,202
...,...,...,...,...
8415,XMY-KM-47,10124800030001,LGP(액화석유가스),220
8416,YKM-KM-12,10016400440000,LGP(액화석유가스),54
8417,YKM-KM-12,10016400450000,LGP(액화석유가스),2
8418,YKM-KM-12,10150600000000,LGP(액화석유가스),13


In [182]:
temp4 = gas_df[gas_df['배출가스인증번호'].isnull() == True].groupby(['제원관리번호', '연료'])['차대번호'].count().reset_index()
temp4

,제원관리번호,연료,차대번호
0,00030110800000304,휘발유,1
1,00030112400001628,휘발유,1
2,00031105000011116,휘발유,1
3,00031105000011203,휘발유,1
4,00031105000011412,경유,1
...,...,...,...
9126,BD410001600003309,경유,1
9127,BD410001700003309,경유,1
9128,BDV10000100003308,경유,1
9129,BEU10001700013310,경유,1


In [183]:
ttemp3_4 = temp3.merge(temp4, on='제원관리번호', how='left')
ttemp3_4

,배출가스인증번호,제원관리번호,연료_x,차대번호_x,연료_y,차대번호_y
0,3MY-FK-08,00031123015731309,휘발유,1,NaN,NaN
1,3MY-FK-08,20018700000001,휘발유,5,NaN,NaN
2,3MY-FK-08,20029400000000,휘발유,12,NaN,NaN
3,3MY-FK-08,20042500000000,휘발유,231,NaN,NaN
4,3MY-FK-08,20042500000001,휘발유,202,NaN,NaN
...,...,...,...,...,...,...
8415,XMY-KM-47,10124800030001,LGP(액화석유가스),220,NaN,NaN
8416,YKM-KM-12,10016400440000,LGP(액화석유가스),54,NaN,NaN
8417,YKM-KM-12,10016400450000,LGP(액화석유가스),2,NaN,NaN
8418,YKM-KM-12,10150600000000,LGP(액화석유가스),13,NaN,NaN


In [184]:
ttemp3_4.columns = [['배출가스인증번호', '제원관리번호', '연료', '차량_수', '배인번호X_연료', '배인번호X_차량_수']]
ttemp3_4.head()

,배출가스인증번호,제원관리번호,연료,차량_수,배인번호X_연료,배인번호X_차량_수
0,3MY-FK-08,00031123015731309,휘발유,1,NaN,NaN
1,3MY-FK-08,20018700000001,휘발유,5,NaN,NaN
2,3MY-FK-08,20029400000000,휘발유,12,NaN,NaN
3,3MY-FK-08,20042500000000,휘발유,231,NaN,NaN
4,3MY-FK-08,20042500000001,휘발유,202,NaN,NaN


In [185]:
ttemp3_4.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_연료별_차량수_통계(미기입 추가).xlsx', index=False)

In [186]:
# about 9s
temp4_1 = pd.DataFrame()
for one in list( set(temp2['제원관리번호']) - (set(array1) & set(array2)) ):
    temp4_1 = pd.concat([temp4_1, temp4[temp4['제원관리번호'] == one]], ignore_index=True)

In [187]:
temp4_1.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호_미스매칭_연료별_통계.xlsx', index=False)

##### 배인번호 통계(배인번호, 제원, 연료, 차종_등록정보, 차종유형)

In [188]:
gas_df.groupby(['배출가스인증번호', '제원관리번호', '연료', '차종_등록정보', '차종유형'])['차대번호'].count()

배출가스인증번호   제원관리번호             연료           차종_등록정보  차종유형
3MY-FK-08  00031123015731309  휘발유          승용       대형        1
           20018700000001     휘발유          승용       대형        5
           20029400000000     휘발유          승용       대형       12
           20042500000000     휘발유          승용       대형      231
           20042500000001     휘발유          승용       대형      202
                                                           ... 
XMY-KM-47  10124800030001     LGP(액화석유가스)  승합       중형      129
YKM-KM-12  10016400440000     LGP(액화석유가스)  화물       경형       54
           10016400450000     LGP(액화석유가스)  화물       경형        2
           10150600000000     LGP(액화석유가스)  화물       경형       13
           99934125020813405  LGP(액화석유가스)  화물       경형        1
Name: 차대번호, Length: 8725, dtype: int64

In [189]:
gas_df.groupby(['배출가스인증번호', '제원관리번호', '연료', '차종_등록정보', '차종유형'])['차대번호'].count().to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_연료별_차종등록정보별_차종유형별_차량수_통계.xlsx')

In [190]:
temp5 = gas_df.groupby(['배출가스인증번호', '제원관리번호', '연료', '차종_등록정보', '차종유형'])['차대번호'].count().reset_index()
temp5

,배출가스인증번호,제원관리번호,연료,차종_등록정보,차종유형,차대번호
0,3MY-FK-08,00031123015731309,휘발유,승용,대형,1
1,3MY-FK-08,20018700000001,휘발유,승용,대형,5
2,3MY-FK-08,20029400000000,휘발유,승용,대형,12
3,3MY-FK-08,20042500000000,휘발유,승용,대형,231
4,3MY-FK-08,20042500000001,휘발유,승용,대형,202
...,...,...,...,...,...,...
8720,XMY-KM-47,10124800030001,LGP(액화석유가스),승합,중형,129
8721,YKM-KM-12,10016400440000,LGP(액화석유가스),화물,경형,54
8722,YKM-KM-12,10016400450000,LGP(액화석유가스),화물,경형,2
8723,YKM-KM-12,10150600000000,LGP(액화석유가스),화물,경형,13


In [191]:
temp6 = gas_df[gas_df['배출가스인증번호'].isnull() == True].groupby(['제원관리번호', '연료', '차종_등록정보', '차종유형'])['차대번호'].count().reset_index()
temp6

,제원관리번호,연료,차종_등록정보,차종유형,차대번호
0,00030110800000304,휘발유,승용,대형,1
1,00030112400001628,휘발유,승용,대형,1
2,00031105000011116,휘발유,승용,소형,1
3,00031105000011203,휘발유,승용,중형,1
4,00031105000011412,경유,승용,경형,1
...,...,...,...,...,...
9222,BD410001600003309,경유,화물,대형,1
9223,BD410001700003309,경유,화물,대형,1
9224,BDV10000100003308,경유,화물,대형,1
9225,BEU10001700013310,경유,화물,대형,1


In [192]:
ttemp5_6 = temp5.merge(temp6, on='제원관리번호', how='left')
ttemp5_6

,배출가스인증번호,제원관리번호,연료_x,차종_등록정보_x,차종유형_x,차대번호_x,연료_y,차종_등록정보_y,차종유형_y,차대번호_y
0,3MY-FK-08,00031123015731309,휘발유,승용,대형,1,NaN,NaN,NaN,NaN
1,3MY-FK-08,20018700000001,휘발유,승용,대형,5,NaN,NaN,NaN,NaN
2,3MY-FK-08,20029400000000,휘발유,승용,대형,12,NaN,NaN,NaN,NaN
3,3MY-FK-08,20042500000000,휘발유,승용,대형,231,NaN,NaN,NaN,NaN
4,3MY-FK-08,20042500000001,휘발유,승용,대형,202,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8720,XMY-KM-47,10124800030001,LGP(액화석유가스),승합,중형,129,NaN,NaN,NaN,NaN
8721,YKM-KM-12,10016400440000,LGP(액화석유가스),화물,경형,54,NaN,NaN,NaN,NaN
8722,YKM-KM-12,10016400450000,LGP(액화석유가스),화물,경형,2,NaN,NaN,NaN,NaN
8723,YKM-KM-12,10150600000000,LGP(액화석유가스),화물,경형,13,NaN,NaN,NaN,NaN


In [193]:
ttemp5_6.columns = [['배출가스인증번호', '제원관리번호', '연료', '차종_등록정보', '차종유형', '차량_수', '배인번호X_연료', '배인번호X_차종_등록정보', '배인번호X_차종유형', '배인번호X_차량_수']]
ttemp5_6.head()

,배출가스인증번호,제원관리번호,연료,차종_등록정보,차종유형,차량_수,배인번호X_연료,배인번호X_차종_등록정보,배인번호X_차종유형,배인번호X_차량_수
0,3MY-FK-08,00031123015731309,휘발유,승용,대형,1,NaN,NaN,NaN,NaN
1,3MY-FK-08,20018700000001,휘발유,승용,대형,5,NaN,NaN,NaN,NaN
2,3MY-FK-08,20029400000000,휘발유,승용,대형,12,NaN,NaN,NaN,NaN
3,3MY-FK-08,20042500000000,휘발유,승용,대형,231,NaN,NaN,NaN,NaN
4,3MY-FK-08,20042500000001,휘발유,승용,대형,202,NaN,NaN,NaN,NaN


In [194]:
ttemp5_6.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_연료별_차종별_차량수_통계(미기입 추가).xlsx', index=False)

In [195]:
# about 9s
temp6_1 = pd.DataFrame()
for one in list( set(temp2['제원관리번호']) - (set(array1) & set(array2)) ):
    temp6_1 = pd.concat([temp6_1, temp6[temp6['제원관리번호'] == one]], ignore_index=True)

In [196]:
temp6_1.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호_미스매칭_연료별_차종별_통계.xlsx', index=False)

##### 배인번호 통계(배인번호, 제원, 연료, 차종_등록정보, 차종유형, 자동차형식)

In [197]:
gas_df.groupby(['배출가스인증번호', '제원관리번호', '자동차형식', '연료', '차종_등록정보', '차종유형'])['차대번호'].count()

배출가스인증번호   제원관리번호             자동차형식             연료           차종_등록정보  차종유형
3MY-FK-08  00031123015731309  MSV3-27C          휘발유          승용       대형        1
           20018700000001     JFM 3.0 V 5 FEGX  휘발유          승용       대형        5
           20029400000000     MSV3-27B          휘발유          승용       대형       12
           20042500000000     MSV3-27C          휘발유          승용       대형      231
           20042500000001     MSV3-27C          휘발유          승용       대형      202
                                                                             ... 
XMY-KM-47  10124800030001     FC723A            LGP(액화석유가스)  승합       중형      129
YKM-KM-12  10016400440000     AM251L-HL         LGP(액화석유가스)  화물       경형       54
           10016400450000     KM251L-HL         LGP(액화석유가스)  화물       경형        2
           10150600000000     SOK0090           LGP(액화석유가스)  화물       경형       13
           99934125020813405  AM240L-1          LGP(액화석유가스)  화물       경형        1
Name: 차대번호, Length: 871

In [198]:
gas_df.groupby(['배출가스인증번호', '제원관리번호', '자동차형식', '연료', '차종_등록정보', '차종유형'])['차대번호'].count().to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_자동차형식별_연료별_차종등록정보별_차종유형별_차량수_통계.xlsx')

In [199]:
temp7 = gas_df.groupby(['배출가스인증번호', '제원관리번호', '자동차형식', '연료', '차종_등록정보', '차종유형'])['차대번호'].count().reset_index()
temp7

,배출가스인증번호,제원관리번호,자동차형식,연료,차종_등록정보,차종유형,차대번호
0,3MY-FK-08,00031123015731309,MSV3-27C,휘발유,승용,대형,1
1,3MY-FK-08,20018700000001,JFM 3.0 V 5 FEGX,휘발유,승용,대형,5
2,3MY-FK-08,20029400000000,MSV3-27B,휘발유,승용,대형,12
3,3MY-FK-08,20042500000000,MSV3-27C,휘발유,승용,대형,231
4,3MY-FK-08,20042500000001,MSV3-27C,휘발유,승용,대형,202
...,...,...,...,...,...,...,...
8710,XMY-KM-47,10124800030001,FC723A,LGP(액화석유가스),승합,중형,129
8711,YKM-KM-12,10016400440000,AM251L-HL,LGP(액화석유가스),화물,경형,54
8712,YKM-KM-12,10016400450000,KM251L-HL,LGP(액화석유가스),화물,경형,2
8713,YKM-KM-12,10150600000000,SOK0090,LGP(액화석유가스),화물,경형,13


In [200]:
temp8 = gas_df[gas_df['배출가스인증번호'].isnull() == True].groupby(['제원관리번호', '자동차형식', '연료', '차종_등록정보', '차종유형'])['차대번호'].count().reset_index()
temp8

,제원관리번호,자동차형식,연료,차종_등록정보,차종유형,차대번호
0,00030110800000304,GF-MCU15W,휘발유,승용,대형,1
1,00030112400001628,YS-25D-A1,휘발유,승용,대형,1
2,00031105000011116,XNWAY,휘발유,승용,소형,1
3,00031105000011203,202018,휘발유,승용,중형,1
4,00031105000011412,AK00F,경유,승용,경형,1
...,...,...,...,...,...,...
8763,BD410001600003309,SSC 320.12.04/8,경유,화물,대형,1
8764,BD410001700003309,SSC 210.08.03/6,경유,화물,대형,1
8765,BDV10000100003308,MTS450,경유,화물,대형,1
8766,BEU10001700013310,KCT4.5PD-BC6CT1,경유,화물,대형,1


In [201]:
ttemp7_8 = temp7.merge(temp8, on='제원관리번호', how='left')
ttemp7_8

,배출가스인증번호,제원관리번호,자동차형식_x,연료_x,차종_등록정보_x,차종유형_x,차대번호_x,자동차형식_y,연료_y,차종_등록정보_y,차종유형_y,차대번호_y
0,3MY-FK-08,00031123015731309,MSV3-27C,휘발유,승용,대형,1,NaN,NaN,NaN,NaN,NaN
1,3MY-FK-08,20018700000001,JFM 3.0 V 5 FEGX,휘발유,승용,대형,5,NaN,NaN,NaN,NaN,NaN
2,3MY-FK-08,20029400000000,MSV3-27B,휘발유,승용,대형,12,NaN,NaN,NaN,NaN,NaN
3,3MY-FK-08,20042500000000,MSV3-27C,휘발유,승용,대형,231,NaN,NaN,NaN,NaN,NaN
4,3MY-FK-08,20042500000001,MSV3-27C,휘발유,승용,대형,202,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8710,XMY-KM-47,10124800030001,FC723A,LGP(액화석유가스),승합,중형,129,NaN,NaN,NaN,NaN,NaN
8711,YKM-KM-12,10016400440000,AM251L-HL,LGP(액화석유가스),화물,경형,54,NaN,NaN,NaN,NaN,NaN
8712,YKM-KM-12,10016400450000,KM251L-HL,LGP(액화석유가스),화물,경형,2,NaN,NaN,NaN,NaN,NaN
8713,YKM-KM-12,10150600000000,SOK0090,LGP(액화석유가스),화물,경형,13,NaN,NaN,NaN,NaN,NaN


In [202]:
ttemp7_8.columns = [['배출가스인증번호', '제원관리번호', '자동차형식', '연료', '차종_등록정보', '차종유형', '차량_수', '배인번호X_자동차형식', '배인번호X_연료', '배인번호X_차종_등록정보', '배인번호X_차종유형', '배인번호X_차량_수']]
ttemp7_8.head()

,배출가스인증번호,제원관리번호,자동차형식,연료,차종_등록정보,차종유형,차량_수,배인번호X_자동차형식,배인번호X_연료,배인번호X_차종_등록정보,배인번호X_차종유형,배인번호X_차량_수
0,3MY-FK-08,00031123015731309,MSV3-27C,휘발유,승용,대형,1,NaN,NaN,NaN,NaN,NaN
1,3MY-FK-08,20018700000001,JFM 3.0 V 5 FEGX,휘발유,승용,대형,5,NaN,NaN,NaN,NaN,NaN
2,3MY-FK-08,20029400000000,MSV3-27B,휘발유,승용,대형,12,NaN,NaN,NaN,NaN,NaN
3,3MY-FK-08,20042500000000,MSV3-27C,휘발유,승용,대형,231,NaN,NaN,NaN,NaN,NaN
4,3MY-FK-08,20042500000001,MSV3-27C,휘발유,승용,대형,202,NaN,NaN,NaN,NaN,NaN


In [203]:
ttemp7_8.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호별_자동차형식별_연료별_차종별_차량수_통계(미기입 추가).xlsx', index=False)

In [204]:
# about 9s
temp8_1 = pd.DataFrame()
for one in list( set(temp2['제원관리번호']) - (set(array1) & set(array2)) ):
    temp8_1 = pd.concat([temp8_1, temp8[temp8['제원관리번호'] == one]], ignore_index=True)

In [205]:
temp8_1.to_excel('analysis2/[G4][배출가스인증번호]제원관리번호_미스매칭_자동차형식별_연료별_차종별_통계.xlsx', index=False)

#### 배출가스인증번호 미부여 현황
- 필요한 컬럼 : 배출가스인증번호, 차량연식, 차량제작일자, 차명, 차종_제원정보, 차종분류, 차종유형, 자동차형식, 제작사명, 연료

In [206]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [207]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['배출가스인증번호', '차량연식', '차량제작일자', '차명', '차종_제원정보', '차종분류', '차종유형', '제작사명', '연료']].head()

,배출가스인증번호,차량연식,차량제작일자,차명,차종_제원정보,차종분류,차종유형,제작사명,연료
0,NaN,2008,20071224.0,메가트럭,화물,일반형-카고,대형,현대자동차(주),경유
1,NaN,2008,20080924.0,메가트럭군물자수송,화물,특수용도형-특수용도형,대형,현대자동차(주),경유
2,NaN,2008,20080605.0,신정4㎥급진공노면총소차,화물,특수용도형-특수용도형,대형,신정개발특장차 주식회사,경유
4,NaN,2008,20080221.0,윈스톰2.0S디젤,승용,다목적,중형,한국지엠주식회사,경유
18,NaN,2006,20060608.0,대우9.5톤장축카고트럭,화물,일반형-카고,대형,타타대우상용차(주),경유


In [208]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [209]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True].shape

(123700, 46)

In [210]:
null_num_df = gas_df.loc[gas_df['배출가스인증번호'].isnull() == True]

In [211]:
null_num_df.shape

(123700, 46)

In [213]:
gas_df.shape

(1511211, 46)

In [212]:
gas_df[gas_df['차량말소YN'] == 'N'].shape

(1511211, 46)

##### [출력] 배인번호 미부여 샘플

In [212]:
# [출력]4등급 차량 배출가스인증번호 미부여 샘플
null_num_df.to_csv('analysis2/[G4][배출가스인증번호]미부여_샘플.csv', index=False, encoding='cp949')

In [216]:
null_num_df[['엔진형식', '차명', '차량연식','차대번호']].head()

,엔진형식,차명,차량연식,차대번호
0,D6GA,메가트럭,2008,KMFLA18KP8C028585
1,D6GA,메가트럭군물자수송,2008,KMCLK18KP8C035195
2,D6GA,신정4㎥급진공노면총소차,2008,KMFLA18KP8C031274
4,Z20S,윈스톰2.0S디젤,2008,KLACA26RD8B226753
18,DL08,대우9.5톤장축카고트럭,2006,KL3F7CJF56K000010


In [217]:
null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False)

엔진형식          차명               
B3            프라이드                 9870
F8C           티코(TICO)             9402
A15MF         에스페로1.5MT            4916
C18LE         프린스1.8AT             4576
              프린스1.8MT             4256
                                   ... 
62N           포르쉐911CARRERA           1
62P           PORSCHE CARRERA 2       1
63            PORSCHE 911             1
63T           포르쉐911카레라               1
vg30-858076a  닛산세드릭                   1
Name: 차대번호, Length: 3579, dtype: int64

In [218]:
null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False).to_excel('analysis2/[G4][배출가스인증번호]미부여샘플_통계.xlsx')

##### 배출가스인증번호 미부여 데이터 확인

In [219]:
# 배출가스인증번호 미부여 데이터 중
# - 동일 엔진형식 중 1천대 이상의 샘플만 배출가스인증번호 유무 확인

In [220]:
miss_gas_num_df = null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False).reset_index()
miss_gas_num_df

,엔진형식,차명,차대번호
0,B3,프라이드,9870
1,F8C,티코(TICO),9402
2,A15MF,에스페로1.5MT,4916
3,C18LE,프린스1.8AT,4576
4,C18LE,프린스1.8MT,4256
...,...,...,...
3574,62N,포르쉐911CARRERA,1
3575,62P,PORSCHE CARRERA 2,1
3576,63,PORSCHE 911,1
3577,63T,포르쉐911카레라,1


In [221]:
temp = miss_gas_num_df[miss_gas_num_df['차대번호'] >= 1000]
temp

,엔진형식,차명,차대번호
0,B3,프라이드,9870
1,F8C,티코(TICO),9402
2,A15MF,에스페로1.5MT,4916
3,C18LE,프린스1.8AT,4576
4,C18LE,프린스1.8MT,4256
5,F8C,티코(TICO)오토,2699
6,G15SF,르망,2570
7,C20LE,수퍼살롱2.0AT,2295
8,A15DMS,누비라1.5,2081
9,G4AJ,프레스토,2068


In [222]:
# 제원정보에서 해당 엔진형식을 통해 배출가스인증번호 확인 여부 확인
count = 0
for one, two in zip(temp['엔진형식'], temp['차명']):
    if gas_df.loc[ (gas_df['엔진형식'] == one) & (gas_df['차명'] == two), ['엔진형식' ,'배출가스인증번호', '차명'] ].drop_duplicates(['배출가스인증번호', '차명'], keep='first').dropna(subset=['배출가스인증번호']).shape[0] != 0:
        print(one)
        print( gas_df.loc[ (gas_df['엔진형식'] == one) & (gas_df['차명'] == two), ['엔진형식' ,'배출가스인증번호', '차명'] ].drop_duplicates(['배출가스인증번호', '차명'], keep='first').dropna(subset=['배출가스인증번호']))
        print()
        count += 1 

B3
        엔진형식    배출가스인증번호    차명
1463278   B3  TY87-배K-03  프라이드

G15SF
          엔진형식    배출가스인증번호  차명
1332095  G15SF  TY87-배K-02  르망

G4CS
         엔진형식   배출가스인증번호      차명
1207761  G4CS  LMY-HD-08  그랜저2.4

B3
        엔진형식   배출가스인증번호   차명
1208778   B3  PMY-KM-20  아벨라

CD800
          엔진형식   배출가스인증번호   차명
1247396  CD800  SMY-KM-04  타우너



In [223]:
count

5

In [224]:
gas_df.loc[gas_df['엔진형식'] == 'B3', ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명'])

,엔진형식,배출가스인증번호,차명
2684,B3,NaN,프라이드베타
210301,B3,PMY-KM-20,KIA AVELLA
1154949,B3,NaN,프라이드
1156504,B3,NaN,프라이드베타오토
1156509,B3,NaN,kia Pride
1156570,B3,NaN,기아 프라이드
1156901,B3,NaN,프라이드오토
1156912,B3,NaN,아벨라
1157111,B3,NaN,KIA PRIDE
1157331,B3,NaN,기아 B3


In [225]:
gas_df.loc[gas_df['엔진형식'] == 'B3', ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명']).dropna(subset=['배출가스인증번호'])

,엔진형식,배출가스인증번호,차명
210301,B3,PMY-KM-20,KIA AVELLA
1186223,B3,PMY-KM-20,델타
1208778,B3,PMY-KM-20,아벨라
1211488,B3,PMY-KM-20,아벨라운전교습차
1212189,B3,PMY-KM-20,아벨라델타운전교습차
1288021,B3,TY87-배K-03,프라이드베타오토
1322367,B3,PMY-KM-20,AVELLA
1355916,B3,RMY-KM-25,프라이드베타
1358813,B3,TY87-배K-03,프라이드베타
1463278,B3,TY87-배K-03,프라이드


In [226]:
# 제원정보에서 해당 엔진형식을 통해 배출가스인증번호 확인 여부 확인
t_temp = pd.DataFrame()
for one in temp['엔진형식']:
    temp = gas_df.loc[gas_df['엔진형식'] == one, ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명']).dropna(subset=['배출가스인증번호'])
    if temp.shape[0] != 0:
        t_temp = pd.concat([t_temp, temp], ignore_index=True)

In [227]:
t_temp

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,PMY-KM-20,델타
2,B3,PMY-KM-20,아벨라
3,B3,PMY-KM-20,아벨라운전교습차
4,B3,PMY-KM-20,아벨라델타운전교습차
5,B3,TY87-배K-03,프라이드베타오토
6,B3,PMY-KM-20,AVELLA
7,B3,RMY-KM-25,프라이드베타
8,B3,TY87-배K-03,프라이드베타
9,B3,TY87-배K-03,프라이드


In [228]:
t_temp.drop_duplicates(['엔진형식', '배출가스인증번호', '차명']).shape

(29, 3)

In [229]:
t_temp = t_temp.drop_duplicates(['엔진형식', '배출가스인증번호', '차명'])
t_temp

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,PMY-KM-20,델타
2,B3,PMY-KM-20,아벨라
3,B3,PMY-KM-20,아벨라운전교습차
4,B3,PMY-KM-20,아벨라델타운전교습차
5,B3,TY87-배K-03,프라이드베타오토
6,B3,PMY-KM-20,AVELLA
7,B3,RMY-KM-25,프라이드베타
8,B3,TY87-배K-03,프라이드베타
9,B3,TY87-배K-03,프라이드


###### [출력] 배인번호 미부여 데이터 중 처리가능 샘플

In [230]:
# 미부여 데이터 처리 가능 샘플 추출(엔진형식으로 매칭)
t_temp.to_excel('analysis2/[G4][배출가스인증번호]미부여_처리용_샘플.xlsx', index=False)

##### 배인번호별, 제원관리번호별 통계

In [231]:
null_num_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [232]:
t_temp2 = t_temp.drop_duplicates(['엔진형식', '배출가스인증번호'], keep='first').reset_index(drop=True)
t_temp2

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,TY87-배K-03,프라이드베타오토
2,B3,RMY-KM-25,프라이드베타
3,G15SF,TY87-배K-02,르망
4,G4AJ,KMY-HD-04,엑셀오토메틱
5,G4AJ,KMY-HD-02,엑셀
6,G4AJ,KMY-HD-01,엑셀
7,G4CS,TMY-HD-26,스타렉스9인승오토매틱
8,G4CS,LMY-HD-08,그랜저2.4
9,G4CS,KMY-HD-03,쏘나타 오토매틱


In [233]:
null_num_df.shape

(123700, 46)

In [234]:
null_num_df2 = null_num_df.merge(t_temp2, on=['배출가스인증번호', '엔진형식'], how='left')
null_num_df2.shape

(123700, 47)

In [235]:
null_num_df2.groupby(['배출가스인증번호', '제원관리번호']).count()

,,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,승인일자,구동형식,변속기종류,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN,차명_y
배출가스인증번호,제원관리번호,,,,,,,,,,,,,,,,,,,,,


##### 제원관리번호 이용 미부여 배인 번호 부여

In [236]:
gas_df['제원관리번호'].isnull().sum()

0

In [237]:
len(gas_df['제원관리번호'].unique())

17390

In [238]:
temp = gas_df[['차대번호', '제원관리번호', '배출가스인증번호', '엔진형식', '차명']].drop_duplicates(['제원관리번호', '배출가스인증번호', '엔진형식', '차명'], keep='first').sort_values(['제원관리번호', '배출가스인증번호', '엔진형식', '차명'])
temp

,차대번호,제원관리번호,배출가스인증번호,엔진형식,차명
1157057,KL90E3ACGXSZ00001,00030110800000304,NaN,5S,도요다 해리어
1157326,KMHWF35V41A387514,00030112400001628,NaN,G6BV,쏘나타2.5GOLD
2285,KL90B2M9GWSZ00001,00031105000011116,NaN,12A,ROVER MINI
1157423,KL90A3AAGVSZ00002,00031105000011203,NaN,111921,벤츠C180
3147,KNAMC76439S281271,00031105000011314,7MY-KM-14-64,J3,카니발
...,...,...,...,...,...
214,YV2XT40K5JA834376,BRX20004600013317,FMY-VTK-14-1,D13,FM 카고
908,YV2V001D2HZ114313,BRX20004700003317,FMY-VTK-14-3,D8,FE 카고
905,YV2V001D3HZ114286,BRX20004800003317,FMY-VTK-14-3,D8,FE 윙바디
595,YV2XT40K9JA819380,BZB20003100003318,FMY-VTK-14-1,D13,삼부26톤볼보트럭


###### [출력] 동일 제원관리번호 차량 샘플

In [239]:
temp.to_excel('analysis2/[G4][제원관리번호]동일차량확인.xlsx')

In [240]:
temp = temp.reset_index()
temp

,index,차대번호,제원관리번호,배출가스인증번호,엔진형식,차명
0,1157057,KL90E3ACGXSZ00001,00030110800000304,NaN,5S,도요다 해리어
1,1157326,KMHWF35V41A387514,00030112400001628,NaN,G6BV,쏘나타2.5GOLD
2,2285,KL90B2M9GWSZ00001,00031105000011116,NaN,12A,ROVER MINI
3,1157423,KL90A3AAGVSZ00002,00031105000011203,NaN,111921,벤츠C180
4,3147,KNAMC76439S281271,00031105000011314,7MY-KM-14-64,J3,카니발
...,...,...,...,...,...,...
17615,214,YV2XT40K5JA834376,BRX20004600013317,FMY-VTK-14-1,D13,FM 카고
17616,908,YV2V001D2HZ114313,BRX20004700003317,FMY-VTK-14-3,D8,FE 카고
17617,905,YV2V001D3HZ114286,BRX20004800003317,FMY-VTK-14-3,D8,FE 윙바디
17618,595,YV2XT40K9JA819380,BZB20003100003318,FMY-VTK-14-1,D13,삼부26톤볼보트럭


In [241]:
no_list = []
m_no = temp['제원관리번호'].to_list()
for one in tqdm(temp['제원관리번호']):
    n = m_no.count(one)
    no_list.append(n)

100%|██████████| 17620/17620 [00:03<00:00, 4609.44it/s]


In [242]:
m_no[:10]

['00030110800000304',
 '00030112400001628',
 '00031105000011116',
 '00031105000011203',
 '00031105000011314',
 '00031105000011412',
 '00031105000021116',
 '00031105000031116',
 '00031105000051211',
 '00031105000061214']

In [243]:
m_no.count('00030110800000304')

1

In [244]:
no_list.index(2)

1489

In [245]:
no_list.index(3)

3352

In [246]:
m_no[3352]

'10021000040002'

In [247]:
no_list.index(4)

16669

In [248]:
m_no[16669]

'B3C10016600003307'

In [249]:
no_list.index(5)

3602

In [250]:
temp['동일제원관리번호_개수'] = no_list

In [251]:
temp.groupby(['동일제원관리번호_개수'])['제원관리번호'].count()

동일제원관리번호_개수
1    17173
2      414
3       24
4        4
5        5
Name: 제원관리번호, dtype: int64

In [252]:
temp.loc[temp['동일제원관리번호_개수'] == 2, '제원관리번호'].head()

1489    00120002200031307
1490    00120002200031307
2457       10014700070001
2458       10014700070001
2891       10016000000000
Name: 제원관리번호, dtype: object

In [253]:
gas_df.loc[(gas_df['제원관리번호'] == '00120002200031307') & (gas_df['배출가스인증번호'].isnull() == True), '차대번호']

Series([], Name: 차대번호, dtype: object)

In [254]:
len(gas_df.loc[(gas_df['제원관리번호'] == '00120002200031307') & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

0

In [255]:
count2 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 2, '제원관리번호'].unique()
for one in tqdm(tes):
    count2 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 207/207 [00:27<00:00,  7.46it/s]


In [256]:
count2

5

In [257]:
count3 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 3, '제원관리번호'].unique()
for one in tqdm(tes):
    count3 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 8/8 [00:01<00:00,  7.15it/s]


In [258]:
count3

0

In [259]:
count4 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 4, '제원관리번호'].unique()
for one in tqdm(tes):
    count4 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 1/1 [00:00<00:00,  7.16it/s]


In [260]:
count4

1

In [261]:
count5 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 5, '제원관리번호'].unique()
for one in tqdm(tes):
    count5 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 1/1 [00:00<00:00,  7.34it/s]


In [262]:
count5

0

###### 동일 제원관리번호로 기입 할 수 있는 샘플

In [263]:
# 동일 제원관리번호로 기입 할 수 있는 제원관리번호 샘플 수 : 6
count2 + count3 + count4 + count5

6

### 배출가스인증번호별 통계

In [264]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [265]:
gas_df.shape

(1511211, 46)

In [266]:
len(gas_df['차대번호'].unique())

1511211

In [267]:
gas_df.loc[gas_df.duplicated('차대번호', keep=False) == True, ['차량관리번호', '차량등록번호', '차대번호', '배출가스인증번호']].sort_values('차대번호')

,차량관리번호,차량등록번호,차대번호,배출가스인증번호


In [268]:
gas_df.groupby('배출가스인증번호')['차대번호'].count()

배출가스인증번호
3MY-FK-08       1064
4MY-AD-20         34
4MY-DC-33          7
4MY-PA-06         13
5MY-AD-13-06     378
                ... 
XMY-HD-47        887
XMY-HD-48        354
XMY-HD-55       2469
XMY-KM-47       3098
YKM-KM-12         70
Name: 차대번호, Length: 302, dtype: int64

In [269]:
gas_df['배출가스인증번호'].isnull().sum()

123700

#### [출력] 배인번호별 통계

In [270]:
gas_df.groupby('배출가스인증번호')['차대번호'].count().to_excel('analysis2/[G4][배출가스인증번호]배인번호별_통계.xlsx')

### 총중량으로 분류 현황

In [271]:
gas_df.shape

(1511211, 46)

In [272]:
gas_df['총중량'].unique()

array([10005., 10825., 11075., ...,  1512.,  1197.,  1436.])

In [273]:
type(gas_df['총중량'][0])

numpy.float64

In [274]:
gas_df['총중량'].isnull().sum()

25

In [275]:
gas_df['총중량'] = gas_df['총중량'].fillna(-1)

In [276]:
gas_df['총중량'].isnull().sum()

0

In [277]:
weight_list = []
for i, one in tqdm(enumerate(gas_df['총중량'])):
    if one == -1:
        weight_list.append('미기입')
    elif one < 1500:
        weight_list.append('1.5ton 미만')
    elif one >= 1500 and one < 2000:
        weight_list.append('1.5ton 이상 2.0ton 미만')
    elif one >= 2000 and one < 2500:
        weight_list.append('2.0ton 이상 2.5ton 미만')
    elif one >= 2500 and one < 3000:
        weight_list.append('2.5ton 이상 3.0ton 미만')
    elif one >= 3000 and one < 3500:
        weight_list.append('3.0ton 이상 3.5ton 미만')
    elif one >= 3500 and one < 4000:
        weight_list.append('3.5ton 이상 4.0ton 미만')
    elif one >= 4000 and one < 4500:
        weight_list.append('4.0ton 이상 4.5ton 미만')
    elif one >= 4500 and one < 5000:
        weight_list.append('4.5ton 이상 5.0ton 미만')
    elif one >= 5000 and one < 5500:
        weight_list.append('5.0ton 이상 5.5ton 미만')
    elif one >= 5500 and one < 6000:
        weight_list.append('5.5ton 이상 6.0ton 미만')
    elif one >= 6000 and one < 8000:
        weight_list.append('6.0ton 이상 8.0ton 미만')
    elif one >= 8000 and one < 10000:
        weight_list.append('8.0ton 이상 10.0ton 미만')
    elif one >= 10000:
        weight_list.append('10.0ton 이상')

1511211it [00:01, 1026519.11it/s]


In [278]:
len(weight_list)

1511211

In [279]:
gas_df['총중량_분류'] = weight_list

In [280]:
total_weight_df = gas_df.groupby('총중량_분류')['차대번호'].count().reset_index()
total_weight_df

,총중량_분류,차대번호
0,1.5ton 미만,197325
1,1.5ton 이상 2.0ton 미만,325380
2,10.0ton 이상,30310
3,2.0ton 이상 2.5ton 미만,236758
4,2.5ton 이상 3.0ton 미만,432450
5,3.0ton 이상 3.5ton 미만,259727
6,3.5ton 이상 4.0ton 미만,134
7,4.0ton 이상 4.5ton 미만,192
8,4.5ton 이상 5.0ton 미만,23
9,5.0ton 이상 5.5ton 미만,883


#### [출력] [총중량] 분류

In [281]:
total_weight_df.to_excel('analysis2/[G4][총중량]분류_통계.xlsx')

#### 총중량 code 화

In [282]:
weight_code_list = []
for i, one in tqdm(enumerate(gas_df['총중량'])):
    if one < 250:
        weight_code_list.append('0025')
    elif one >= 250 and one < 500:
        weight_code_list.append('0050')
    elif one >= 500 and one < 750:
        weight_code_list.append('0075')
    elif one >= 750 and one < 1000:
        weight_code_list.append('0100')
    elif one >= 1000 and one < 1250:
        weight_code_list.append('0125')
    elif one >= 1250 and one < 1500:
        weight_code_list.append('0150')
    elif one >= 1500 and one < 1750:
        weight_code_list.append('0175')
    elif one >= 1750 and one < 2000:
        weight_code_list.append('0200')
    elif one >= 2000 and one < 2250:
        weight_code_list.append('0225')
    elif one >= 2250 and one < 2500:
        weight_code_list.append('0250')
    elif one >= 2500 and one < 2750:
        weight_code_list.append('0275')
    elif one >= 2750 and one < 3000:
        weight_code_list.append('0300')
    elif one >= 3000 and one < 3250:
        weight_code_list.append('0325')
    elif one >= 3250 and one < 3500:
        weight_code_list.append('0350')
    elif one >= 3500 and one < 3750:
        weight_code_list.append('0375')
    elif one >= 3750 and one < 4000:
        weight_code_list.append('0400')
    elif one >= 4000 and one < 4250:
        weight_code_list.append('0425')
    elif one >= 4250 and one < 4500:
        weight_code_list.append('0450')
    elif one >= 4500 and one < 4750:
        weight_code_list.append('0475')
    elif one >= 4750 and one < 5000:
        weight_code_list.append('0500')
    elif one >= 5000 and one < 5250:
        weight_code_list.append('0525')
    elif one >= 5250 and one < 5500:
        weight_code_list.append('0550')
    elif one >= 5500 and one < 5750:
        weight_code_list.append('0575')
    elif one >= 5750 and one < 6000:
        weight_code_list.append('0600')
    elif one >= 6000 and one < 6250:
        weight_code_list.append('0625')
    elif one >= 6250 and one < 6500:
        weight_code_list.append('0650')
    elif one >= 6500 and one < 6750:
        weight_code_list.append('0675')
    elif one >= 6750 and one < 7000:
        weight_code_list.append('0700')
    elif one >= 7000 and one < 7250:
        weight_code_list.append('0725')
    elif one >= 7250 and one < 7500:
        weight_code_list.append('0750')
    elif one >= 7500 and one < 7750:
        weight_code_list.append('0775')
    elif one >= 7750 and one < 8000:
        weight_code_list.append('0800')
    elif one >= 8000 and one < 8250:
        weight_code_list.append('0825')
    elif one >= 8250 and one < 8500:
        weight_code_list.append('0850')
    elif one >= 8500 and one < 8750:
        weight_code_list.append('0875')
    elif one >= 8750 and one < 9000:
        weight_code_list.append('0900')
    elif one >= 9000 and one < 9250:
        weight_code_list.append('0925')
    elif one >= 9250 and one < 9500:
        weight_code_list.append('0950')
    elif one >= 9500 and one < 9750:
        weight_code_list.append('0975')
    elif one >= 9750 and one < 10000:
        weight_code_list.append('1000')
    elif one >= 10000 and one < 10250:
        weight_code_list.append('1025')
    elif one >= 10250 and one < 10500:
        weight_code_list.append('1050')
    elif one >= 10500 and one < 10750:
        weight_code_list.append('1075')
    elif one >= 10750 and one < 11000:
        weight_code_list.append('1100')
    elif one >= 11000 and one < 11250:
        weight_code_list.append('1125')
    elif one >= 11250 and one < 11500:
        weight_code_list.append('1150')
    elif one >= 11500 and one < 11750:
        weight_code_list.append('1175')
    elif one >= 11750 and one < 12000:
        weight_code_list.append('1200')
    elif one >= 12000 and one < 12250:
        weight_code_list.append('1225')
    elif one >= 12250 and one < 12500:
        weight_code_list.append('1250')
    elif one >= 12500 and one < 12750:
        weight_code_list.append('1275')
    elif one >= 12750 and one < 13000:
        weight_code_list.append('1300')
    elif one >= 13000 and one < 13250:
        weight_code_list.append('1325')
    elif one >= 13250 and one < 13500:
        weight_code_list.append('1350')
    elif one >= 13500 and one < 13750:
        weight_code_list.append('1375')
    elif one >= 13750 and one < 14000:
        weight_code_list.append('1400')
    elif one >= 14000 and one < 14250:
        weight_code_list.append('1425')
    elif one >= 14250 and one < 14500:
        weight_code_list.append('1450')
    elif one >= 14500 and one < 14750:
        weight_code_list.append('1475')
    elif one >= 14750 and one < 15000:
        weight_code_list.append('1500')
    elif one >= 15000 and one < 15250:
        weight_code_list.append('1525')
    elif one >= 15250 and one < 15500:
        weight_code_list.append('1550')
    elif one >= 15500 and one < 15750:
        weight_code_list.append('1575')
    elif one >= 15750 and one < 16000:
        weight_code_list.append('1600')
    elif one >= 16000 and one < 16250:
        weight_code_list.append('1625')
    elif one >= 16250 and one < 16500:
        weight_code_list.append('1650')
    elif one >= 16500 and one < 16750:
        weight_code_list.append('1675')
    elif one >= 16750 and one < 17000:
        weight_code_list.append('1700')
    elif one >= 17000 and one < 17250:
        weight_code_list.append('1725')
    elif one >= 17250 and one < 17500:
        weight_code_list.append('1750')
    elif one >= 17500 and one < 17750:
        weight_code_list.append('1775')
    elif one >= 17750 and one < 18000:
        weight_code_list.append('1800')
    elif one >= 18000 and one < 18250:
        weight_code_list.append('1825')
    elif one >= 18250 and one < 18500:
        weight_code_list.append('1850')
    elif one >= 18500 and one < 18750:
        weight_code_list.append('1875')
    elif one >= 18750 and one < 19000:
        weight_code_list.append('1900')
    elif one >= 19000 and one < 19250:
        weight_code_list.append('1925')
    elif one >= 19250 and one < 19500:
        weight_code_list.append('1950')
    elif one >= 19500 and one < 19750:
        weight_code_list.append('1975')
    elif one >= 19750 and one < 20000:
        weight_code_list.append('2000')
    elif one >= 20000 and one < 20250:
        weight_code_list.append('2025')
    elif one >= 20250 and one < 20500:
        weight_code_list.append('2050')
    elif one >= 20500 and one < 20750:
        weight_code_list.append('2075')
    elif one >= 20750 and one < 21000:
        weight_code_list.append('2100')
    elif one >= 21000 and one < 21250:
        weight_code_list.append('2125')
    elif one >= 21250 and one < 21500:
        weight_code_list.append('2150')
    elif one >= 21500 and one < 21750:
        weight_code_list.append('2175')
    elif one >= 21750 and one < 22000:
        weight_code_list.append('2200')
    elif one >= 22000 and one < 22250:
        weight_code_list.append('2225')
    elif one >= 22250 and one < 22500:
        weight_code_list.append('2250')
    elif one >= 22500 and one < 22750:
        weight_code_list.append('2275')
    elif one >= 22750 and one < 23000:
        weight_code_list.append('2300')
    elif one >= 23000 and one < 23250:
        weight_code_list.append('2325')
    elif one >= 23250 and one < 23500:
        weight_code_list.append('2350')
    elif one >= 23500 and one < 23750:
        weight_code_list.append('2375')
    elif one >= 23750 and one < 24000:
        weight_code_list.append('2400')
    elif one >= 24000 and one < 24250:
        weight_code_list.append('2425')
    elif one >= 24250 and one < 24500:
        weight_code_list.append('2450')
    elif one >= 24500 and one < 24750:
        weight_code_list.append('2475')
    elif one >= 24750 and one < 25000:
        weight_code_list.append('2500')
    elif one >= 25000 and one < 25250:
        weight_code_list.append('2525')
    elif one >= 25250 and one < 25500:
        weight_code_list.append('2550')
    elif one >= 25500 and one < 25750:
        weight_code_list.append('2575')
    elif one >= 25750 and one < 26000:
        weight_code_list.append('2600')
    elif one >= 26000 and one < 26250:
        weight_code_list.append('2625')
    elif one >= 26250 and one < 26500:
        weight_code_list.append('2650')
    elif one >= 26500 and one < 26750:
        weight_code_list.append('2675')
    elif one >= 26750 and one < 27000:
        weight_code_list.append('2700')
    elif one >= 27000 and one < 27250:
        weight_code_list.append('2725')
    elif one >= 27250 and one < 27500:
        weight_code_list.append('2750')
    elif one >= 27500 and one < 27750:
        weight_code_list.append('2775')
    elif one >= 27750 and one < 28000:
        weight_code_list.append('2800')
    elif one >= 28000 and one < 28250:
        weight_code_list.append('2825')
    elif one >= 28250 and one < 28500:
        weight_code_list.append('2850')
    elif one >= 28500 and one < 28750:
        weight_code_list.append('2875')
    elif one >= 28750 and one < 29000:
        weight_code_list.append('2900')
    elif one >= 29000 and one < 29250:
        weight_code_list.append('2925')
    elif one >= 29250 and one < 29500:
        weight_code_list.append('2950')
    elif one >= 29500 and one < 29750:
        weight_code_list.append('2975')
    elif one >= 29750 and one < 30000:
        weight_code_list.append('3000')
    elif one >= 30000 and one < 30250:
        weight_code_list.append('3025')
    elif one >= 30250 and one < 30500:
        weight_code_list.append('3050')
    elif one >= 30500 and one < 30750:
        weight_code_list.append('3075')
    elif one >= 30750 and one < 31000:
        weight_code_list.append('3100')
    elif one >= 31000 and one < 31250:
        weight_code_list.append('3125')
    elif one >= 31250 and one < 31500:
        weight_code_list.append('3150')
    elif one >= 31500 and one < 31750:
        weight_code_list.append('3175')
    elif one >= 31750 and one < 32000:
        weight_code_list.append('3200')
    elif one >= 32000 and one < 32250:
        weight_code_list.append('3225')
    elif one >= 32250 and one < 32500:
        weight_code_list.append('3250')
    elif one >= 32500 and one < 32750:
        weight_code_list.append('3275')
    elif one >= 32750 and one < 33000:
        weight_code_list.append('3300')
    elif one >= 33000 and one < 33250:
        weight_code_list.append('3325')
    elif one >= 33250 and one < 33500:
        weight_code_list.append('3350')
    elif one >= 33500 and one < 33750:
        weight_code_list.append('3375')
    elif one >= 33750 and one < 34000:
        weight_code_list.append('3400')
    elif one >= 34000 and one < 34250:
        weight_code_list.append('3425')
    elif one >= 34250 and one < 34500:
        weight_code_list.append('3450')
    elif one >= 34500 and one < 34750:
        weight_code_list.append('3475')
    elif one >= 34750 and one < 35000:
        weight_code_list.append('3500')
    elif one >= 35000 and one < 35250:
        weight_code_list.append('3525')
    elif one >= 35250 and one < 35500:
        weight_code_list.append('3550')
    elif one >= 35500 and one < 35750:
        weight_code_list.append('3575')
    elif one >= 35750 and one < 36000:
        weight_code_list.append('3600')
    elif one >= 36000 and one < 36250:
        weight_code_list.append('3625')
    elif one >= 36250 and one < 36500:
        weight_code_list.append('3650')
    elif one >= 36500 and one < 36750:
        weight_code_list.append('3675')
    elif one >= 36750 and one < 37000:
        weight_code_list.append('3700')
    elif one >= 37000 and one < 37250:
        weight_code_list.append('3725')
    elif one >= 37250 and one < 37500:
        weight_code_list.append('3750')
    elif one >= 37500 and one < 37750:
        weight_code_list.append('3775')
    elif one >= 37750 and one < 38000:
        weight_code_list.append('3800')
    elif one >= 38000 and one < 38250:
        weight_code_list.append('3825')
    elif one >= 38250 and one < 38500:
        weight_code_list.append('3850')
    elif one >= 38500 and one < 38750:
        weight_code_list.append('3875')
    elif one >= 38750 and one < 39000:
        weight_code_list.append('3900')
    elif one >= 39000 and one < 39250:
        weight_code_list.append('3925')
    elif one >= 39250 and one < 39500:
        weight_code_list.append('3950')
    elif one >= 39500 and one < 39750:
        weight_code_list.append('3975')
    elif one >= 39750 and one < 40000:
        weight_code_list.append('4000')
    elif one >= 40000 and one < 40250:
        weight_code_list.append('4025')
    elif one >= 40250 and one < 40500:
        weight_code_list.append('4050')
    elif one >= 40500 and one < 40750:
        weight_code_list.append('4075')
    elif one >= 40750 and one < 41000:
        weight_code_list.append('4100')
    elif one >= 41000 and one < 41250:
        weight_code_list.append('4125')
    elif one >= 41250 and one < 41500:
        weight_code_list.append('4050')
    elif one >= 41500 and one < 41750:
        weight_code_list.append('4175')
    elif one >= 41750 and one < 42000:
        weight_code_list.append('4200')
    elif one >= 42000 and one < 42250:
        weight_code_list.append('4225')
    elif one >= 42250 and one < 42500:
        weight_code_list.append('4250')
    elif one >= 42500 and one < 42750:
        weight_code_list.append('4275')
    elif one >= 42750 and one < 43000:
        weight_code_list.append('4300')
    elif one >= 43000 and one < 43250:
        weight_code_list.append('4325')
    elif one >= 43250 and one < 43500:
        weight_code_list.append('4350')
    elif one >= 43500 and one < 43750:
        weight_code_list.append('4375')
    elif one >= 43750 and one < 44000:
        weight_code_list.append('4400')
    elif one >= 44000 and one < 44250:
        weight_code_list.append('4425')
    elif one >= 44250 and one < 44500:
        weight_code_list.append('4450')
    elif one >= 44500 and one < 44750:
        weight_code_list.append('4475')
    elif one >= 44750 and one < 45000:
        weight_code_list.append('4500')
    elif one >= 45000 and one < 45250:
        weight_code_list.append('4525')
    elif one >= 45250 and one < 45500:
        weight_code_list.append('4550')
    elif one >= 45500 and one < 45750:
        weight_code_list.append('4575')
    elif one >= 45750 and one < 46000:
        weight_code_list.append('4600')
    elif one >= 46000 and one < 46250:
        weight_code_list.append('4625')
    elif one >= 46250 and one < 46500:
        weight_code_list.append('4650')
    elif one >= 46500 and one < 46750:
        weight_code_list.append('4675')
    elif one >= 46750 and one < 47000:
        weight_code_list.append('4700')
    elif one >= 47000 and one < 47250:
        weight_code_list.append('4725')
    elif one >= 47250 and one < 47500:
        weight_code_list.append('4750')
    elif one >= 47500 and one < 47750:
        weight_code_list.append('4775')
    elif one >= 47750 and one < 48000:
        weight_code_list.append('4800')
    elif one >= 48000 and one < 48250:
        weight_code_list.append('4825')
    elif one >= 48250 and one < 48500:
        weight_code_list.append('4850')
    elif one >= 48500 and one < 48750:
        weight_code_list.append('4875')
    elif one >= 48750 and one < 49000:
        weight_code_list.append('4900')
    elif one >= 49000 and one < 49250:
        weight_code_list.append('4925')
    elif one >= 49250 and one < 49500:
        weight_code_list.append('4950')
    elif one >= 49500 and one < 49750:
        weight_code_list.append('4975')
    elif one >= 49750 and one < 50000:
        weight_code_list.append('5000')
    elif one >= 50000:
        weight_code_list.append('9999')

1511211it [00:02, 522708.85it/s]


In [283]:
len(weight_code_list)

1511211

In [284]:
gas_df.shape

(1511211, 47)

In [285]:
gas_df['총중량_코드'] = weight_code_list

In [286]:
n = 250
f'{n:0>4}'

'0250'

In [287]:
# 오류 : 9000개만 처리됨
# weight_code_list = []
# n1 = 250
# for i, one in tqdm(enumerate(df5['총중량'])):
#     for i in range(200):
#         n1 += 250 * i
#         n2 = n1 + 250
#         s = f'{n2:0>4}'
#         if one < 250:
#             weight_code_list.append('0025')
#         elif n1 <= one < n2:
#             weight_code_list.append(s)
#         elif one >= 50000:
#             weight_code_list.append('9999')
# len(weight_code_list)

### 배기량으로 분류 현황

In [288]:
gas_df['배기량'].value_counts()

2497.0      408658
1991.0      240155
2902.0      138275
2959.0       74617
1995.0       68056
             ...  
2171.0           1
3972.0           1
693001.0         1
2687.0           1
2286.0           1
Name: 배기량, Length: 652, dtype: int64

In [289]:
gas_df['배기량'].isnull().sum()

11

In [290]:
gas_df['배기량'] = gas_df['배기량'].fillna(-1)

In [291]:
gas_volume_list = []
for i, one in tqdm(enumerate(gas_df['배기량'])):
    if one == -1:
        gas_volume_list.append('미기입')
    elif one < 800:
        gas_volume_list.append('0.8L 미만')
    elif one >= 800 and one < 1000:
        gas_volume_list.append('0.8L 이상 1.0L 미만')
    elif one >= 1000 and one < 1600:
        gas_volume_list.append('1.0L 이상 1.6L 미만')
    elif one >= 1600 and one < 2000:
        gas_volume_list.append('1.6L 이상 2.0L 미만')
    elif one >= 2000 and one < 3000:
        gas_volume_list.append('2.0L 이상 3.0L 미만')
    elif one >= 3000 and one < 4000:
        gas_volume_list.append('3.0L 이상 4.0L 미만')
    elif one >= 4000 and one < 5000:
        gas_volume_list.append('4.0L 이상 5.0L 미만')
    elif one >= 5000 and one < 6000:
        gas_volume_list.append('5.0L 이상 6.0L 미만')
    elif one >= 6000 and one < 8000:
        gas_volume_list.append('6.0L 이상 8.0L 미만')
    elif one >= 8000 and one < 10000:
        gas_volume_list.append('8.0L 이상 10.0L 미만')
    elif one >= 10000:
        gas_volume_list.append('10.0L 이상')

1511211it [00:01, 956304.11it/s] 


In [292]:
len(gas_volume_list)

1511211

In [293]:
gas_df['배기량_분류'] = gas_volume_list

In [294]:
total_gas_volume_df = gas_df.groupby(['배기량_분류'])['차대번호'].count().reset_index()
total_gas_volume_df

,배기량_분류,차대번호
0,0.8L 미만,28455
1,0.8L 이상 1.0L 미만,173
2,1.0L 이상 1.6L 미만,164188
3,1.6L 이상 2.0L 미만,502716
4,10.0L 이상,16746
5,2.0L 이상 3.0L 미만,748421
6,3.0L 이상 4.0L 미만,31325
7,4.0L 이상 5.0L 미만,1362
8,5.0L 이상 6.0L 미만,15473
9,6.0L 이상 8.0L 미만,702


#### [출력] [배기량]분류

In [295]:
total_gas_volume_df.to_excel('analysis2/[G4][배기량]분류_통계.xlsx')

#### 배기량 code화

In [296]:
gas_df['배기량'].describe()

count    1.511211e+06
mean     2.408763e+03
std      1.361713e+03
min     -1.000000e+00
25%      1.991000e+03
50%      2.199000e+03
75%      2.497000e+03
max      6.930010e+05
Name: 배기량, dtype: float64

In [297]:
gas_df[gas_df['배기량'] > 30000].shape

(1, 49)

In [298]:
gas_volume_code_list = []
for i, one in tqdm(enumerate(gas_df['배기량'])):
    if one < 800:
        gas_volume_code_list.append('008')
    elif one >= 800 and one < 1000:
        gas_volume_code_list.append('010')
    elif one >= 1000 and one < 1600:
        gas_volume_code_list.append('016')
    elif one >= 1600 and one < 2000:
        gas_volume_code_list.append('020')
    elif one >= 2000 and one < 2500:
        gas_volume_code_list.append('025')
    elif one >= 2500 and one < 3000:
        gas_volume_code_list.append('030')
    elif one >= 3000 and one < 3500:
        gas_volume_code_list.append('035')
    elif one >= 3500 and one < 4000:
        gas_volume_code_list.append('040')
    elif one >= 4000 and one < 4500:
        gas_volume_code_list.append('045')
    elif one >= 4500 and one < 5000:
        gas_volume_code_list.append('050')
    elif one >= 5000 and one < 5500:
        gas_volume_code_list.append('055')
    elif one >= 5500 and one < 6000:
        gas_volume_code_list.append('060')
    elif one >= 6000 and one < 6500:
        gas_volume_code_list.append('065')
    elif one >= 6500 and one < 7000:
        gas_volume_code_list.append('070')
    elif one >= 7000 and one < 7500:
        gas_volume_code_list.append('075')
    elif one >= 7500 and one < 8000:
        gas_volume_code_list.append('080')
    elif one >= 8000 and one < 8500:
        gas_volume_code_list.append('085')
    elif one >= 8500 and one < 9000:
        gas_volume_code_list.append('090')
    elif one >= 9000 and one < 9500:
        gas_volume_code_list.append('095')
    elif one >= 9500 and one < 10000:
        gas_volume_code_list.append('100')
    elif one >= 10000 and one < 10500:
        gas_volume_code_list.append('105')
    elif one >= 10500 and one < 11000:
        gas_volume_code_list.append('110')
    elif one >= 11000 and one < 11500:
        gas_volume_code_list.append('115')
    elif one >= 11500 and one < 12000:
        gas_volume_code_list.append('120')
    elif one >= 12000 and one < 12500:
        gas_volume_code_list.append('125')
    elif one >= 12500 and one < 13000:
        gas_volume_code_list.append('130')
    elif one >= 13000 and one < 13500:
        gas_volume_code_list.append('135')
    elif one >= 13500 and one < 14000:
        gas_volume_code_list.append('140')
    elif one >= 14000 and one < 14500:
        gas_volume_code_list.append('145')
    elif one >= 14500 and one < 15000:
        gas_volume_code_list.append('150')
    elif one >= 15000 and one < 15500:
        gas_volume_code_list.append('155')
    elif one >= 15500 and one < 16000:
        gas_volume_code_list.append('160')
    elif one >= 16000 and one < 16500:
        gas_volume_code_list.append('165')
    elif one >= 16500 and one < 17000:
        gas_volume_code_list.append('170')
    elif one >= 17000 and one < 18000:
        gas_volume_code_list.append('180')
    elif one >= 18000 and one < 18500:
        gas_volume_code_list.append('185')
    elif one >= 18500 and one < 19000:
        gas_volume_code_list.append('190')
    elif one >= 19000 and one < 19500:
        gas_volume_code_list.append('195')
    elif one >= 19500 and one < 20000:
        gas_volume_code_list.append('200')
    elif one >= 20000 and one < 20500:
        gas_volume_code_list.append('205')
    elif one >= 20500 and one < 21000:
        gas_volume_code_list.append('210')
    elif one >= 21000 and one < 21500:
        gas_volume_code_list.append('215')
    elif one >= 21500 and one < 22000:
        gas_volume_code_list.append('220')
    elif one >= 22000 and one < 22500:
        gas_volume_code_list.append('225')
    elif one >= 22500 and one < 23000:
        gas_volume_code_list.append('230')
    elif one >= 23000 and one < 23500:
        gas_volume_code_list.append('235')
    elif one >= 23500 and one < 24000:
        gas_volume_code_list.append('240')
    elif one >= 24000 and one < 24500:
        gas_volume_code_list.append('245')
    elif one >= 24500 and one < 25000:
        gas_volume_code_list.append('250')
    elif one >= 25000 and one < 25500:
        gas_volume_code_list.append('255')
    elif one >= 25500 and one < 26000:
        gas_volume_code_list.append('260')
    elif one >= 26000 and one < 26500:
        gas_volume_code_list.append('265')
    elif one >= 26500 and one < 27000:
        gas_volume_code_list.append('270')
    elif one >= 27000 and one < 27500:
        gas_volume_code_list.append('275')
    elif one >= 27500 and one < 28000:
        gas_volume_code_list.append('280')
    elif one >= 28000 and one < 28500:
        gas_volume_code_list.append('285')
    elif one >= 28500 and one < 29000:
        gas_volume_code_list.append('290')
    elif one >= 29000 and one < 29500:
        gas_volume_code_list.append('295')
    elif one >= 29500 and one < 30000:
        gas_volume_code_list.append('300')
    else:
        gas_volume_code_list.append('999')


1511211it [00:01, 967205.74it/s] 


In [299]:
len(gas_volume_code_list)

1511211

In [300]:
gas_df['배기량_코드'] = gas_volume_code_list

In [301]:
gas_df['배출가스인증번호'] = gas_df['배출가스인증번호'].str.replace(' ', '')

### 배인 코드화

In [302]:
gas_df['배출가스인증번호'].unique()

array([nan, '7MY-HD-24-28', '7MY-KM-14-65', '7MY-KM-14-64',
       '7MY-HD-24-30', '7MY-HD-24-31', '7MY-HD-14-68', '7MY-HD-24-09',
       '6MY-SS-0M-24', '7MY-HD-13-58', '7MY-KM-14-63', '7MY-HD-14-47',
       '7MY-HD-14-46', '7MY-DB-24-42', '8MY-KM-13-19', '7MY-HD-14-57',
       '9MY-KM-14-41', '5MY-HD-24-59', '7MY-SY-14-24', '7MY-SY-14-60',
       'FMY-VTK-14-1', '6MY-KM-14-61', '6MY-HD-23-55', '5MY-KM-24-17',
       'WMY-HD-32', '7MY-DW-13-07', '7MY-HD-24-29', '5MY-AD-14-18',
       '7MY-KM-23-15', 'WMY-HD-34', 'SMY-HD-21', '6MY-DW-14-18',
       '6MY-HD-14-63', 'FMY-VTK-14-4', '6MY-HD-24-12', '6MY-HD-24-44',
       'RMY-HD-20', '7MY-SY-14-48', '6MY-KM-14-57', '6MY-HD-24-100',
       '6MY-KM-24-81', '6MY-KM-14-58', '5MY-KM-24-64', '7MY-HD-14-45',
       '6MY-HD-14-64', 'FMY-VTK-14-2', 'FMY-VTK-14-3', 'VMY-HD-30',
       '5MY-HD-24-35', '5MY-AD-14-12', '9MY-HD-14-21', 'SMY-KM-27',
       '6MYHD1497', '5MY-AD-14-08', '7MY-SM-14-36', '6MY-HD-14-97',
       '7MY-HD-24-08', 'VMY-HD-31', '

In [303]:
temp = gas_df['배출가스인증번호'].value_counts().reset_index().sort_values('index')['index'].reset_index(drop=True)
temp

0         3MY-FK-08
1         4MY-AD-20
2         4MY-DC-33
3         4MY-PA-06
4      5MY-AD-13-06
           ...     
297       XMY-HD-47
298       XMY-HD-48
299       XMY-HD-55
300       XMY-KM-47
301       YKM-KM-12
Name: index, Length: 302, dtype: object

In [304]:
temp.to_frame()

,index
0,3MY-FK-08
1,4MY-AD-20
2,4MY-DC-33
3,4MY-PA-06
4,5MY-AD-13-06
...,...
297,XMY-HD-47
298,XMY-HD-48
299,XMY-HD-55
300,XMY-KM-47


In [305]:
gas_idx = []
for i in range(1, 301):
    s = f'EN{i:0>3}'
    gas_idx.append(s)
len(gas_idx)

300

In [306]:
gas_idx_df = pd.DataFrame({'배출가스인증번호_코드':gas_idx})
gas_idx_df

,배출가스인증번호_코드
0,EN001
1,EN002
2,EN003
3,EN004
4,EN005
...,...
295,EN296
296,EN297
297,EN298
298,EN299


In [307]:
temp2 = pd.concat([temp, gas_idx_df], axis=1, ignore_index=True)
temp2

,0,1
0,3MY-FK-08,EN001
1,4MY-AD-20,EN002
2,4MY-DC-33,EN003
3,4MY-PA-06,EN004
4,5MY-AD-13-06,EN005
...,...,...
297,XMY-HD-47,EN298
298,XMY-HD-48,EN299
299,XMY-HD-55,EN300
300,XMY-KM-47,NaN


In [308]:
temp2.columns = ['배출가스인증번호', '배출가스인증번호_코드']
temp2

,배출가스인증번호,배출가스인증번호_코드
0,3MY-FK-08,EN001
1,4MY-AD-20,EN002
2,4MY-DC-33,EN003
3,4MY-PA-06,EN004
4,5MY-AD-13-06,EN005
...,...,...
297,XMY-HD-47,EN298
298,XMY-HD-48,EN299
299,XMY-HD-55,EN300
300,XMY-KM-47,NaN


In [309]:
temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   배출가스인증번호     302 non-null    object
 1   배출가스인증번호_코드  300 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


#### [출력] 배인번호 코드화 리스트

In [310]:
temp2[['배출가스인증번호_코드', '배출가스인증번호']].to_excel('analysis2/[G4][배출가스인증번호]코드_리스트.xlsx')

In [311]:
gas_df.shape

(1511211, 50)

In [312]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN',
       '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드'],
      dtype='object')

In [313]:
gas_df = gas_df.merge(temp2, on='배출가스인증번호', how='left')
gas_df.shape

(1511211, 51)

In [314]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN',
       '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드'],
      dtype='object')

#### [출력] 등록정보&제원정보(배인번호_코드_추가)

In [315]:
gas_df.to_csv('data/[ODS][G4]등록정보&제원정보(배인번호_코드_추가)(2022.12.29).csv', index=False)

## 차대번호로 분류

### wmi 코드로 분류

In [316]:
wmi = pd.read_excel('data/20190730_WMI CODE.xlsx', sheet_name='WMI CODE')
wmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   WMI            690 non-null    object 
 1   MF CODE        690 non-null    object 
 2   Identifier     690 non-null    object 
 3   Manufacturer   690 non-null    object 
 4   Country        688 non-null    object 
 5   Vehicle Types  249 non-null    object 
 6   Description    54 non-null     object 
 7   참고             0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 43.2+ KB


In [317]:
df6 = gas_df.copy()

In [318]:
df6['차대번호'].head()

0    KMFLA18KP8C028585
1    KMCLK18KP8C035195
2    KMFLA18KP8C031274
3    KMCGK17HP9C112030
4    KLACA26RD8B226753
Name: 차대번호, dtype: object

In [319]:
df6['wmi'] = df6['차대번호'].str[:3]

In [320]:
df6['wmi'].head()

0    KMF
1    KMC
2    KMF
3    KMC
4    KLA
Name: wmi, dtype: object

In [321]:
df6 = df6.merge(wmi, left_on='wmi', right_on='WMI', how='left')
df6[['차대번호', 'wmi', 'WMI']].head()

,차대번호,wmi,WMI
0,KMFLA18KP8C028585,KMF,KMF
1,KMCLK18KP8C035195,KMC,NaN
2,KMFLA18KP8C031274,KMF,KMF
3,KMCGK17HP9C112030,KMC,NaN
4,KLACA26RD8B226753,KLA,KLA


In [322]:
df6.shape

(1511211, 60)

In [323]:
df6['WMI'].isnull().sum()

74578

In [324]:
df6['Manufacturer'].isnull().sum()

74578

In [325]:
wmi.columns

Index(['WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country',
       'Vehicle Types', 'Description', '참고'],
      dtype='object')

In [326]:
df6[['WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description', '참고']].head()

,WMI,MF CODE,Identifier,Manufacturer,Country,Vehicle Types,Description,참고
0,KMF,HYUN,Hyundai,Hyundai,Korea,Commercail vehicle,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KMF,HYUN,Hyundai,Hyundai,Korea,Commercail vehicle,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KLA,GM D,GM Daewoo,Daewoo General Motors South Korea,Korea,NaN,NaN,NaN


In [327]:
df6['Identifier'].isnull().sum()

74578

In [328]:
df6['Manufacturer'] = df6['Manufacturer'].fillna(-1)
df6['Identifier'] = df6['Identifier'].fillna(-1)

In [329]:
manu_df = df6.groupby('Identifier')['차대번호'].count().reset_index()
manu_df

,Identifier,차대번호
0,-1,74578
1,Alfa Romeo,1
2,Audi,2069
3,Austin-Rover,83
4,BMW,2202
...,...,...
71,Toyota,146
72,Toyota,54
73,Volkswagen,5740
74,Volvo,6610


In [330]:
manu_df = manu_df.sort_values('차대번호', ascending=False).reset_index(drop=True)
manu_df

,Identifier,차대번호
0,Hyundai,877457
1,Kia,354069
2,GM Daewoo,114720
3,-1,74578
4,SsangYong,30632
...,...,...
71,Kenworth,1
72,Dong Feng,1
73,Matra,1
74,Daimler,1


#### [출력] Identifier별 차대번호 통계

In [331]:
# 제작사(Manufacturer)별 차대번호 통계
manu_df.to_excel('analysis2/[G4][차대번호]제작사별_통계.xlsx', index=False)

## 차대번호

In [332]:
# - 오류
#     - 1\. null 값 : 
#     - 2\. 정상(17자리, 영문 포함) : 
#     - 3\. 17자리, 영문X : 
#     - 4\. 17자리X, 영문 : 
#     - 5\. 17자리X, 영문X : 

In [333]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1511211 entries, 0 to 1511210
Data columns (total 60 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   차량관리번호         1511211 non-null  object 
 1   차량등록번호         1511211 non-null  object 
 2   본거지법정동코드       1511211 non-null  int64  
 3   소유자구분          1511211 non-null  object 
 4   차종_등록정보        1511209 non-null  object 
 5   용도             1511211 non-null  object 
 6   차대번호           1511211 non-null  object 
 7   최초등록일자         1511211 non-null  int64  
 8   차량연식           1511211 non-null  int64  
 9   차량제작일자         1342438 non-null  float64
 10  배출가스검사유효일자     1511199 non-null  float64
 11  제원관리번호         1511211 non-null  object 
 12  배출가스인증번호       1387511 non-null  object 
 13  배출가스등급         1511211 non-null  object 
 14  저공해조치종류        189 non-null      float64
 15  저공해조치구조변경YN    1511211 non-null  object 
 16  차량말소YN         1511211 non-null  object 
 17  취득일자    

In [334]:
df6['차대번호'].isnull().sum()

0

In [335]:
len('KMFZCY7JBCU784032')

17

### 차대번호 정상 확인

In [336]:
# 17자리, 영문 -> 정상 : 1508498
count = 0
p61 = re.compile('[a-zA-Z]+')
idx61 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    if len(str(one)) == 17 and p61.search(str(one)):
        idx61.append(i)
        count += 1
count

1511211it [00:02, 648362.47it/s]


1508498

In [337]:
# 정상에서 "I, O, Q" 포함한 값 : 1059
temp61 = df6.loc[idx61, '차대번호'].to_numpy()
idx61_1 = []
for i, one in tqdm(enumerate(temp61)):
    if ('I' in one) or ('O' in one) or ('Q' in one):
        idx61_1.append(idx61[i])
len(idx61_1)

1508498it [00:00, 1921510.90it/s]


1059

In [338]:
df6.iloc[idx61_1, 6][:5]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_13044\2454213598.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df6.iloc[idx61_1, 6][:5]


1356      WPOZZZ96ZLS402952
117458    WPOZZZ96ZKS400603
328413    YV2ABSOC4AA695671
343494    YV2ABSOC2AA693157
344096    YV2ABSOC1AA694168
Name: 차대번호, dtype: object

In [339]:
# 17자리, 영문X  => 숫자만 : 1
count = 0
p62 = re.compile('[a-zA-Z]+')
idx62 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    n = 0
    for j in str(one):
        if p62.search(j):
            n += 1
    if len(str(one)) == 17 and n == 0:
        idx62.append(i)
        count += 1
count

1511211it [00:12, 121011.76it/s]


1

In [340]:
df6.loc[:, '차대번호'].reindex(idx62)

1454427    01901321371111111
Name: 차대번호, dtype: object

In [341]:
# 17자리 X, 영문 : 57
count = 0
p63 = re.compile('[a-zA-Z]+')
idx63 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    if len(str(one)) != 17 and p63.search(one):
        idx63.append(i)
        count += 1
count

1511211it [00:00, 1537011.05it/s]


136

In [342]:
df6.loc[:, '차대번호'].reindex(idx63)[:5]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_13044\115020966.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df6.loc[:, '차대번호'].reindex(idx63)[:5]


1207       FD3S403706
1758    RZN1850034584
2359        SF5087998
2709      H51A5206077
3332      ECR33112768
Name: 차대번호, dtype: object

In [343]:
# 17자리 X, 영문X : 2573
count = 0
p64 = re.compile('[a-zA-Z]+')
idx64 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    n = 0
    for j in one:
        if p64.search(j):
            n += 1
    if len(str(one)) != 17 and n == 0:
        idx64.append(i)
        count += 1
count

1511211it [00:11, 127042.41it/s]


2576

In [344]:
df6.loc[:, '차대번호'].reindex(idx64)[:5]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_13044\4149012222.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df6.loc[:, '차대번호'].reindex(idx64)[:5]


726        119311327021
1131113    289711309283
1147421    119811229792
1155762    119811328945
1156094    119411329496
Name: 차대번호, dtype: object

#### 차대번호 길이, IOQ포함 여부 통계

In [345]:
# 차대번호 길이 종류 확인
len_vin = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    len_vin.append(len(str(one)))
set(len_vin)

1511211it [00:01, 1378041.22it/s]


{9, 10, 11, 12, 13, 14, 15, 16, 17}

In [346]:
df6['VIN_length'] = [ len(one) for one in df6['차대번호'] ]

In [347]:
df6['VIN_length'].unique()

array([17, 12, 10, 13,  9, 11, 16, 15, 14], dtype=int64)

In [348]:
# IOQ 포함 여부
IOQ = []
for one in df6['차대번호']:
    if ('I' in one) or ('O' in one) or ('Q' in one):
        IOQ.append('Y')
    else:
        IOQ.append('N')
df6['IOQ'] = IOQ

In [349]:
df6['IOQ'].unique()

array(['N', 'Y'], dtype=object)

In [350]:
# 알파벳 포함
included_alpha = []
count = 0
p65 = re.compile('[a-zA-Z]+')
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    n = 0
    for j in one:
        if p65.search(j):
            n += 1
    if n == 0:
        included_alpha.append('N')
        count += 1
    else:
        included_alpha.append('Y')
count

1511211it [00:11, 129104.07it/s]


2577

In [351]:
df6['alpha'] = included_alpha

In [352]:
df6.groupby(['VIN_length', 'alpha', 'IOQ'])['차대번호'].count()

VIN_length  alpha  IOQ
9           Y      N            1
10          N      N            1
            Y      N           12
11          N      N          209
            Y      N           18
12          N      N         2135
            Y      N            5
13          N      N          154
            Y      N            8
                   Y            1
14          N      N           22
15          N      N           38
            Y      N            6
                   Y            3
16          N      N           17
            Y      N           75
                   Y            7
17          N      N            1
            Y      N      1507439
                   Y         1059
Name: 차대번호, dtype: int64

In [353]:
df6.groupby(['VIN_length', 'IOQ'])['차대번호'].count()

VIN_length  IOQ
9           N            1
10          N           13
11          N          227
12          N         2140
13          N          162
            Y            1
14          N           22
15          N           44
            Y            3
16          N           92
            Y            7
17          N      1507440
            Y         1059
Name: 차대번호, dtype: int64

In [354]:
V_I_dict = df6.groupby(['VIN_length', 'IOQ'])['차대번호'].groups

In [355]:
V_I_dict.keys()

dict_keys([(9, 'N'), (10, 'N'), (11, 'N'), (12, 'N'), (13, 'N'), (13, 'Y'), (14, 'N'), (15, 'N'), (15, 'Y'), (16, 'N'), (16, 'Y'), (17, 'N'), (17, 'Y')])

In [356]:
g = df6.groupby(['VIN_length', 'IOQ'], as_index=False)['차대번호'].count()
g

,VIN_length,IOQ,차대번호
0,9,N,1
1,10,N,13
2,11,N,227
3,12,N,2140
4,13,N,162
5,13,Y,1
6,14,N,22
7,15,N,44
8,15,Y,3
9,16,N,92


In [357]:
V_I_df = g.sort_values(['VIN_length', 'IOQ'], ascending=[True, True])
V_I_df

,VIN_length,IOQ,차대번호
0,9,N,1
1,10,N,13
2,11,N,227
3,12,N,2140
4,13,N,162
5,13,Y,1
6,14,N,22
7,15,N,44
8,15,Y,3
9,16,N,92


##### 차대번호 길이, IOQ포함 여부 별 예시

In [358]:
for v in range(10, 18):
    for i in ['N', 'Y']:
        try:
            idx = V_I_dict[(v, i)]
            print(f'[VIN_length : {v}, "IOQ"포함 : {i}]')
            print('예시)')
            print(df6.loc[idx[:3], '차대번호'])
            print()
        except:
            pass

[VIN_length : 10, "IOQ"포함 : N]
예시)
1207       FD3S403706
1154912    HP10204570
1156198    ER34003066
Name: 차대번호, dtype: object

[VIN_length : 11, "IOQ"포함 : N]
예시)
2709      H51A5206077
3332      ECR33112768
433198    ECR33011448
Name: 차대번호, dtype: object

[VIN_length : 12, "IOQ"포함 : N]
예시)
726        119311327021
23373      UCF210036303
1131113    289711309283
Name: 차대번호, dtype: object

[VIN_length : 13, "IOQ"포함 : N]
예시)
1758      RZN1850034584
210020    JZS1470052828
391117    JZS1470124257
Name: 차대번호, dtype: object

[VIN_length : 13, "IOQ"포함 : Y]
예시)
1157610    DOT312AS2M241
Name: 차대번호, dtype: object

[VIN_length : 14, "IOQ"포함 : N]
예시)
1167902    27911130156505
1168772    27931130152305
1332242    21891130315401
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : N]
예시)
1156908    11-97-113-29992
1157213    11-97-113-28686
1157359    41-96-113-35598
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : Y]
예시)
1174739    JNIPB2IJU525943
1422837    KLATA19TILB3753
1510807    128513

In [359]:
df6.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T

VIN_length,9,10,11,12,13,14,15,16,17
연료,,,,,,,,,
CNG(압축천연가스),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
LGP(액화석유가스),NaN,NaN,NaN,15.0,NaN,NaN,NaN,2.0,26072.0
경유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1152687.0
기타연료,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
알코올,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
휘발유,1.0,13.0,227.0,2114.0,163.0,22.0,47.0,86.0,329680.0


##### [출력] vin 길이별, 연료별 차대번호 통계

In [360]:
# vin 길이별, 연료별 차대번호 개수 통계
df6.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]길이_통계.xlsx')

##### [출력] vin IOQ 포함 여부별, 연료별 차대번호 통계

In [361]:
df6.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T

IOQ,N,Y
연료,,
CNG(압축천연가스),1.0,NaN
LGP(액화석유가스),26085.0,4.0
경유,1152502.0,195.0
기타연료,1.0,NaN
알코올,1.0,NaN
휘발유,331486.0,867.0


In [362]:
# vin IOQ 포함별, 연료별 차대번호 개수 통계
df6.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]IOQ포함여부_통계.xlsx')

#### 차대번호 10번째 자리 문자로 연식 확인

In [363]:
df6['차량연식'].unique()

array([2008, 2009, 2010, 2006, 2007, 2013, 2012, 2011, 2001, 2019, 2020,
       1992, 2002, 1994, 1996, 2018, 1991, 1993, 2005, 1999, 2000, 1998,
       1989, 1990, 1995, 1997, 1988, 2017, 2016, 2004, 2003, 2015, 1987,
       2014, 1986, 1983, 1985, 1981, 1978, 1984, 1973, 1901, 1980, 1979,
       1968], dtype=int64)

In [364]:
df6.groupby(['차량연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LGP(액화석유가스),경유,기타연료,알코올,휘발유
차량연식,,,,,,
1901,NaN,NaN,NaN,NaN,NaN,1.0
1968,NaN,NaN,NaN,NaN,NaN,1.0
1973,NaN,NaN,NaN,NaN,NaN,1.0
1978,NaN,NaN,NaN,NaN,NaN,1.0
1979,NaN,NaN,NaN,NaN,NaN,3.0
1980,NaN,NaN,NaN,NaN,NaN,1.0
1981,NaN,NaN,NaN,NaN,NaN,2.0
1983,NaN,NaN,NaN,NaN,NaN,5.0
1984,NaN,NaN,NaN,NaN,NaN,4.0


##### [출력] 차량연식별, 연료별 차대번호 통계

In [365]:
# 차량연식별, 연료별 차대번호 개수 통계
df6.groupby(['차량연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]통계.xlsx')

In [366]:
df6['차대번호_10자리'] = df6['차대번호'].str[9]
df6['차대번호_10자리'].unique()

array(['8', '9', 'A', '6', '7', 'D', 'C', 'B', '1', 'K', 'L', 'N', '2',
       'R', 'T', 'J', 'M', 'P', '5', 'X', 'Y', 'W', 'S', 'V', 'H', '0',
       'G', '4', '3', 'F', 'E', nan, 'U', 'O', '-', 'Z'], dtype=object)

In [367]:
vin_10_dict = {'A':1980, 'B':1981, 'C':1982, 'D':1983, 'E':1984, 'F':1985, 'G':1986, 'H':1987, 'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009}

In [368]:
df6['차대번호_10자리_기준연식'] = df6['차대번호_10자리'].replace(vin_10_dict)

In [369]:
df6['차대번호_10자리_기준연식'].unique()

array([2008, 2009, 1980, 2006, 2007, 1983, 1982, 1981, 2001, 1989, 1990,
       1992, 2002, 1994, 1996, 1988, 1991, 1993, 2005, 1999, 2000, 1998,
       1995, 1997, 1987, '0', 1986, 2004, 2003, 1985, 1984, nan, 'U', 'O',
       '-', 'Z'], dtype=object)

In [370]:
df6.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LGP(액화석유가스),경유,기타연료,알코올,휘발유
차대번호_10자리_기준연식,,,,,,
1980,NaN,NaN,250181.0,NaN,NaN,10.0
1981,NaN,NaN,235847.0,NaN,NaN,10.0
1982,NaN,NaN,93535.0,NaN,NaN,57.0
1983,NaN,NaN,30.0,NaN,NaN,69.0
1984,NaN,NaN,14.0,NaN,NaN,3.0
1985,NaN,1.0,463.0,NaN,NaN,7.0
1986,NaN,NaN,482.0,NaN,NaN,16.0
1987,NaN,NaN,732.0,NaN,NaN,1307.0
1988,NaN,91.0,484.0,NaN,1.0,11664.0


##### [출력] vin 10자리문자기준별, 연료별 차대번호 통계

In [371]:
# vin 10자리 문자기준별, 연료별 차대번호 통계
df6.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]차대번호10번째자리문자기준_통계.xlsx')

In [372]:
df6.shape

(1511211, 65)

In [373]:
df6.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN',
       '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI',
       'MF CODE', 'Identifier', 'Manufacturer', 'Country', 'Vehicle Types',
       'Description', '참고', 'VIN_length', 'IOQ', 'alpha', '차대번호_10자리',
       '차대번호_10자리_기준연식'],
      dtype='object')

# [출력] '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description', '참고', 'VIN_length', 'IOQ', 'alpha', '차대번호_10자리', '차대번호_10자리_기준연식'

In [374]:
# 약 42 s
df6.to_csv('data/[G4]등록정보&제원정보(wmi,vin관련컬럼추가)(2022-12-29).csv')

## [로드]정기/정밀 검사 데이터 로드

In [3]:
# 약 4m
per_ins = pd.read_csv('data/[정기검사]20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_6508\4283608937.py:2: DtypeWarning: Columns (36,39,42,45,48,63) have mixed types. Specify dtype option on import or set low_memory=False.
  per_ins = pd.read_csv('data/[정기검사]20190701_20220630.csv', index_col=0)


In [4]:
per_ins.shape

(16535477, 63)

In [5]:
# 약 11m
per_d_ins = pd.read_csv('data/[정밀검사]20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_6508\1915627214.py:2: DtypeWarning: Columns (13,44,45,46,49,53,54,55,59,60,61,65,66,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  per_d_ins = pd.read_csv('data/[정밀검사]20190701_20220630.csv', index_col=0)


In [6]:
per_d_ins.shape

(22397343, 75)

In [7]:
per_ins.columns

Index(['검사소명', '정기검사일자', '검사종류', '검사접수번호', '통합접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일자', '제작일자',
       '검사전 유효기간 만료일', '관능검사판정', '관능부적합항목', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '검사판정', '재검사기간', '검사후 갱신유효기간', '관능검사원', '배출가스검사원', '책임검사원', '검사 판정시간',
       '자료구분코드', '파일명'],
      dtype='object')

In [675]:
per_ins['공기온도'].unique()

array([nan])

In [8]:
per_d_ins.columns

Index(['정밀검사소명', '정밀검사일자', '정밀검사종류', '정밀검사접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '원동기배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일',
       '정밀검사유효기간', '관능검사판정', '관능검사부적합내용', '정밀검사측정방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '정밀검사판정', '재검사기간', '정밀검사유효기간2', '관능검사등록자', '정밀검사등록자', '정밀검사소대표명',
       '기타내용1', '기타내용2', '기타내용3', '기타내용4', '기타내용5', '원본대장테이블', '정밀검사시간정보',
       '통합접수번호', '데이터생성일자', '수신파일명', '제작등록일자', '데이터구분', '삭제구분', '삭제사유1',
       '삭제사유2', '삭제사유3', '삭제사유4'],
      dtype='object')

In [679]:
per_d_ins['산소값'].unique()

array([  0.  ,   3.7 ,    nan, ...,  19.17, 159.6 ,  30.9 ])

In [680]:
per_d_ins['산소값'].value_counts()

0.00     13318535
0.10       706340
0.20       275318
14.80      228366
14.70      222326
           ...   
8.89            1
19.73           1
10.01           1
31.40           1
30.90           1
Name: 산소값, Length: 2268, dtype: int64

In [678]:
per_d_ins['공기온도'].unique()

array([  0. ,   nan,  30.8, ..., 102. , 167. , 113. ])

In [677]:
per_d_ins['공기압력'].unique()

array([ 0.000e+00,        nan,  1.019e+02, ..., -2.000e-01,  9.998e+02,
        6.000e+00])

In [681]:
per_d_ins['공기압력'].max()

6552.0

In [682]:
per_d_ins['공기압력'].min()

-1499.7

In [9]:
per_ins['검사방법'].unique()

array(['무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [683]:
per_ins['무부하매연측정치6'].max()

3.0

In [684]:
per_ins['무부하매연측정치6'].min()

0.0

In [685]:
per_ins['무부하매연측정치6'].value_counts()

1.00    1584200
1.01     968540
0.99     696920
1.02     356388
1.03     197479
         ...   
2.09          1
2.47          1
0.36          1
2.82          1
2.53          1
Name: 무부하매연측정치6, Length: 241, dtype: int64

In [690]:
per_ins.shape

(16535477, 63)

In [689]:
per_ins[(per_ins['무부하매연측정치6'] > 1.2) | (per_ins['무부하매연측정치6'] < 0.8)].shape

(5639, 63)

In [10]:
per_d_ins['정밀검사측정방법'].unique()

array(['부하검사(ASM-Idling)', '부하검사(KD-147)', '무부하검사(TSI)', '무부하검사(급가속)',
       '부하검사(LUG DOWN)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

In [11]:
per_ins['검사 판정시간'].head()

0    20190702 14:51:40
1    20190702 14:52:45
2    20190702 14:49:28
3    20190702 15:58:57
4    20190702 14:52:04
Name: 검사 판정시간, dtype: object

In [13]:
per_d_ins['정밀검사시간정보'].head()

0    20190701 17:13:00
1    20190701 17:45:15
2                  NaN
3    20190701 17:17:41
4    20190701 17:23:59
Name: 정밀검사시간정보, dtype: object

In [14]:
per_ins['검사 판정시간'][0]

'20190702 14:51:40'

In [15]:
per_d_ins['정밀검사시간정보'][0]

'20190701 17:13:00'

In [16]:
per_ins['검사 판정시간'][0] > per_d_ins['정밀검사시간정보'][0]

True

In [17]:
per_ins['검사 판정시간'][0] < per_d_ins['정밀검사시간정보'][0]

False

### 정기 검사 데이터 확인

In [38]:
per_ins[['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6']].isnull().sum()

무부하매연측정치1     1148260
무부하매연허용치1     1148260
무부하매연판정1      1148260
무부하매연측정치2    10820017
무부하매연허용치2    10820017
무부하매연판정2     10820017
무부하매연측정치3    10820017
무부하매연허용치3    10820052
무부하매연판정3     10820052
무부하매연측정치4    12029796
무부하매연허용치4    12029796
무부하매연판정4     12029796
무부하매연측정치5    12029796
무부하매연허용치5    12029796
무부하매연판정5     12029796
무부하매연측정치6    12029796
무부하매연허용치6    12029806
무부하매연판정6     12029806
dtype: int64

In [24]:
per_ins[per_ins['무부하매연측정치6'].isnull() == False].index

Int64Index([    1056,     1057,     1058,     1059,     1060,     1063,
                1064,     1067,     1068,     1070,
            ...
            16533202, 16533713, 16533714, 16533716, 16533722, 16533760,
            16533761, 16533766, 16533767, 16533769],
           dtype='int64', length=4505681)

In [26]:
# 6번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치6'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True) | (per_ins['무부하매연측정치3'].isnull() == True) | (per_ins['무부하매연측정치4'].isnull() == True) | (per_ins['무부하매연측정치5'].isnull() == True))].index

Int64Index([], dtype='int64')

In [27]:
# 5번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치5'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True) | (per_ins['무부하매연측정치3'].isnull() == True) | (per_ins['무부하매연측정치4'].isnull() == True))].index

Int64Index([], dtype='int64')

In [28]:
# 4번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치4'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True) | (per_ins['무부하매연측정치3'].isnull() == True))].index

Int64Index([], dtype='int64')

In [29]:
# 3번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치3'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True))].index

Int64Index([], dtype='int64')

In [30]:
# 2번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치3'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True))].index

Int64Index([], dtype='int64')

In [31]:
per_ins.loc[1056, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간']].to_list()

[0.0,
 1.0,
 'Y',
 15.0,
 120.0,
 'Y',
 1.01,
 0.1,
 'Y',
 0.1,
 1.0,
 'Y',
 31.0,
 120.0,
 'Y',
 1.0,
 0.1,
 'Y',
 '20190701 13:44:39']

In [43]:
temp = per_ins.loc[0, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간']]
temp

무부하매연측정치1                  2.0
무부하매연허용치1                 25.0
무부하매연판정1                     Y
무부하매연측정치2                  NaN
무부하매연허용치2                  NaN
무부하매연판정2                   NaN
무부하매연측정치3                  NaN
무부하매연허용치3                  NaN
무부하매연판정3                   NaN
무부하매연측정치4                  NaN
무부하매연허용치4                  NaN
무부하매연판정4                   NaN
무부하매연측정치5                  NaN
무부하매연허용치5                  NaN
무부하매연판정5                   NaN
무부하매연측정치6                  NaN
무부하매연허용치6                  NaN
무부하매연판정6                   NaN
검사 판정시간      20190702 14:51:40
Name: 0, dtype: object

In [44]:
temp['무부하매연측정치1']

2.0

In [76]:
np.isnan(temp['무부하매연측정치2'])

True

### 정기검사 Grade 점수 계산 테스트

In [47]:
float('0.0')

0.0

In [53]:
float(np.nan)

nan

In [62]:
gas_mea_value_p = re.compile('\d[.]\d')
gas_mea_value_p.search('0.0')

<re.Match object; span=(0, 3), match='0.0'>

In [69]:
gas_mea_value_p.search('20190701 13:44:39')

In [64]:
gas_mea_value_p.search('H5262021070100191')

In [65]:
gas_mea_value_p.search('20210702')

In [92]:
temp['무부하매연측정치1']

2.0

In [81]:
np.isnan(temp['무부하매연측정치1'])

False

In [84]:
temp

무부하매연측정치1                  2.0
무부하매연허용치1                 25.0
무부하매연판정1                     Y
무부하매연측정치2                  NaN
무부하매연허용치2                  NaN
무부하매연판정2                   NaN
무부하매연측정치3                  NaN
무부하매연허용치3                  NaN
무부하매연판정3                   NaN
무부하매연측정치4                  NaN
무부하매연허용치4                  NaN
무부하매연판정4                   NaN
무부하매연측정치5                  NaN
무부하매연허용치5                  NaN
무부하매연판정5                   NaN
무부하매연측정치6                  NaN
무부하매연허용치6                  NaN
무부하매연판정6                   NaN
검사 판정시간      20190702 14:51:40
Name: 0, dtype: object

In [86]:
gas_mea_value_p.search(str(temp['무부하매연측정치1']))

<re.Match object; span=(0, 3), match='2.0'>

In [91]:
temp['무부하매연판정1']

'Y'

In [94]:
temp['무부하매연판정6']

nan

In [93]:
np.isnan(temp['무부하매연판정6'])

True

In [96]:
temp['무부하매연판정1'] not in ['Y', 'N']

False

In [97]:
# 1개의 배출가스 인증번호에 대한 처리 테스트
gas_mea_value_p = re.compile('\d[.]\d')
g_mea_list = []
p_f_list = []

# 변수 생성(측정치, 문자열)
sv1 = str(temp['무부하매연측정치1'])

sv3 = str(temp['무부하매연측정치3'])
sv4 = str(temp['무부하매연측정치4'])
sv5 = str(temp['무부하매연측정치5'])
sv6 = str(temp['무부하매연측정치6'])

# 변수 생성(측정치, 문자열)
v1 = temp['무부하매연측정치1']

v3 = temp['무부하매연측정치3']
v4 = temp['무부하매연측정치4']
v5 = temp['무부하매연측정치5']
v6 = temp['무부하매연측정치6']

# 변수 생성(판정)
f1 = temp['무부하매연판정1']
f3 = temp['무부하매연판정3']
f4 = temp['무부하매연판정4']
f5 = temp['무부하매연판정5']
f6 = temp['무부하매연판정6']

# 값 확인
if np.isnan(v6) or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
    print('6번째 값 부재')
    if np.isnan(v5) or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
        print('5번째 값 부재')
        if np.isnan(v4) or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
            print('4번째 값 부재')
            if np.isnan(v3) or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                print('3번째 값 부재')
                if np.isnan(v1) or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                    print('1번째 값 부재')
                    g = np.nan
                    f = np.nan
                else: # 1번째 값 존재
                    g = v1
                    f = f1
            else: # 3번째 값 존재
                g = temp['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3'].mean()
                f = f3 
        else: # 4번째 값 존재
            g = temp['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4'].mean()
            f = f4
    else: # 5번째 값 존재
        g = temp['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5'].mean()
        f = f5
else: # 6번째 값 존재
    g = temp['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6'].mean()
    f = f6
g_mea_list.append(g)
p_f_list.append(f)

6번째 값 부재
5번째 값 부재
4번째 값 부재
3번째 값 부재


In [98]:
g_mea_list, p_f_list

([2.0], ['Y'])

### 정밀 검사 데이터 확인

In [32]:
per_d_ins[per_d_ins['무부하매연측정치6'].isnull() == False].index

Int64Index([       0,        1,        3,        4,        6,        7,
                   9,       11,       12,       13,
            ...
            22397333, 22397334, 22397335, 22397336, 22397337, 22397338,
            22397339, 22397340, 22397341, 22397342],
           dtype='int64', length=17552908)

In [33]:
per_d_ins.loc[0, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '정밀검사시간정보']].to_list()

[0.03,
 0.46,
 'Y',
 3.0,
 37.0,
 'Y',
 3.0,
 610.0,
 'Y',
 0.0,
 1.0,
 'Y',
 2.0,
 120.0,
 'Y',
 1.02,
 0.1,
 'Y',
 '20190701 17:13:00']

In [34]:
# 6번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치6'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True) | (per_d_ins['무부하매연측정치3'].isnull() == True) | (per_d_ins['무부하매연측정치4'].isnull() == True) | (per_d_ins['무부하매연측정치5'].isnull() == True))].index

Int64Index([13915437, 13915439, 13915442, 13915443, 13915455, 13915456,
            13915460, 13915461, 13915464, 13915465,
            ...
            22397306, 22397307, 22397309, 22397311, 22397316, 22397317,
            22397320, 22397324, 22397331, 22397335],
           dtype='int64', length=2314645)

In [35]:
per_d_ins.loc[13915437, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '정밀검사시간정보']].to_list()

[0.0,
 15.0,
 'Y',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '20210701 15:50:27',
 'H5262021070100191',
 '20210702',
 nan]

In [36]:
per_d_ins.loc[13915439, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '정밀검사시간정보']].to_list()

[2.0,
 20.0,
 'Y',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '20210701 16:41:21',
 'L0102021070100311',
 '20210702',
 nan]

In [99]:
# 5번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치5'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True) | (per_d_ins['무부하매연측정치3'].isnull() == True) | (per_d_ins['무부하매연측정치4'].isnull() == True))].index

Int64Index([], dtype='int64')

In [100]:
# 4번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치4'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True) | (per_d_ins['무부하매연측정치3'].isnull() == True))].index

Int64Index([], dtype='int64')

In [101]:
# 3번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치3'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True))].index

Int64Index([], dtype='int64')

In [102]:
# 2번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치3'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True))].index

Int64Index([], dtype='int64')

### 제원정보와 정기/정밀 매칭 안되는 샘플

In [583]:
df4['차대번호'].isnull().sum()

0

In [584]:
len(df4['차대번호'].unique())

1528557

In [585]:
len(per_ins['차대번호'].unique())

10328354

In [586]:
len(per_d_ins['차대번호'].unique())

14203268

In [587]:
len(set(df4['차대번호']))

1528557

In [588]:
len(set(df4['차대번호']) - set(per_ins['차대번호']))

1047521

In [589]:
len(set(df4['차대번호']) - set(per_d_ins['차대번호']))

540567

In [590]:
len( set(df4['차대번호']) - set(per_ins['차대번호']) - set(per_d_ins['차대번호']) )

323516

## 제원정보, 정기 검사, 정밀 검사 병합(차대번호 기준)

In [273]:
df6.shape

(1528557, 62)

In [275]:
gas_col1 = ['차대번호', '검사방법', 
            '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', 
            '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
            '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
            '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
            '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
            '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', 
            '검사 판정시간',]
gas_col2 = ['차대번호', '정밀검사측정방법', 
            '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', 
            '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
            '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
            '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', 
            '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', 
            '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6',
            '정밀검사시간정보']

In [276]:
# 약 50s
tdf1 = df6.merge(per_ins[gas_col1], on='차대번호', how='left')
tdf1.shape

(2232455, 82)

In [277]:
# 약 1m
tdf2 = df6.merge(per_d_ins[gas_col2], on='차대번호', how='left')
tdf2.shape

(2873363, 82)

In [278]:
# 약 2m 40s
tdf1.to_csv('data/[G4][통합]제원정보_정기_20190701_20220630.csv')
tdf2.to_csv('data/[G4][통합]제원정보_정밀_20190701_20220630.csv')

# [로드] 제원+정기, 제원+정밀
- [G4][통합]제원정보_정기_20190701_20220630.csv
- [G4][통합]제원정보_정밀_20190701_20220630.csv

In [2]:
# 약 1m
tdf1 = pd.read_csv('data/[G4][통합]제원정보_정기_20190701_20220630.csv', index_col=0)
tdf2 = pd.read_csv('data/[G4][통합]제원정보_정밀_20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_24840\679683512.py:2: DtypeWarning: Columns (12,15,17,29,37,38,58,69,72,75,78,81) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf1 = pd.read_csv('data/[G4][통합]제원정보_정기_20190701_20220630.csv', index_col=0)
C:\Users\kbjung\AppData\Local\Temp\ipykernel_24840\679683512.py:3: DtypeWarning: Columns (12,15,17,29,37,38,58,81) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf2 = pd.read_csv('data/[G4][통합]제원정보_정밀_20190701_20220630.csv', index_col=0)


In [3]:
tdf1.shape

(2232455, 82)

In [4]:
tdf2.shape

(2873363, 82)

In [5]:
tdf1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간'],
      dtype='object')

In [6]:
tdf2.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '정밀검사측정방법', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정밀검사시간정보'],
      dtype='object')

In [7]:
tdf1['차종유형'].unique()

array(['대형', '중형', '소형', '경형'], dtype=object)

In [8]:
tdf1['차종_등록정보'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

In [9]:
tdf1['연료'].unique()

array(['경유', '휘발유', '기타연료', 'LPG(액화석유가스)', 'CNG(압축천연가스)', '알코올'],
      dtype=object)

In [10]:
tdf1['차종분류'].unique()

array(['특수용도형-특수용도형', '다목적', '구난', '일반형-카고', '일반', '밴', '기타', '견인', '덤프',
       '특수작업형', '일반형-픽업', '특수용도형-피견인', '특수용도형-탱크로리', '특수', '승용겸화물'],
      dtype=object)

In [11]:
tdf2['차종유형'].unique()

array(['대형', '중형', '소형', '경형'], dtype=object)

In [12]:
tdf2['차종_등록정보'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

In [13]:
tdf2['연료'].unique()

array(['경유', '휘발유', '기타연료', 'LPG(액화석유가스)', 'CNG(압축천연가스)', '알코올'],
      dtype=object)

### 제원_정기 파일 전처리

In [14]:
tdf1['검사방법'].unique()

array([nan, '무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [15]:
# 제원_정기 파일 정렬
# 차대번호, 검사방법, 검사 판정시간
    # 최신순으로
# 차대번호 기준 중복 제거

In [174]:
tdf1.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사 판정시간
0,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KLACA26RD8B226753-01,153구6446,2826011000,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KMCGK17HP8C086937-01,88조4842,4476032000,법인,화물,자가용,KMCGK17HP8C086937,20210830,2008,20080315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,KMCYSS7JP7U084697,20210826,2007,20070227.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
tdf1.sort_values(['차대번호', '검사방법', '검사 판정시간']).head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사 판정시간
2198852,018013212A7-01,경기65가1508,4790036046,주민,승용,자가용,018013212A7,19900118,1980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2225371,01831300012-01,01조2678,1144010100,주민,승용,자가용,01831300012,19880823,1988,19880823.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2215936,4199경기53라2586,경기53라2586,4111113700,주민,승용,자가용,01841300699,19890328,1989,19890328.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2221443,01871300044-01,대전1노5517,3020010300,주민,승용,자가용,01871300044,19881124,1987,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2218846,01881100314-01,경북74거1335,4773038040,법인,승합,자가용,01881100314,19890125,1988,19890125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
tdf1 = tdf1.sort_values(['차대번호', '검사방법', '검사 판정시간']).drop_duplicates('차대번호', keep='first').reset_index(drop=True)
tdf1.shape

(1528557, 82)

In [178]:
tdf1['배출가스인증번호'].isnull().sum()

123003

In [179]:
tdf1['검사방법'].isnull().sum()

1047521

In [180]:
tdf1['무부하매연측정치1'].isnull().sum()

1047552

## 제원_정밀 파일 전처리

In [24]:
tdf2['정밀검사측정방법'].unique()

array([nan, '부하검사(LUG DOWN)', '부하검사(KD-147)', '부하검사(ASM-Idling)',
       '무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [25]:
# 제원_정밀 파일 정렬
# 차대번호, 정밀검사측정방법, 정밀검사시간정보
    # 최신순으로
# 차대번호 기준 중복 제거

In [181]:
tdf2.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,정밀검사시간정보
0,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KLACA26RD8B226753-01,153구6446,2826011000,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KMCGK17HP8C086937-01,88조4842,4476032000,법인,화물,자가용,KMCGK17HP8C086937,20210830,2008,20080315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,KMCYSS7JP7U084697,20210826,2007,20070227.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
tdf2.sort_values(['차대번호', '정밀검사측정방법', '정밀검사시간정보']).head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,정밀검사시간정보
2839486,018013212A7-01,경기65가1508,4790036046,주민,승용,자가용,018013212A7,19900118,1980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866198,01831300012-01,01조2678,1144010100,주민,승용,자가용,01831300012,19880823,1988,19880823.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2856680,4199경기53라2586,경기53라2586,4111113700,주민,승용,자가용,01841300699,19890328,1989,19890328.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2862220,01871300044-01,대전1노5517,3020010300,주민,승용,자가용,01871300044,19881124,1987,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2859618,01881100314-01,경북74거1335,4773038040,법인,승합,자가용,01881100314,19890125,1988,19890125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
tdf2 = tdf2.sort_values(['차대번호', '정밀검사측정방법', '정밀검사시간정보']).drop_duplicates('차대번호', keep='first').reset_index(drop=True)
tdf2.shape

(1528557, 82)

In [184]:
tdf2.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,정밀검사시간정보
0,018013212A7-01,경기65가1508,4790036046,주민,승용,자가용,018013212A7,19900118,1980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01831300012-01,01조2678,1144010100,주민,승용,자가용,01831300012,19880823,1988,19880823.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4199경기53라2586,경기53라2586,4111113700,주민,승용,자가용,01841300699,19890328,1989,19890328.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01871300044-01,대전1노5517,3020010300,주민,승용,자가용,01871300044,19881124,1987,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01881100314-01,경북74거1335,4773038040,법인,승합,자가용,01881100314,19890125,1988,19890125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
tdf2['배출가스인증번호'].isnull().sum()

123003

In [186]:
tdf2['정밀검사측정방법'].isnull().sum()

540567

In [187]:
tdf2['무부하매연측정치1'].isnull().sum()

545132

## 두 파일 concat(제원+정기, 제원+정밀)

In [188]:
tdf1['정기정밀'] = '정기'

In [189]:
tdf1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간', '정기정밀'],
      dtype='object')

In [190]:
tdf2['정기정밀'] = '정밀'

In [191]:
tdf2.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '정밀검사측정방법', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정밀검사시간정보', '정기정밀'],
      dtype='object')

In [192]:
tdf1.shape, tdf2.shape

((1528557, 83), (1528557, 83))

In [193]:
tdf1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간', '정기정밀'],
      dtype='object')

In [256]:
tdf1.columns = ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사판정시간', '정기정밀']

In [257]:
tdf2.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [258]:
tdf2.columns = ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀']

In [259]:
tdf1.shape, tdf2.shape

((1528557, 83), (1528557, 83))

In [260]:
total = pd.concat([tdf1, tdf2], ignore_index=True)
total.shape

(3057114, 83)

In [261]:
total.shape[0] == tdf1.shape[0] + tdf2.shape[0]

True

### [출력] 통합 제원, 정기 concat 정밀

In [262]:
# about 1m 44s
total.to_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630.csv')

## [로드] 제원, 정기 concat 정밀(total)

In [7]:
# 약 8s
total = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3288\1756450304.py:2: DtypeWarning: Columns (14,17,20,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  total = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630.csv', index_col=0)


## 검사판정시간으로 정렬

In [263]:
# 6개 index : 6, 1528566
total.sort_values(['차대번호', '검사방법', '검사판정시간']).head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,018013212A7-01,경기65가1508,4790036046,주민,승용,자가용,018013212A7,19900118,1980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정기
1528557,018013212A7-01,경기65가1508,4790036046,주민,승용,자가용,018013212A7,19900118,1980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정밀
1,01831300012-01,01조2678,1144010100,주민,승용,자가용,01831300012,19880823,1988,19880823.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정기
1528558,01831300012-01,01조2678,1144010100,주민,승용,자가용,01831300012,19880823,1988,19880823.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정밀
2,4199경기53라2586,경기53라2586,4111113700,주민,승용,자가용,01841300699,19890328,1989,19890328.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정기


In [264]:
total.loc[1, '차량제작일자']

19880823.0

In [265]:
type(total.loc[1, '차량제작일자'])

numpy.float64

In [266]:
total.loc[13, '검사판정시간']

'20210706 15:59:05'

In [267]:
total.loc[13, '검사판정시간'] < '20221212'

True

In [268]:
total.loc[13, '총중량']

2765.0

In [269]:
type(total.loc[13, '총중량'])

numpy.float64

In [270]:
total.loc[0, '차량제작일자']

nan

In [271]:
total.loc[0, '차량제작일자']

nan

In [272]:
total.shape

(3057114, 83)

### 차대번호, 검사방법, 검사판정시간 정렬 차대번호 기준 중복 제거

In [273]:
d_total = total.sort_values(['차대번호', '검사방법', '검사판정시간']).drop_duplicates('차대번호', keep='first').reset_index(drop=True)
d_total.shape

(1528557, 83)

In [274]:
d_total['배출가스인증번호'].isnull().sum()

123003

In [275]:
uni = d_total['배출가스인증번호'].unique()
len(uni)

301

In [276]:
uni = [x for x in uni if str(x) != 'nan']
len(uni)

300

In [277]:
d_total['검사방법'].unique()

array([nan, '무부하검사(TSI)', '부하검사(ASM-Idling)', '부하검사(KD-147)',
       '무부하검사(정지가동)', '무부하검사(급가속)', '부하검사(LUG DOWN)'], dtype=object)

In [278]:
d_total['검사방법'].isnull().sum()

323516

In [279]:
d_total['연료'].unique()

array(['휘발유', 'LPG(액화석유가스)', '경유', '알코올', '기타연료', 'CNG(압축천연가스)'],
      dtype=object)

In [280]:
str(np.nan)

'nan'

In [281]:
str(np.nan) == 'nan'

True

In [282]:
d_total.shape

(1528557, 83)

### 측정치, 허용치 float 형태로 타입 변환

In [283]:
d_total['무부하매연측정치1'] = pd.to_numeric(d_total['무부하매연측정치1'], errors='coerce')
d_total['무부하매연측정치2'] = pd.to_numeric(d_total['무부하매연측정치2'], errors='coerce')
d_total['무부하매연측정치3'] = pd.to_numeric(d_total['무부하매연측정치3'], errors='coerce')
d_total['무부하매연측정치4'] = pd.to_numeric(d_total['무부하매연측정치4'], errors='coerce')
d_total['무부하매연측정치5'] = pd.to_numeric(d_total['무부하매연측정치5'], errors='coerce')
d_total['무부하매연측정치6'] = pd.to_numeric(d_total['무부하매연측정치6'], errors='coerce')

d_total[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528557 entries, 0 to 1528556
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   무부하매연측정치1  1203560 non-null  float64
 1   무부하매연측정치2  347567 non-null   float64
 2   무부하매연측정치3  317220 non-null   float64
 3   무부하매연측정치4  315988 non-null   float64
 4   무부하매연측정치5  315988 non-null   float64
 5   무부하매연측정치6  296808 non-null   float64
dtypes: float64(6)
memory usage: 70.0 MB


### [출력] d_total

In [284]:
# about 53s
d_total.to_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630(d_total).csv')

In [285]:
d_total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

### [로드] d_total

In [3]:
d_total = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630(d_total).csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_9684\2691981318.py:1: DtypeWarning: Columns (14,17,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  d_total = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630(d_total).csv', index_col=0)


In [286]:
d_total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [287]:
d_total.shape

(1528557, 83)

## [EG] 경유(d_total2)

In [288]:
d_total2 = d_total[d_total['연료'] == '경유'].reset_index(drop=True).dropna(subset=['검사방법', '배출가스인증번호'])
d_total2.shape

(1132386, 83)

In [289]:
uni2 = d_total2['배출가스인증번호'].unique()
len(uni2)

204

In [290]:
# nan 값 존재 여부 확인
for i in uni2:
    if str(i) == 'nan':
        print(i)

In [291]:
d_total2['배출가스인증번호'].unique()[3]

'7MY-HD-14-46'

In [292]:
d_total2['배출가스인증번호'].value_counts()

7MY-HD-14-68     261128
8MY-KM-13-19      90562
6MY-KM-14-61      75495
9MY-KM-14-41      66558
7MY-HD-14-46      59127
                  ...  
9MY-PP-14-409         1
8MY-SN-0M-24          1
8MY-FM-0M-24          1
8MY-ET-0M-24          1
7MY-KL-0M-24          1
Name: 배출가스인증번호, Length: 204, dtype: int64

In [293]:
d_total2['검사방법'].unique()

array(['부하검사(KD-147)', '무부하검사(급가속)', '부하검사(LUG DOWN)', '무부하검사(TSI)',
       '무부하검사(정지가동)', '부하검사(ASM-Idling)'], dtype=object)

In [294]:
temp = d_total2[d_total2['배출가스인증번호'] == '7MY-HD-14-68'].reset_index(drop=True)
temp.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,KMFZCS7JP8U392608-01,75수4549,1162010200,주민,승합,자가용,KM99H4ACD8CZ00001,20080722,2008,20080705.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20200304 11:08:28,정밀
1,KMF2CY7JABU693781-01,88머8072,4421025000,법인,화물,자가용,KMF2CY7JABU693781,20101124,2011,20101118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191216 11:09:20,정기
2,KMFACS7JP9U482553-01,80서6659,4128511300,주민,화물,자가용,KMFACS7JP9U482553,20090105,2009,20081215.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200104 12:10:22,정밀
3,KMFACY7JABU652019-01,86우4807,4831011000,주민,화물,자가용,KMFACY7JABU652019,20100709,2011,20100706.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210723 10:02:36,정기
4,KMFACZ7JABU750953-01,89주5354,4677025300,주민,화물,자가용,KMFACZ7JABU750953,20110513,2011,20110429.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190821 10:04:40,정기


In [295]:
temp.shape

(261128, 83)

In [296]:
temp.loc[0]

차량관리번호       KMFZCS7JP8U392608-01
차량등록번호                    75수4549
본거지법정동코드               1162010200
소유자구분                          주민
차종등록정보                         승합
                     ...         
무부하매연측정치6                     0.0
무부하매연허용치6                     0.0
무부하매연판정6                      NaN
검사판정시간          20200304 11:08:28
정기정밀                           정밀
Name: 0, Length: 83, dtype: object

In [297]:
temp.loc[0]['검사방법']

'부하검사(KD-147)'

In [298]:
temp['검사방법'][0]

'부하검사(KD-147)'

In [299]:
temp['무부하매연측정치1'][0]

0.0

In [300]:
pd.Series([1, 2])

0    1
1    2
dtype: int64

In [301]:
pd.Series([1, 2]).shape

(2,)

In [302]:
car_num = d_total2['차대번호'].unique()
len(car_num)

1132386

In [303]:
d_total2[(d_total2['무부하매연측정치1'].isnull() == True) & (d_total2['무부하매연측정치1'].isnull() == True) & (d_total2['무부하매연측정치2'].isnull() == True) & (d_total2['무부하매연측정치3'].isnull() == True) & (d_total2['무부하매연측정치4'].isnull() == True) & (d_total2['무부하매연측정치5'].isnull() == True) & (d_total2['무부하매연측정치6'].isnull() == True)].shape

(1390, 83)

In [304]:
d_total2[d_total2['무부하매연측정치1'].isnull() == True].shape

(1390, 83)

In [305]:
d_total2['배출가스인증번호'].isnull().sum()

0

In [306]:
temp

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,KMFZCS7JP8U392608-01,75수4549,1162010200,주민,승합,자가용,KM99H4ACD8CZ00001,20080722,2008,20080705.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20200304 11:08:28,정밀
1,KMF2CY7JABU693781-01,88머8072,4421025000,법인,화물,자가용,KMF2CY7JABU693781,20101124,2011,20101118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191216 11:09:20,정기
2,KMFACS7JP9U482553-01,80서6659,4128511300,주민,화물,자가용,KMFACS7JP9U482553,20090105,2009,20081215.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200104 12:10:22,정밀
3,KMFACY7JABU652019-01,86우4807,4831011000,주민,화물,자가용,KMFACY7JABU652019,20100709,2011,20100706.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210723 10:02:36,정기
4,KMFACZ7JABU750953-01,89주5354,4677025300,주민,화물,자가용,KMFACZ7JABU750953,20110513,2011,20110429.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190821 10:04:40,정기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261123,KNFZCD7JP9U509805-01,94우4342,4580025024,주민,화물,자가용,KNFZCD7JP9U509805,20090417,2009,20090415.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200414 15:22:19,정기
261124,KNFZCD7JP9U523649-01,92수3438,2920010100,주민,화물,자가용,KNFZCD7JP9U523649,20090611,2009,20090530.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200511 14:56:01,정밀
261125,KNFZCZ7JAAU607457-01,81두7756,4571025626,주민,화물,자가용,KNFZCZ7JAAU607457,20100302,2010,20100130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220318 10:35:50,정기
261126,KNFZCZ7JBAU579855-01,경북88자3118,4723031024,주민,화물,영업용,KNFZCZ7JBAU579855,20091111,2010,20091110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191129 15:26:53,정기


In [307]:
temp['무부하매연측정치6']

0         0.0
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
261123    NaN
261124    NaN
261125    NaN
261126    NaN
261127    NaN
Name: 무부하매연측정치6, Length: 261128, dtype: float64

In [308]:
test = pd.DataFrame({1:[1,2,3], 2:[1, np.nan, 3], 3:[1, 2, 's']})
test

,1,2,3
0,1,1.0,1
1,2,NaN,2
2,3,3.0,s


In [309]:
test.loc[1]

1      2
2    NaN
3      2
Name: 1, dtype: object

In [310]:
test.loc[1].mean()

2.0

In [311]:
one = '7MY-KM-14-63'

In [312]:
# 1개 테스트
btemp = d_total2[d_total2['배출가스인증번호'] == one].reset_index(drop=True)
# 1개의 배출가스 인증번호에 대한 처리 테스트
g_mea_list = [] # 샘플 1개에 대한 측정치 저장 리스트
p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트
gas_mea_value_p = re.compile('\d[.]\d') # 값이 소수형인지 확인
for i in range(btemp.shape[0]):
    temp = btemp.loc[i]

    test_method = temp['검사방법'] # 변수 생성(검사방법)

    product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

    test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

    total_weight = temp['총중량'] # 변수 생성(총중량)

    ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
    ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

    # 변수 생성(측정치, 문자열)
    sv1 = str(temp['무부하매연측정치1'])
    sv2 = str(temp['무부하매연측정치2'])
    sv3 = str(temp['무부하매연측정치3'])
    sv4 = str(temp['무부하매연측정치4'])
    sv5 = str(temp['무부하매연측정치5'])
    sv6 = str(temp['무부하매연측정치6'])

    # 변수 생성(측정치, 문자열)
    v1 = temp['무부하매연측정치1']
    v2 = temp['무부하매연측정치2']
    v3 = temp['무부하매연측정치3']
    v4 = temp['무부하매연측정치4']
    v5 = temp['무부하매연측정치5']
    v6 = temp['무부하매연측정치6']

    # 변수 생성(판정)
    f1 = temp['무부하매연판정1']
    f2 = temp['무부하매연판정2']
    f3 = temp['무부하매연판정3']
    f4 = temp['무부하매연판정4']
    f5 = temp['무부하매연판정5']
    f6 = temp['무부하매연판정6']
    
    # 검사판정시간
    date = temp['검사판정시간']

    # 평균 매연, 불합격률 계산
    if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
        if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
            # print('1번째 값 부재')
            g = np.nan
            f = np.nan
        else: # 1번째 값 존재
            g = v1
            f = f1

    elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
        if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
            # print('1번째 값 부재')
            g = np.nan
            f = np.nan
        else: # 1번째 값 존재
            g = v1
            f = f1
    else: # 무부하 검사
        if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
            # print('6번째 값 부재')
            if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                # print('5번째 값 부재')
                if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                    # print('4번째 값 부재')
                    if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                        # print('3번째 값 부재')
                        if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                            # print('1번째 값 부재')
                            g = np.nan
                            f = np.nan
                        else: # 1번째 값 존재
                            g = v1
                            f = f1                                
                    else: # 3번째 값 존재
                        g = temp[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].mean()
                        f = f3
                else: # 4번째 값 존재
                    g = temp[['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4']].mean()
                    f = f4
            else: # 5번째 값 존재
                g = temp[['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5']].mean()
                f = f5
        else: # 6번째 값 존재
            g = temp[['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].mean()
            f = f6

    g_mea_list.append(g)
    p_f_list.append(f)

# df 생성
one_sample = pd.DataFrame({'측정치':g_mea_list, '판정':p_f_list})
    
# Grade 계산
m = one_sample.loc[one_sample['판정'] == 'Y', '측정치'].mean()
p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
if p_f_rate_df.shape[0] == 1:
    p_f_rate = 0
else:
    p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

In [313]:
btemp.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,1J4GAE4509L702033-01,59우4900,1171011200,주민,승용,자가용,1J4GAE4509L702033,20081127,2009,20081127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201116 14:27:44,정밀
1,1J4GAE4509L732357-01,40루8356,1168010600,주민,승용,자가용,1J4GAE4509L732357,20090522,2009,20090326.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20210428 10:30:49,정밀
2,1J4GAE4509L732360-01,24모3924,4159013400,주민,승용,자가용,1J4GAE4509L732360,20090119,2009,20090114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210209 11:04:05,정밀
3,1J4GAE4509L739096-01,18나4858,2714010300,주민,승용,자가용,1J4GAE4509L739096,20090720,2009,20090720.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20210624 13:43:48,정밀
4,1J4GAE4509L739101-01,65도3933,2726010500,주민,승용,자가용,1J4GAE4509L739101,20090703,2009,20090128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190705 17:19:03,정밀


In [314]:
one_sample

,측정치,판정
0,0.0,Y
1,0.0,Y
2,0.0,Y
3,0.0,Y
4,0.0,Y
...,...,...
21679,11.0,Y
21680,4.0,Y
21681,0.0,Y
21682,12.0,Y


In [315]:
one_sample.loc[one_sample['판정'] == 'Y', '측정치']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
21679    11.0
21680     4.0
21681     0.0
21682    12.0
21683    15.0
Name: 측정치, Length: 19441, dtype: float64

In [316]:
m = one_sample.loc[one_sample['판정'] == 'Y', '측정치'].mean()
m

8.231315261560619

In [317]:
p_f_rate_df

,index,판정
0,Y,0.897139
1,N,0.102861


In [318]:
p_f_rate_df.shape[0]

2

In [319]:
p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정']

1    0.102861
Name: 판정, dtype: float64

In [320]:
p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

0.10286109829257037

In [321]:
p_f_rate

0.10286109829257037

### total 계산

In [322]:
uni2 = d_total2['배출가스인증번호'].unique()
len(uni2)

204

In [323]:
# nan값 있는지 확인
for x in uni2:
    if str(x) == 'nan':
        print(x)

In [324]:
emno_list = [] # 배출가스인증번호 저장 리스트
car_num_list = [] # 차대번호 저장 리스트
total_point_list = [] # total 점수 저장 리스트
gas_expose_vol_list = [] # 평균 매연 저장 리스트
fail_rate_list = [] # 불량률 저장 리스트
gas_mea_value_p = re.compile('\d[.]\d') # 값이 소수형인지 확인

for one in tqdm(uni2):
    btemp = d_total2[d_total2['배출가스인증번호'] == one].reset_index(drop=True)
    # 1개의 배출가스 인증번호에 대한 처리 테스트
    g_mea_list = [] # 샘플 1개에 대한 측정치 저장 리스트
    p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트
    for i in range(btemp.shape[0]):
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])
        sv2 = str(temp['무부하매연측정치2'])
        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(측정치, 문자열)
        v1 = temp['무부하매연측정치1']
        v2 = temp['무부하매연측정치2']
        v3 = temp['무부하매연측정치3']
        v4 = temp['무부하매연측정치4']
        v5 = temp['무부하매연측정치5']
        v6 = temp['무부하매연측정치6']

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']
        f2 = temp['무부하매연판정2']
        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        # 평균 매연, 불합격률 계산
        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1
        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                g = np.nan
                                f = np.nan
                            else: # 1번째 값 존재
                                g = v1
                                f = f1                                
                        else: # 3번째 값 존재
                            g = temp[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].mean()
                            f = f3
                    else: # 4번째 값 존재
                        g = temp[['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4']].mean()
                        f = f4
                else: # 5번째 값 존재
                    g = temp[['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5']].mean()
                    f = f5
            else: # 6번째 값 존재
                g = temp[['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].mean()
                f = f6

        g_mea_list.append(g)
        p_f_list.append(f)

    # df 생성
    one_sample = pd.DataFrame({'측정치':g_mea_list, '판정':p_f_list})
        
    # 배인 번호 1개당 매연 평균, 불량률 계산
    m = one_sample.loc[one_sample['판정'] == 'Y', '측정치'].mean()
    p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
    if p_f_rate_df.shape[0] == 1 or p_f_rate_df.shape[0] == 0:
        p_f_rate = 0
    else:
        p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

    for i in range(btemp.shape[0]): # 배인번호 1의 df의 한줄씩 처리
        car_num = btemp.loc[i, '차대번호'] # 샘플의 차대번호
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])
        sv2 = str(temp['무부하매연측정치2'])
        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']
        f2 = temp['무부하매연판정2']
        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0 
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(total_weight):
                    std = 0
                elif total_weight <= 5500:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 70
                    elif 19960101 <= product_date <= 20001231:
                        std = 60
                    elif 20010101 <= product_date <= 20071231:
                        std = 50
                    elif 20080101 <= product_date <= 20160831:
                        std = 20
                    elif 20160901 <= product_date: # 2023.01.01 시행규칙 변경사항
                        std = 8
                else:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 50
                    elif 19960101 <= product_date <= 20001231:
                        std = 45
                    elif 20010101 <= product_date <= 20071231:
                        std = 30
                    elif 20080101 <= product_date <= 20160831:
                        std = 15
                    elif 20160901 <= product_date:
                        std = 8 # 2023.01.01 시행규칙 변경사항

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(product_date):
                    std = 0
                elif product_date < 19921231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 50
                    else:
                        std = 45
                elif 19930101 <= product_date <= 19951231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 45
                    else:
                        std = 40
                elif 19960101 <= product_date <= 20001231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 40
                    else:
                        std = 35
                elif 20010101 <= product_date <= 20071231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 30
                    else:
                        std = 25
                elif 20080101 <= product_date <= 20160831:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 20
                    else:
                        std = 15
                elif 20160901 <= product_date <= 20171231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8
                elif 20180101 <= product_date:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8

        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                std = 0
                            else: # 1번째 값 존재
                                # 매연 허용치
                                if str(ci) == 'nan' or str(ci2) == 'nan':
                                    std = 0
                                elif ci == '경형' or ci2 == '승용':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                else:
                                    if ci2 == '소형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19951231:
                                            std = 60
                                        elif 19960101 <= product_date <= 20001231:
                                            std = 55
                                        elif 20010101 <= product_date <= 20031231:
                                            std = 45
                                        elif 20040101 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '중형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '대형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date: # 2023.01.01 시행규칙 변경사항
                                            std = 10
                                    
                        else: # 3번째 값 존재
                            # 매연 허용치
                            if str(ci) == 'nan' or str(ci2) == 'nan':
                                std = 0
                            elif ci == '경형' or ci2 == '승용':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            else:
                                if ci2 == '소형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '중형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '대형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date:
                                        std = 20
                    else: # 4번째 값 존재
                        # 매연 허용치
                        if str(ci) == 'nan' or str(ci2) == 'nan':
                            std = 0
                        elif ci == '경형' or ci2 == '승용':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        else:
                            if ci2 == '소형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '중형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '대형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date:
                                    std = 20
                else: # 5번째 값 존재
                    # 매연 허용치
                    if str(ci) == 'nan' or str(ci2) == 'nan':
                        std = 0
                    elif ci == '경형' or ci2 == '승용':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    else:
                        if ci2 == '소형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '중형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '대형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date:
                                std = 20
            else: # 6번째 값 존재
                # 매연 허용치
                if str(ci) == 'nan' or str(ci2) == 'nan':
                    std = 0
                elif ci == '경형' or ci2 == '승용':
                    if np.isnan(product_date):
                        std = 0
                    elif product_date < 19951231:
                        std = 60
                    elif 19960101 <= product_date <= 20001231:
                        std = 55
                    elif 20010101 <= product_date <= 20031231:
                        std = 45
                    elif 20040101 <= product_date <= 20071231:
                        std = 40
                    elif 20080101 <= product_date <= 20160831:
                        std = 20
                    elif 20160901 <= product_date:
                        std = 10
                else:
                    if ci2 == '소형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '중형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '대형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date:
                            std = 20

        # Total 값 계산
        if std == 0:
            gas_point = 100 - m*100
        else:
            gas_point = 100 - m*100/std

        fail_point = 100 - p_f_rate * 100
        t = gas_point*0.7 + fail_point*0.3

        # 리스트에 저장
        gas_expose_vol_list.append(m)
        fail_rate_list.append(p_f_rate)
        emno_list.append(one)
        car_num_list.append(car_num)
        total_point_list.append(t)

100%|██████████| 204/204 [08:21<00:00,  2.46s/it]


In [325]:
len(car_num_list), len(emno_list), len(gas_expose_vol_list), len(fail_rate_list), len(total_point_list)

(1132386, 1132386, 1132386, 1132386, 1132386)

In [326]:
emno_total_df = pd.DataFrame({'차대번호':car_num_list, '배출가스인증번호':emno_list, '평균매연':gas_expose_vol_list, '불량률':fail_rate_list, 'total':total_point_list})

In [327]:
emno_total_df.head()

,차대번호,배출가스인증번호,평균매연,불량률,total
0,1J4GAE4509L702033,7MY-KM-14-63,8.231315,0.102861,58.501362
1,1J4GAE4509L732357,7MY-KM-14-63,8.231315,0.102861,68.104564
2,1J4GAE4509L732360,7MY-KM-14-63,8.231315,0.102861,58.501362
3,1J4GAE4509L739096,7MY-KM-14-63,8.231315,0.102861,58.501362
4,1J4GAE4509L739101,7MY-KM-14-63,8.231315,0.102861,58.501362


In [328]:
d_total2.shape

(1132386, 83)

In [329]:
d_total2['배출가스인증번호'].isnull().sum()

0

In [330]:
d_total2[d_total2['차대번호'] == '1J4GAE4509L702033']

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
1312,1J4GAE4509L702033-01,59우4900,1171011200,주민,승용,자가용,1J4GAE4509L702033,20081127,2009,20081127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201116 14:27:44,정밀


In [331]:
emno_total_df.isnull().sum()

차대번호        0
배출가스인증번호    0
평균매연        1
불량률         0
total       1
dtype: int64

In [332]:
emno_total_df[emno_total_df['평균매연'].isnull() == True]

,차대번호,배출가스인증번호,평균매연,불량률,total
400309,KL3C6KHF18K000578,8MY-DH-0M-24,NaN,0.0,NaN


In [333]:
emno_total_df[emno_total_df['차대번호'] == 'KL3C6KHF18K000578']

,차대번호,배출가스인증번호,평균매연,불량률,total
400309,KL3C6KHF18K000578,8MY-DH-0M-24,NaN,0.0,NaN


In [334]:
d_total2[d_total2['차대번호'] == 'KL3C6KHF18K000578']

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
2795,KL3C6KHF18K000578-01,제주96바9114,5011011300,주민,화물,영업용,KL3C6KHF18K000578,20080131,2008,20080115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190730 14:27:22,정기


In [335]:
d_total2_1 = d_total2.merge(emno_total_df, on=['차대번호', '배출가스인증번호'], how='left')
d_total2_1.shape

(1132386, 86)

In [336]:
d_total2_1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사판정시간', '정기정밀', '평균매연', '불량률', 'total'],
      dtype='object')

In [337]:
d_total2_1['total'].dtype

dtype('float64')

In [338]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > 75:
        grade_list.append('A')
    elif 75 >= one > 50:
        grade_list.append('B')
    elif 50 >= one > 25:
        grade_list.append('C')
    elif 25 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('D')

100%|██████████| 1132386/1132386 [00:00<00:00, 1507926.26it/s]


In [339]:
d_total2_1['Grade'] = grade_list

In [340]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
1,A,0.261768
0,B,0.716059
2,C,0.014825
3,D,0.007348


In [341]:
d_total2_1['Grade'].value_counts().reset_index()

,index,Grade
0,B,810855
1,A,296422
2,C,16788
3,D,8321


In [342]:
temp = d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).merge(d_total2_1['Grade'].value_counts().reset_index(), on='index')
temp

,index,Grade_x,Grade_y
0,A,0.261768,296422
1,B,0.716059,810855
2,C,0.014825,16788
3,D,0.007348,8321


In [343]:
temp.columns = ['등급', '비율', '수']
temp

,등급,비율,수
0,A,0.261768,296422
1,B,0.716059,810855
2,C,0.014825,16788
3,D,0.007348,8321


In [344]:
temp.to_excel('analysis/[G4]Grade(mod_rate).xlsx')

#### 4분위수로 등급 조정

In [345]:
q3 = d_total2_1['total'].quantile(q=0.75)
q3

77.81568308250456

In [346]:
q2 = d_total2_1['total'].quantile(q=0.5)
q2

71.88475050632576

In [347]:
q1 = d_total2_1['total'].quantile(q=0.25)
q1

65.2024084868234

In [348]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > q3:
        grade_list.append('A')
    elif q3 >= one > q2:
        grade_list.append('B')
    elif q2 >= one > q1:
        grade_list.append('C')
    elif q1 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('D')

100%|██████████| 1132386/1132386 [00:01<00:00, 1007329.38it/s]


In [349]:
d_total2_1['Grade'] = grade_list
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
2,A,0.248474
1,B,0.250722
3,C,0.179203
0,D,0.321601


In [350]:
d_total2_1['Grade'].value_counts().reset_index()

,index,Grade
0,D,364177
1,B,283914
2,A,281368
3,C,202927


In [351]:
temp = d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).merge(d_total2_1['Grade'].value_counts().reset_index(), on='index')
temp

,index,Grade_x,Grade_y
0,A,0.248474,281368
1,B,0.250722,283914
2,C,0.179203,202927
3,D,0.321601,364177


In [352]:
temp.columns = ['등급', '비율', '수']
temp

,등급,비율,수
0,A,0.248474,281368
1,B,0.250722,283914
2,C,0.179203,202927
3,D,0.321601,364177


In [353]:
temp.to_excel('analysis/[G4]Grade(quantile).xlsx')

#### 매연 점수, 불합격 점수 반영 비율 조정(0.9, 0.1)

In [354]:
emno_list = [] # 배출가스인증번호 저장 리스트
car_num_list = [] # 차대번호 저장 리스트
total_point_list = [] # total 점수 저장 리스트
gas_expose_vol_list = [] # 평균 매연 저장 리스트
fail_rate_list = [] # 불량률 저장 리스트
gas_mea_value_p = re.compile('\d[.]\d') # 값이 소수형인지 확인

for one in tqdm(uni2):
    btemp = d_total2[d_total2['배출가스인증번호'] == one].reset_index(drop=True)
    # 1개의 배출가스 인증번호에 대한 처리 테스트
    g_mea_list = [] # 샘플 1개에 대한 측정치 저장 리스트
    p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트
    for i in range(btemp.shape[0]):
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])
        sv2 = str(temp['무부하매연측정치2'])
        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(측정치, 문자열)
        v1 = temp['무부하매연측정치1']
        v2 = temp['무부하매연측정치2']
        v3 = temp['무부하매연측정치3']
        v4 = temp['무부하매연측정치4']
        v5 = temp['무부하매연측정치5']
        v6 = temp['무부하매연측정치6']

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']
        f2 = temp['무부하매연판정2']
        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        # 평균 매연, 불합격률 계산
        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1
        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                g = np.nan
                                f = np.nan
                            else: # 1번째 값 존재
                                g = v1
                                f = f1                                
                        else: # 3번째 값 존재
                            g = temp[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].mean()
                            f = f3
                    else: # 4번째 값 존재
                        g = temp[['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4']].mean()
                        f = f4
                else: # 5번째 값 존재
                    g = temp[['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5']].mean()
                    f = f5
            else: # 6번째 값 존재
                g = temp[['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].mean()
                f = f6

        g_mea_list.append(g)
        p_f_list.append(f)

    # df 생성
    one_sample = pd.DataFrame({'측정치':g_mea_list, '판정':p_f_list})
        
    # 배인 번호 1개당 매연 평균, 불량률 계산
    m = one_sample.loc[one_sample['판정'] == 'Y', '측정치'].mean()
    p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
    if p_f_rate_df.shape[0] == 1 or p_f_rate_df.shape[0] == 0:
        p_f_rate = 0
    else:
        p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

    for i in range(btemp.shape[0]): # 배인번호 1의 df의 한줄씩 처리
        car_num = btemp.loc[i, '차대번호'] # 샘플의 차대번호
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])
        sv2 = str(temp['무부하매연측정치2'])
        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']
        f2 = temp['무부하매연판정2']
        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0 
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(total_weight):
                    std = 0
                elif total_weight <= 5500:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 70
                    elif 19960101 <= product_date <= 20001231:
                        std = 60
                    elif 20010101 <= product_date <= 20071231:
                        std = 50
                    elif 20080101 <= product_date:
                        std = 20
                else:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 50
                    elif 19960101 <= product_date <= 20001231:
                        std = 45
                    elif 20010101 <= product_date <= 20071231:
                        std = 30
                    elif 20080101 <= product_date:
                        std = 15

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(product_date):
                    std = 0
                elif product_date < 19921231:
                    if str(test_date) == 'nan':
                        std = 45
                    elif test_date <= '20111231':
                        std = 50
                    else:
                        std = 45
                elif 19930101 <= product_date <= 19951231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 45
                    else:
                        std = 40
                elif 19960101 <= product_date <= 20001231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 40
                    else:
                        std = 35
                elif 20010101 <= product_date <= 20071231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 30
                    else:
                        std = 25
                elif 20080101 <= product_date <= 20160831:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 20
                    else:
                        std = 15
                elif 20160901 <= product_date <= 20171231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8
                elif 20180101 <= product_date:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8

        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                std = 0
                            else: # 1번째 값 존재
                                # 매연 허용치
                                if str(ci) == 'nan' or str(ci2) == 'nan':
                                    std = 0
                                elif ci == '경형' or ci2 == '승용':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                else:
                                    if ci2 == '소형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19951231:
                                            std = 60
                                        elif 19960101 <= product_date <= 20001231:
                                            std = 55
                                        elif 20010101 <= product_date <= 20031231:
                                            std = 45
                                        elif 20040101 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '중형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '대형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date:
                                            std = 20
                                    
                        else: # 3번째 값 존재
                            # 매연 허용치
                            if str(ci) == 'nan' or str(ci2) == 'nan':
                                std = 0
                            elif ci == '경형' or ci2 == '승용':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            else:
                                if ci2 == '소형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '중형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '대형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date:
                                        std = 20
                    else: # 4번째 값 존재
                        # 매연 허용치
                        if str(ci) == 'nan' or str(ci2) == 'nan':
                            std = 0
                        elif ci == '경형' or ci2 == '승용':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        else:
                            if ci2 == '소형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '중형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '대형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date:
                                    std = 20
                else: # 5번째 값 존재
                    # 매연 허용치
                    if str(ci) == 'nan' or str(ci2) == 'nan':
                        std = 0
                    elif ci == '경형' or ci2 == '승용':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    else:
                        if ci2 == '소형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '중형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '대형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date:
                                std = 20
            else: # 6번째 값 존재
                # 매연 허용치
                if str(ci) == 'nan' or str(ci2) == 'nan':
                    std = 0
                elif ci == '경형' or ci2 == '승용':
                    if np.isnan(product_date):
                        std = 0
                    elif product_date < 19951231:
                        std = 60
                    elif 19960101 <= product_date <= 20001231:
                        std = 55
                    elif 20010101 <= product_date <= 20031231:
                        std = 45
                    elif 20040101 <= product_date <= 20071231:
                        std = 40
                    elif 20080101 <= product_date <= 20160831:
                        std = 20
                    elif 20160901 <= product_date:
                        std = 10
                else:
                    if ci2 == '소형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '중형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '대형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date:
                            std = 20

        # Total 값 계산
        if std == 0:
            gas_point = 100 - m*100
        else:
            gas_point = 100 - m*100/std

        fail_point = 100 - p_f_rate * 100
        t = gas_point*0.9 + fail_point*0.1

        # 리스트에 저장
        gas_expose_vol_list.append(m)
        fail_rate_list.append(p_f_rate)
        emno_list.append(one)
        car_num_list.append(car_num)
        total_point_list.append(t)

100%|██████████| 204/204 [08:17<00:00,  2.44s/it]


In [355]:
len(car_num_list), len(emno_list), len(gas_expose_vol_list), len(fail_rate_list), len(total_point_list)

(1132386, 1132386, 1132386, 1132386, 1132386)

In [356]:
emno_total_df = pd.DataFrame({'차대번호':car_num_list, '배출가스인증번호':emno_list, '평균매연':gas_expose_vol_list, '불량률':fail_rate_list, 'total':total_point_list})

In [357]:
emno_total_df.head()

,차대번호,배출가스인증번호,평균매연,불량률,total
0,1J4GAE4509L702033,7MY-KM-14-63,8.231315,0.102861,49.583497
1,1J4GAE4509L732357,7MY-KM-14-63,8.231315,0.102861,61.930470
2,1J4GAE4509L732360,7MY-KM-14-63,8.231315,0.102861,49.583497
3,1J4GAE4509L739096,7MY-KM-14-63,8.231315,0.102861,49.583497
4,1J4GAE4509L739101,7MY-KM-14-63,8.231315,0.102861,49.583497


In [358]:
d_total2.shape

(1132386, 83)

In [359]:
d_total2['배출가스인증번호'].isnull().sum()

0

In [360]:
d_total2[d_total2['차대번호'] == '1J4GAE4509L702033']

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
1312,1J4GAE4509L702033-01,59우4900,1171011200,주민,승용,자가용,1J4GAE4509L702033,20081127,2009,20081127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201116 14:27:44,정밀


In [361]:
emno_total_df.isnull().sum()

차대번호        0
배출가스인증번호    0
평균매연        1
불량률         0
total       1
dtype: int64

In [362]:
emno_total_df[emno_total_df['평균매연'].isnull() == True]

,차대번호,배출가스인증번호,평균매연,불량률,total
400309,KL3C6KHF18K000578,8MY-DH-0M-24,NaN,0.0,NaN


In [363]:
emno_total_df[emno_total_df['차대번호'] == 'KL3C6KHF18K000578']

,차대번호,배출가스인증번호,평균매연,불량률,total
400309,KL3C6KHF18K000578,8MY-DH-0M-24,NaN,0.0,NaN


In [364]:
d_total2[d_total2['차대번호'] == 'KL3C6KHF18K000578']

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
2795,KL3C6KHF18K000578-01,제주96바9114,5011011300,주민,화물,영업용,KL3C6KHF18K000578,20080131,2008,20080115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190730 14:27:22,정기


In [365]:
d_total2_1 = d_total2.merge(emno_total_df, on=['차대번호', '배출가스인증번호'], how='left')
d_total2_1.shape

(1132386, 86)

In [366]:
d_total2_1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사판정시간', '정기정밀', '평균매연', '불량률', 'total'],
      dtype='object')

In [367]:
d_total2_1['total'].dtype

dtype('float64')

In [368]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > 75:
        grade_list.append('A')
    elif 75 >= one > 50:
        grade_list.append('B')
    elif 50 >= one > 25:
        grade_list.append('C')
    elif 25 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('D')

100%|██████████| 1132386/1132386 [00:00<00:00, 1537824.55it/s]


In [369]:
d_total2_1['Grade'] = grade_list

In [370]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
1,A,0.259266
0,B,0.654423
2,C,0.076691
3,D,0.009620


In [371]:
d_total2_1['Grade'].value_counts().reset_index()

,index,Grade
0,B,741060
1,A,293589
2,C,86844
3,D,10893


In [372]:
temp = d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).merge(d_total2_1['Grade'].value_counts().reset_index(), on='index')
temp

,index,Grade_x,Grade_y
0,A,0.259266,293589
1,B,0.654423,741060
2,C,0.076691,86844
3,D,0.009620,10893


In [373]:
temp.columns = ['등급', '비율', '수']
temp

,등급,비율,수
0,A,0.259266,293589
1,B,0.654423,741060
2,C,0.076691,86844
3,D,0.009620,10893


In [374]:
temp.to_excel('analysis/[G4]Grade(mod_rate)(반영비율조정).xlsx')

### 배출가스등급 코드화

In [165]:
d_total2_1['배출가스등급'] = 4

### [출력] d_total2_1

In [166]:
d_total2_1.to_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630(d_total2_1).csv')

### [로드] d_total2_1

In [94]:
d_total2_1 = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630(d_total2_1).csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_21172\3468532723.py:1: DtypeWarning: Columns (14,17,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  d_total2_1 = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630(d_total2_1).csv', index_col=0)


In [167]:
d_total2_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132386 entries, 0 to 1132385
Data columns (total 35 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   차대번호       1132386 non-null  object 
 1   차량제작일자     1132385 non-null  float64
 2   배출가스인증번호   1132386 non-null  object 
 3   총중량        1132386 non-null  float64
 4   검사방법       1132386 non-null  object 
 5   차종유형       1132386 non-null  object 
 6   차종등록정보     1132386 non-null  object 
 7   연료         1132386 non-null  object 
 8   적재중량       1132180 non-null  float64
 9   승차인원       1132386 non-null  int64  
 10  무부하매연측정치1  1130996 non-null  float64
 11  무부하매연허용치1  1130983 non-null  float64
 12  무부하매연판정1   1130983 non-null  object 
 13  무부하매연측정치2  279067 non-null   float64
 14  무부하매연허용치2  249392 non-null   float64
 15  무부하매연판정2   18814 non-null    object 
 16  무부하매연측정치3  249397 non-null   float64
 17  무부하매연허용치3  249390 non-null   float64
 18  무부하매연판정3   18386 non-null    object 
 19  

In [168]:
d_total2_1.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '적재중량', '승차인원', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2',
       '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3',
       '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5',
       '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀',
       '평균매연', '불량률', 'total', 'Grade', '배출가스등급'],
      dtype='object')

#### [로드] 저감장치 부착 상세내역

In [169]:
# 
dpf = pd.read_excel('data/01. 저감장치 부착 상세 내역_Total.xlsx')
dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993097 entries, 0 to 993096
Data columns (total 55 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   저감등록번호     993097 non-null  int64  
 1   저감등록상태     993096 non-null  object 
 2   시도         993097 non-null  object 
 3   시군구        992311 non-null  object 
 4   차량등록번호     993097 non-null  object 
 5   현재차량등록번호   76290 non-null   object 
 6   차대번호       993097 non-null  object 
 7   차종         993097 non-null  object 
 8   차량용도       993094 non-null  object 
 9   차량연식       993095 non-null  float64
 10  최초등록일자     957898 non-null  float64
 11  소유자명       990870 non-null  object 
 12  사용자전화번호    974774 non-null  object 
 13  부착업체명      983955 non-null  object 
 14  현재시군구      574316 non-null  object 
 15  부착일자       993095 non-null  object 
 16  저감장치구분     993097 non-null  object 
 17  저감장치종류     769194 non-null  object 
 18  인증모델번호     993094 non-null  object 
 19  장치일련번호     865034 non-n

In [170]:
dpf.columns

Index(['저감등록번호', '저감등록상태', '시도', '시군구', '차량등록번호', '현재차량등록번호', '차대번호', '차종',
       '차량용도', '차량연식', '최초등록일자', '소유자명', '사용자전화번호', '부착업체명', '현재시군구', '부착일자',
       '저감장치구분', '저감장치종류', '인증모델번호', '장치일련번호', '검사소명', '구조변경일자', '주행거리',
       '제작사코드', '차명', '입력일자', '입력자명', '접수지자체코드', '보조금대상YN', '보조금대상제외사유',
       '협회확인반려사유', '협회확인승인일자', '협회확인자명', '협회확인기관코드', '보조금신청YN', '보조금신청일자',
       '보조금신청자명', '보조금지급승인일자', '보조금지급승인자명', '보조금지급확정일자', '보조금지급확정자명',
       '보조금지급일자', '보조금액', '보조금자부담액', '보조금사업년도', '총중량', '차량배기량', '엔진형식', '배기온도',
       '최대배기압력', '평균배기압력', '엔진출력', '차량정원', '차량법인구분', '차량연료'],
      dtype='object')

In [ ]:
d_total2_2 = d_total2_1.merge(dpf, how='left', on='차대번호')
d_total2_2.shape

## 연료에 따른 세분류
- '경유':'D', '휘발유':'G', 'LPG(액화석유가스)':'L', 'CNG(압축천연가스)':'C', '알코올':'A' 

In [ ]:
fuel_div_dict = {'경유':'D', '휘발유':'G', 'LPG(액화석유가스)':'L', 'CNG(압축천연가스)':'C', '알코올':'A'}

In [646]:
d_total2['연료'].unique()

array(['경유'], dtype=object)

## [EG] 휘발유/가스(d_total3)

In [72]:
d_total['연료'].unique()

array(['휘발유', 'LPG(액화석유가스)', '경유', '알코올', '기타연료', 'CNG(압축천연가스)'],
      dtype=object)

In [73]:
d_total.shape

(1528557, 30)

In [74]:
d_total[d_total['연료'] == '경유'].shape

(1167605, 30)

In [75]:
d_total3 = d_total[(d_total['연료'] == '휘발유') | (d_total['연료'] == '알코올') | (d_total['연료'] == 'LPG(액화석유가스)') | (d_total['연료'] == 'CNG(압축천연가스)')]
d_total3.shape

(360951, 30)

In [76]:
d_total3.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '적재중량', '승차인원', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2',
       '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3',
       '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5',
       '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [77]:
d_total3['검사방법'].unique()

array([nan, '무부하검사(TSI)', '부하검사(ASM-Idling)', '부하검사(KD-147)',
       '무부하검사(정지가동)', '무부하검사(급가속)'], dtype=object)

### 휘발유/가스

In [12]:
gasoline_df = d_total3.dropna(subset=['검사방법', '배출가스인증번호']).reset_index(drop=True)
gasoline_df.shape

(48530, 28)

In [13]:
gasoline_df['검사방법'].unique()

array(['부하검사(ASM-Idling)', '무부하검사(TSI)', '무부하검사(정지가동)', '무부하검사(급가속)'],
      dtype=object)

In [14]:
gasoline_df['정기정밀'].unique()

array(['정밀', '정기'], dtype=object)

In [15]:
uni_gasoline = gasoline_df['배출가스인증번호'].unique()
len(uni_gasoline)

85

In [16]:
gasoline_df['무부하매연측정치1'] = pd.to_numeric(gasoline_df['무부하매연측정치1'], errors='coerce')
gasoline_df['무부하매연측정치2'] = pd.to_numeric(gasoline_df['무부하매연측정치2'], errors='coerce')
gasoline_df['무부하매연측정치3'] = pd.to_numeric(gasoline_df['무부하매연측정치3'], errors='coerce')
gasoline_df['무부하매연측정치4'] = pd.to_numeric(gasoline_df['무부하매연측정치4'], errors='coerce')
gasoline_df['무부하매연측정치5'] = pd.to_numeric(gasoline_df['무부하매연측정치5'], errors='coerce')
gasoline_df['무부하매연측정치6'] = pd.to_numeric(gasoline_df['무부하매연측정치6'], errors='coerce')

gasoline_df[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48530 entries, 0 to 48529
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   무부하매연측정치1  48493 non-null  float64
 1   무부하매연측정치2  48492 non-null  float64
 2   무부하매연측정치3  48492 non-null  float64
 3   무부하매연측정치4  47858 non-null  float64
 4   무부하매연측정치5  47858 non-null  float64
 5   무부하매연측정치6  47349 non-null  float64
dtypes: float64(6)
memory usage: 2.2 MB


In [17]:
gasoline_df.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [18]:
gasoline_df.head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,1FTDF1721VNC75710,19970820.0,AMY-PPE-14-691,2975.0,부하검사(ASM-Idling),소형,화물,휘발유,0.00,0.49,...,1.2,Y,5.0,220.0,Y,0.99,0.1,Y,20190812 14:45:41,정밀
1,1FTDF1722VNC75716,19971231.0,AMY-PPE-14-631,2675.0,무부하검사(TSI),소형,화물,휘발유,0.00,2.50,...,2.5,Y,0.0,400.0,Y,1.00,0.1,Y,20191031 15:05:37,정기
2,1FTYR10U6WTA78758,20100831.0,AMY-PPS-14-180,2040.0,부하검사(ASM-Idling),소형,화물,휘발유,1.45,0.48,...,1.2,Y,4.0,220.0,Y,1.01,0.1,Y,20191226 15:13:06,정밀
3,1GBDM19W9XB140831,19991231.0,AMY-PPE-14-1051,2595.0,무부하검사(TSI),대형,승용,휘발유,0.00,1.20,...,1.2,Y,47.0,220.0,Y,1.00,0.1,Y,20190730 16:47:39,정기
4,1GBEL19W3XB192382,19991231.0,9MY-PP-14-W38,2570.0,무부하검사(TSI),대형,승용,휘발유,0.40,1.00,...,1.0,Y,53.0,120.0,Y,1.00,0.1,Y,20191113 13:27:27,정밀


In [19]:
gasoline_df['검사방법'].unique()

array(['부하검사(ASM-Idling)', '무부하검사(TSI)', '무부하검사(정지가동)', '무부하검사(급가속)'],
      dtype=object)

In [22]:
gasoline_df[gasoline_df['검사방법'] == '무부하검사(정지가동)'].shape

(652, 28)

In [23]:
gasoline_df[gasoline_df['정기정밀'] == '정밀'].shape

(39767, 28)

In [24]:
gasoline_df[gasoline_df['정기정밀'] == '정밀'].isnull().sum()

차대번호            0
차량제작일자       5477
배출가스인증번호        0
총중량             0
검사방법            0
차종유형            0
차종등록정보          0
연료              0
무부하매연측정치1      19
무부하매연허용치1      20
무부하매연판정1       20
무부하매연측정치2      19
무부하매연허용치2      20
무부하매연판정2       20
무부하매연측정치3      19
무부하매연허용치3      20
무부하매연판정3       20
무부하매연측정치4      19
무부하매연허용치4      20
무부하매연판정4       20
무부하매연측정치5      19
무부하매연허용치5      20
무부하매연판정5       20
무부하매연측정치6     528
무부하매연허용치6      11
무부하매연판정6       11
검사판정시간        528
정기정밀            0
dtype: int64

In [25]:
gasoline_df.isnull().sum()

차대번호            0
차량제작일자       6537
배출가스인증번호        0
총중량             0
검사방법            0
차종유형            0
차종등록정보          0
연료              0
무부하매연측정치1      37
무부하매연허용치1      38
무부하매연판정1       38
무부하매연측정치2      38
무부하매연허용치2      39
무부하매연판정2       39
무부하매연측정치3      38
무부하매연허용치3      39
무부하매연판정3       39
무부하매연측정치4     672
무부하매연허용치4     673
무부하매연판정4      673
무부하매연측정치5     672
무부하매연허용치5     673
무부하매연판정5      673
무부하매연측정치6    1181
무부하매연허용치6     664
무부하매연판정6      664
검사판정시간        528
정기정밀            0
dtype: int64

In [26]:
gasoline_df[gasoline_df['정기정밀'] == '정기']['검사방법'].unique()

array(['무부하검사(TSI)', '무부하검사(정지가동)', '무부하검사(급가속)'], dtype=object)

In [27]:
gasoline_df[gasoline_df['정기정밀'] == '정기'].head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
1,1FTDF1722VNC75716,19971231.0,AMY-PPE-14-631,2675.0,무부하검사(TSI),소형,화물,휘발유,0.00,2.5,...,2.5,Y,0.0,400.0,Y,1.00,0.1,Y,20191031 15:05:37,정기
3,1GBDM19W9XB140831,19991231.0,AMY-PPE-14-1051,2595.0,무부하검사(TSI),대형,승용,휘발유,0.00,1.2,...,1.2,Y,47.0,220.0,Y,1.00,0.1,Y,20190730 16:47:39,정기
14,1MELM53U5RA648808,NaN,3MY-FK-08,1785.0,무부하검사(TSI),대형,승용,휘발유,0.90,1.2,...,1.2,Y,192.0,220.0,Y,1.02,0.1,Y,20190902 11:59:09,정기
18,280134407749,20010821.0,YKM-KM-12,1705.0,무부하검사(정지가동),경형,화물,LPG(액화석유가스),0.04,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190926 11:52:50,정기
33,KL3K8CGFA5K000018,20050323.0,5MY-DC-23-06,26290.0,무부하검사(급가속),대형,화물,CNG(압축천연가스),6.00,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220506 11:43:41,정기


In [28]:
uni = gasoline_df['배출가스인증번호'].unique()
len(uni)

85

In [29]:
gasoline_df.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

### total 계산

In [30]:
emno_list = [] # 배출가스인증번호 저장 리스트
car_num_list = [] # 차대번호 저장 리스트
total_point_list = [] # total 점수 저장 리스트
mean_co_list = [] # 평균 CO 저장 리스트
mean_hc_list = [] # 평균 HC 저장 리스트
mean_nox_list = [] # 평균 NOx 저장 리스트
fail_rate_list = [] # 불량률 저장 리스트
gas_mea_value_p = re.compile('\d[.]\d') # 값이 소수형인지 확인

for one in tqdm(uni):
    btemp = gasoline_df[gasoline_df['배출가스인증번호'] == one].reset_index(drop=True)
    # 1개의 배출가스 인증번호에 대한 처리 테스트
    co_list = [] # 샘플 1개에 대한 CO 측정치 저장 리스트
    hc_list = [] # 샘플 1개에 대한 HC 측정치 저장 리스트
    nox_list = [] # 샘플 1개에 대한 NOx 측정치 저장 리스트
    p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트

    for i in range(btemp.shape[0]):
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        # 변수 생성(측정치, 문자열)
        co1 = temp['무부하매연측정치1'] # CO 측정치 1
        hc1 = temp['무부하매연측정치2'] # HC 측정치 1
        nox1 = temp['무부하매연측정치3'] # NOx 측정치 1 
        co2 = temp['무부하매연측정치4'] # CO 측정치 2
        hc2 = temp['무부하매연측정치5'] # HC 측정치 2
        nox2 = temp['무부하매연측정치6'] # NOx 측정치 2 or 공기과잉 률 측정치

        # 변수 생성(판정)
        fco1 = temp['무부하매연판정1'] # CO 판정 1
        fhc1 = temp['무부하매연판정2'] # HC 판정 1
        fnox1 = temp['무부하매연판정3'] # NOx 판정 1
        fco2 = temp['무부하매연판정4'] # CO 판정 2
        fhc2 = temp['무부하매연판정5'] # HC 판정 2
        fnox2 = temp['무부하매연판정6'] # NOx 판정 2 or 공기과잉 률 판정
        
        # 검사판정시간
        date = temp['검사판정시간']

        # 평균 매연, 불합격률 계산
        if test_method == '부하검사(ASM-Idling)': # CO, HC, NOx 각각의 1번째 측정치, 판정치 사용
            if ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox1) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # Y Y Y
                co, hc, nox = co1, hc1, nox1
                f = fnox1
            elif ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox1) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox1)) ): # Y Y N
                co, hc, nox = co1, hc1, np.nan
                f = fhc1
            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox1) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # Y N Y
                co, hc, nox = co1, np.nan, nox1
                f = fnox1
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox1) != 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # N Y Y 
                co, hc, nox = np.nan, hc1, nox1
                f = fnox1
            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox1) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox1)) ): # Y N N 
                co, hc, nox = co1, np.nan, np.nan
                f = fco1
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox1) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox1)) ): # N Y N 
                co, hc, nox = np.nan, hc1, np.nan
                f = fhc1
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox1) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # N N Y
                co, hc, nox = np.nan, np.nan, nox1
                f = fnox1
            else: # N N N
                co, hc, nox = np.nan, np.nan, np.nan
                f = np.nan

        else: # '무부하검사(TSI)', '무부하검사(정지가동)', '무부하검사(급가속)' # CO(무부하매연측정치1), HC(무부하매연측정치2), 공기과잉률(무부하매연측정치6), 무부하매연판정6 사용
            if ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox2) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # Y Y Y
                co, hc, nox = co1, hc1, nox2
                f = fnox2
            elif ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox2) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox2)) ): # Y Y N
                co, hc, nox = co1, hc1, np.nan
                f = fnox2
            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox2) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # Y N Y
                co, hc, nox = co1, np.nan, nox2
                f = fnox2
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox2) != 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # N Y Y 
                co, hc, nox = np.nan, hc1, nox2
                f = fnox2
            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox2) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox2)) ): # Y N N 
                co, hc, nox = co1, np.nan, np.nan
                f = fnox2
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox2) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox2)) ): # N Y N 
                co, hc, nox = np.nan, hc1, np.nan
                f = fnox2
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox2) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # N N Y
                co, hc, nox = np.nan, np.nan, nox2
                f = fnox2
            else: # N N N
                co, hc, nox = np.nan, np.nan, np.nan
                f = np.nan
 
        co_list.append(co)
        hc_list.append(hc)
        nox_list.append(nox)
        p_f_list.append(f)

    # df 생성
    one_sample = pd.DataFrame({'CO_측정치':co_list, 'HC_측정치':hc_list, 'NOx_측정치(또는_공기과잉률)':nox_list, '판정':p_f_list})
        
    # 배인 번호 1개당 매연 평균, 불량률 계산
    mco = one_sample.loc[one_sample['판정'] == 'Y', 'CO_측정치'].mean()
    mhc = one_sample.loc[one_sample['판정'] == 'Y', 'HC_측정치'].mean()
    mnox = one_sample.loc[one_sample['판정'] == 'Y', 'NOx_측정치(또는_공기과잉률)'].mean()
    p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
    if p_f_rate_df.shape[0] == 1 or p_f_rate_df.shape[0] == 0:
        p_f_rate = 0
    else:
        p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

    for i in range(btemp.shape[0]): # 배인번호 1의 df의 한줄씩 처리
        car_num = btemp.loc[i, '차대번호'] # 샘플의 차대번호
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자'] # 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        fuel = temp['연료'] # 경유, 휘발유, 알코올, LPG(액화석유가스), CNG(압축천연가스)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        load_weight = temp['적재중량']
        load_people = temp['승차인원']

        car_weight = total_weight - load_weight - load_people*65

        # 변수 생성(측정치, 문자열)
        co1 = temp['무부하매연측정치1'] # CO 측정치 1
        hc1 = temp['무부하매연측정치2'] # HC 측정치 1
        nox1 = temp['무부하매연측정치3'] # NOx 측정치 1 
        co2 = temp['무부하매연측정치4'] # CO 측정치 2
        hc2 = temp['무부하매연측정치5'] # HC 측정치 2
        nox2 = temp['무부하매연측정치6'] # NOx 측정치 2 or 공기과잉 률 측정치

        # 
        if test_method == '부하검사(ASM-Idling)': # CO, HC, NOx 각각의 1번째 측정치, 판정치 사용
            if ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox1) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # Y Y Y
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            co_std = 4.2
                            hc_std = 1090
                            nox_std = 2110
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 2.4
                            hc_std = 390
                            nox_std = 1800
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.2
                            hc_std = 210
                            nox_std = 1640
                        elif '20040101' <= product_date <= '20051231':
                            co_std = 1
                            hc_std = 180
                            nox_std = 1240
                        elif '20060101' <= product_date:
                            co_std = 1
                            hc_std = 180
                            nox_std = 1240
                        else:
                            co_std = 0
                            hc_std = 0
                            nox_std = 0
                    else:
                        if product_date < '19971231':
                            co_std = 5.46
                            hc_std = 1410
                            nox_std = 2530
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 3.12
                            hc_std = 500
                            nox_std = 2160
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.56
                            hc_std = 270
                            nox_std = 1960
                        elif '20040101' <= product_date <= '20071231':
                            co_std = 1.3
                            hc_std = 230
                            nox_std = 1490
                        elif '20080101' <= product_date:
                            co_std = 1.3
                            hc_std = 180
                            nox_std = 1490
                        else:
                            co_std = 0
                            hc_std = 0
                            nox_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 1.83
                                hc_std = 210
                                nox_std = 1800
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.47
                                hc_std = 170
                                nox_std = 1440
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.21
                                hc_std = 140
                                nox_std = 1190
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.08
                                hc_std = 130
                                nox_std = 1050
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.94
                                hc_std = 120
                                nox_std = 920
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.87
                                hc_std = 110
                                nox_std = 840
                            elif 2000 <= car_weight:
                                co_std = 0.79
                                hc_std = 100
                                nox_std = 760
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 180
                                nox_std = 1400
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 150
                                nox_std = 1120
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.67
                                hc_std = 120
                                nox_std = 930
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 110
                                nox_std = 820
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 100
                                nox_std = 720
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                                nox_std = 660
                            elif 2000 <= car_weight:
                                co_std = 0.45
                                hc_std = 80
                                nox_std = 600
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                co_std = 0.77
                                hc_std = 140
                                nox_std = 1100
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                                hc_std = 110
                                nox_std = 870
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                                hc_std = 100
                                nox_std = 720
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                                hc_std = 90
                                nox_std = 630
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                                hc_std = 80
                                nox_std = 550
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                                hc_std = 70
                                nox_std = 500
                            elif 2000 <= car_weight:
                                co_std = 0.35
                                hc_std = 60
                                nox_std = 450
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                co_std = 0.77
                                hc_std = 59
                                nox_std = 1080
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                                hc_std = 49
                                nox_std = 850
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                                hc_std = 42
                                nox_std = 690
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                                hc_std = 37
                                nox_std = 610
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                                hc_std = 33
                                nox_std = 530
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                                hc_std = 30
                                nox_std = 490
                            elif 2000 <= car_weight:
                                co_std = 0.35
                                hc_std = 30
                                nox_std = 440

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 2.38
                                hc_std = 273
                                nox_std = 2160
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.91
                                hc_std = 221
                                nox_std = 1728
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.57
                                hc_std = 182
                                nox_std = 1428
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.4
                                hc_std = 169
                                nox_std = 1260
                            elif 1600 <= car_weight < 1800:
                                co_std = 1.22
                                hc_std = 156
                                nox_std = 1104
                            elif 1800 <= car_weight < 2000:
                                co_std = 1.13
                                hc_std = 143
                                nox_std = 1008
                            elif 2000 <= car_weight:
                                co_std = 1.03
                                hc_std = 130
                                nox_std = 912
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1.3
                                hc_std = 234
                                nox_std = 1680
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.05
                                hc_std = 195
                                nox_std = 1344
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.87
                                hc_std = 156
                                nox_std = 1116
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.78
                                hc_std = 143
                                nox_std = 984
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.69
                                hc_std = 130
                                nox_std = 864
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.64
                                hc_std = 117
                                nox_std = 792
                            elif 2000 <= car_weight:
                                co_std = 0.59
                                hc_std = 104
                                nox_std = 720
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 180
                                nox_std = 1320
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 140
                                nox_std = 1040
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                hc_std = 120
                                nox_std = 860
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 110
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 100
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                                nox_std = 580
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                hc_std = 80
                                nox_std = 530
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 180
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 140
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                hc_std = 130
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 120
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 100
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                                nox_std = 590
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                hc_std = 80
                                nox_std = 530
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 59
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 49
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                hc_std = 42
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 37
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 33
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 30
                                nox_std = 590
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                hc_std = 30
                                nox_std = 530

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.2
                                hc_std = 200
                                nox_std = 1260
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.96
                                hc_std = 180
                                nox_std = 1120
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.68
                                hc_std = 150
                                nox_std = 960
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.52
                                hc_std = 140
                                nox_std = 860
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.39
                                hc_std = 130
                                nox_std = 780
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.24
                                hc_std = 120
                                nox_std = 690
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.14
                                hc_std = 110
                                nox_std = 630
                            elif 3000 <= car_weight:
                                co_std = 1.05
                                hc_std = 100
                                nox_std = 580
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 1.84
                                hc_std = 180
                                nox_std = 1120
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.64
                                hc_std = 160
                                nox_std = 1000
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.41
                                hc_std = 140
                                nox_std = 860
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.28
                                hc_std = 130
                                nox_std = 770
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.17
                                hc_std = 120
                                nox_std = 700
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.05
                                hc_std = 110
                                nox_std = 620
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.96
                                hc_std = 100
                                nox_std = 570
                            elif 3000 <= car_weight:
                                co_std = 0.89
                                hc_std = 90
                                nox_std = 520
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.48
                                hc_std = 100
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                                hc_std = 90
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                                hc_std = 80
                                nox_std = 570
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                                hc_std = 80
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                                hc_std = 70
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                                hc_std = 70
                                nox_std = 420
                            elif 3000 <= car_weight:
                                co_std = 0.34
                                hc_std = 60
                                nox_std = 380
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.48
                                hc_std = 100
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                                hc_std = 90
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                                hc_std = 80
                                nox_std = 560
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                                hc_std = 80
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                                hc_std = 70
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                                hc_std = 70
                                nox_std = 420
                            elif 3000 <= car_weight:
                                co_std = 0.34
                                hc_std = 60
                                nox_std = 380
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.6
                                hc_std = 260
                                nox_std = 1510
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.5
                                hc_std = 240
                                nox_std = 1350
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.8
                                hc_std = 200
                                nox_std = 1150
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.8
                                hc_std = 180
                                nox_std = 1030
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.1
                                hc_std = 170
                                nox_std = 940
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.1
                                hc_std = 160
                                nox_std = 830
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.8
                                hc_std = 140
                                nox_std = 750
                            elif 3000 <= car_weight:
                                co_std = 1.7
                                hc_std = 130
                                nox_std = 700
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 2.9
                                hc_std = 240
                                nox_std = 1350
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.3
                                hc_std = 210
                                nox_std = 1200
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.3
                                hc_std = 180
                                nox_std = 1030
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.6
                                hc_std = 170
                                nox_std = 920
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.2
                                hc_std = 160
                                nox_std = 840
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.7
                                hc_std = 140
                                nox_std = 740
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.5
                                hc_std = 130
                                nox_std = 680
                            elif 3000 <= car_weight:
                                co_std = 1.6
                                hc_std = 120
                                nox_std = 630
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.2
                                hc_std = 130
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                                hc_std = 120
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                                hc_std = 120
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                                hc_std = 110
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                                hc_std = 100
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                                hc_std = 90
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                                hc_std = 90
                                nox_std = 500
                            elif 3000 <= car_weight:
                                co_std = 0.4
                                hc_std = 80
                                nox_std = 460
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.2
                                hc_std = 130
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                                hc_std = 120
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                                hc_std = 120
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                                hc_std = 110
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                                hc_std = 100
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                                hc_std = 90
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                                hc_std = 90
                                nox_std = 500
                            elif 3000 <= car_weight:
                                co_std = 0.4
                                hc_std = 80
                                nox_std = 460

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                co_std = 2.51
                                hc_std = 200
                                nox_std = 1950
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.22
                                hc_std = 180
                                nox_std = 1720
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.01
                                hc_std = 160
                                nox_std = 1550
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.82
                                hc_std = 150
                                nox_std = 1400
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.62
                                hc_std = 130
                                nox_std = 1230
                            elif 2750 <= car_weight < 3000:
                                co_std = 1.49
                                hc_std = 120
                                nox_std = 1120
                            elif 3000 <= car_weight <= 3250:
                                co_std = 1.37
                                hc_std = 110
                                nox_std = 1040
                            elif 3250 <= car_weight:
                                co_std = 1.36
                                hc_std = 110
                                nox_std = 1040
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 1.89
                                hc_std = 170
                                nox_std = 1720
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                                hc_std = 150
                                nox_std = 1530
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                                hc_std = 140
                                nox_std = 1370
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                                hc_std = 130
                                nox_std = 1240
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                                hc_std = 110
                                nox_std = 1090
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                                hc_std = 100
                                nox_std = 990
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                                hc_std = 100
                                nox_std = 920
                            elif 3000 <= car_weight:
                                co_std = 1.04
                                hc_std = 100
                                nox_std = 920
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 1.89
                                hc_std = 170
                                nox_std = 1500
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                                hc_std = 150
                                nox_std = 1330
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                                hc_std = 140
                                nox_std = 1190
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                                hc_std = 130
                                nox_std = 1080
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                                hc_std = 110
                                nox_std = 950
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                                hc_std = 100
                                nox_std = 870
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                                hc_std = 100
                                nox_std = 810
                            elif 3000 <= car_weight:
                                co_std = 1.04
                                hc_std = 100
                                nox_std = 810
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 3.26
                                hc_std = 260
                                nox_std = 2340
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.89
                                hc_std = 230
                                nox_std = 2070
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.61
                                hc_std = 210
                                nox_std = 1860
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.37
                                hc_std = 190
                                nox_std = 1680
                            elif 2250 <= car_weight < 2500:
                                co_std = 2.11
                                hc_std = 170
                                nox_std = 1480
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.94
                                hc_std = 160
                                nox_std = 1340
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.78
                                hc_std = 150
                                nox_std = 1250
                            elif 3000 <= car_weight:
                                co_std = 1.77
                                hc_std = 150
                                nox_std = 1240
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 2.46
                                hc_std = 220
                                nox_std = 2070
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                                hc_std = 200
                                nox_std = 1830
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                                hc_std = 180
                                nox_std = 1640
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                                hc_std = 160
                                nox_std = 1480
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                                hc_std = 150
                                nox_std = 1310
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                                hc_std = 140
                                nox_std = 1190
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                                hc_std = 130
                                nox_std = 1110
                            elif 3000 <= car_weight:
                                co_std = 1.35
                                hc_std = 130
                                nox_std = 1110
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 2.46
                                hc_std = 220
                                nox_std = 1800
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                                hc_std = 200
                                nox_std = 1590
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                                hc_std = 180
                                nox_std = 1430
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                                hc_std = 160
                                nox_std = 1290
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                                hc_std = 150
                                nox_std = 1140
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                                hc_std = 140
                                nox_std = 1040
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                                hc_std = 130
                                nox_std = 970
                            elif 3000 <= car_weight:
                                co_std = 1.35
                                hc_std = 130
                                nox_std = 970

            elif ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox1) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox1)) ): # Y Y N
                nox_std = 1
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            co_std = 4.2
                            hc_std = 1090
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 2.4
                            hc_std = 390
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.2
                            hc_std = 210
                        elif '20040101' <= product_date <= '20051231':
                            co_std = 1
                            hc_std = 180
                        elif '20060101' <= product_date:
                            co_std = 1
                            hc_std = 180
                        else:
                            co_std = 0
                            hc_std = 0
                    else:
                        if product_date < '19971231':
                            co_std = 5.46
                            hc_std = 1410
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 3.12
                            hc_std = 500
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.56
                            hc_std = 270
                        elif '20040101' <= product_date <= '20071231':
                            co_std = 1.3
                            hc_std = 230
                        elif '20080101' <= product_date:
                            co_std = 1.3
                            hc_std = 180
                        else:
                            co_std = 0
                            hc_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 1.83
                                hc_std = 210
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.47
                                hc_std = 170
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.21
                                hc_std = 140
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.08
                                hc_std = 130
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.94
                                hc_std = 120
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.87
                                hc_std = 110
                            elif 2000 <= car_weight:
                                co_std = 0.79
                                hc_std = 100
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 180
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 150
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.67
                                hc_std = 120
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 110
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 100
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                            elif 2000 <= car_weight:
                                co_std = 0.45
                                hc_std = 80
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                co_std = 0.77
                                hc_std = 140
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                                hc_std = 110
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                                hc_std = 100
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                                hc_std = 90
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                                hc_std = 80
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                                hc_std = 70
                            elif 2000 <= car_weight:
                                co_std = 0.35
                                hc_std = 60
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                co_std = 0.77
                                hc_std = 59
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                                hc_std = 49
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                                hc_std = 42
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                                hc_std = 37
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                                hc_std = 33
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                                hc_std = 30
                            elif 2000 <= car_weight:
                                co_std = 0.35
                                hc_std = 30

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 2.38
                                hc_std = 273
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.91
                                hc_std = 221
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.57
                                hc_std = 182
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.4
                                hc_std = 169
                            elif 1600 <= car_weight < 1800:
                                co_std = 1.22
                                hc_std = 156
                            elif 1800 <= car_weight < 2000:
                                co_std = 1.13
                                hc_std = 143
                            elif 2000 <= car_weight:
                                co_std = 1.03
                                hc_std = 130
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1.3
                                hc_std = 234
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.05
                                hc_std = 195
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.87
                                hc_std = 156
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.78
                                hc_std = 143
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.69
                                hc_std = 130
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.64
                                hc_std = 117
                            elif 2000 <= car_weight:
                                co_std = 0.59
                                hc_std = 104
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 180
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 140
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                hc_std = 120
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 110
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 100
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                hc_std = 80
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 180
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 140
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                hc_std = 130
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 120
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 100
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                hc_std = 80
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                co_std = 1
                                hc_std = 59
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                hc_std = 49
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                hc_std = 42
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                hc_std = 37
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                hc_std = 33
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 30
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                hc_std = 30

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.2
                                hc_std = 200
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.96
                                hc_std = 180
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.68
                                hc_std = 150
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.52
                                hc_std = 140
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.39
                                hc_std = 130
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.24
                                hc_std = 120
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.14
                                hc_std = 110
                            elif 3000 <= car_weight:
                                co_std = 1.05
                                hc_std = 100
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 1.84
                                hc_std = 180
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.64
                                hc_std = 160
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.41
                                hc_std = 140
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.28
                                hc_std = 130
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.17
                                hc_std = 120
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.05
                                hc_std = 110
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.96
                                hc_std = 100
                            elif 3000 <= car_weight:
                                co_std = 0.89
                                hc_std = 90
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.48
                                hc_std = 100
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                                hc_std = 90
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                                hc_std = 80
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                                hc_std = 80
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                                hc_std = 70
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                                hc_std = 70
                            elif 3000 <= car_weight:
                                co_std = 0.34
                                hc_std = 60
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.48
                                hc_std = 100
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                                hc_std = 90
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                                hc_std = 90
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                                hc_std = 80
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                                hc_std = 80
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                                hc_std = 70
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                                hc_std = 70
                            elif 3000 <= car_weight:
                                co_std = 0.34
                                hc_std = 60
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.6
                                hc_std = 260
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.5
                                hc_std = 240
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.8
                                hc_std = 200
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.8
                                hc_std = 180
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.1
                                hc_std = 170
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.1
                                hc_std = 160
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.8
                                hc_std = 140
                            elif 3000 <= car_weight:
                                co_std = 1.7
                                hc_std = 130
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 2.9
                                hc_std = 240
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.3
                                hc_std = 210
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.3
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.6
                                hc_std = 170
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.2
                                hc_std = 160
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.7
                                hc_std = 140
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.5
                                hc_std = 130
                            elif 3000 <= car_weight:
                                co_std = 1.6
                                hc_std = 120
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.2
                                hc_std = 130
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                                hc_std = 120
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                                hc_std = 120
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                                hc_std = 110
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                                hc_std = 100
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                                hc_std = 90
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                                hc_std = 90
                            elif 3000 <= car_weight:
                                co_std = 0.4
                                hc_std = 80
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.2
                                hc_std = 130
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                                hc_std = 120
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                                hc_std = 120
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                                hc_std = 110
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                                hc_std = 100
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                                hc_std = 90
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                                hc_std = 90
                            elif 3000 <= car_weight:
                                co_std = 0.4
                                hc_std = 80

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                co_std = 2.51
                                hc_std = 200
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.22
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.01
                                hc_std = 160
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.82
                                hc_std = 150
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.62
                                hc_std = 130
                            elif 2750 <= car_weight < 3000:
                                co_std = 1.49
                                hc_std = 120
                            elif 3000 <= car_weight <= 3250:
                                co_std = 1.37
                                hc_std = 110
                            elif 3250 <= car_weight:
                                co_std = 1.36
                                hc_std = 110
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 1.89
                                hc_std = 170
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                                hc_std = 150
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                                hc_std = 140
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                                hc_std = 130
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                                hc_std = 110
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                                hc_std = 100
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                                hc_std = 100
                            elif 3000 <= car_weight:
                                co_std = 1.04
                                hc_std = 100
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 1.89
                                hc_std = 170
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                                hc_std = 150
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                                hc_std = 140
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                                hc_std = 130
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                                hc_std = 110
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                                hc_std = 100
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                                hc_std = 100
                            elif 3000 <= car_weight:
                                co_std = 1.04
                                hc_std = 100
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 3.26
                                hc_std = 260
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.89
                                hc_std = 230
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.61
                                hc_std = 210
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.37
                                hc_std = 190
                            elif 2250 <= car_weight < 2500:
                                co_std = 2.11
                                hc_std = 170
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.94
                                hc_std = 160
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.78
                                hc_std = 150
                            elif 3000 <= car_weight:
                                co_std = 1.77
                                hc_std = 150
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 2.46
                                hc_std = 220
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                                hc_std = 200
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                                hc_std = 160
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                                hc_std = 150
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                                hc_std = 140
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                                hc_std = 130
                            elif 3000 <= car_weight:
                                co_std = 1.35
                                hc_std = 130
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 2.46
                                hc_std = 220
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                                hc_std = 200
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                                hc_std = 160
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                                hc_std = 150
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                                hc_std = 140
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                                hc_std = 130
                            elif 3000 <= car_weight:
                                co_std = 1.35
                                hc_std = 130

            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox1) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # Y N Y
                hc_std = 1
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            co_std = 4.2
                            nox_std = 2110
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 2.4
                            nox_std = 1800
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.2
                            nox_std = 1640
                        elif '20040101' <= product_date <= '20051231':
                            co_std = 1
                            nox_std = 1240
                        elif '20060101' <= product_date:
                            co_std = 1
                            nox_std = 1240
                        else:
                            co_std = 0
                            nox_std = 0
                    else:
                        if product_date < '19971231':
                            co_std = 5.46
                            nox_std = 2530
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 3.12
                            nox_std = 2160
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.56
                            nox_std = 1960
                        elif '20040101' <= product_date <= '20071231':
                            co_std = 1.3
                            nox_std = 1490
                        elif '20080101' <= product_date:
                            co_std = 1.3
                            nox_std = 1490
                        else:
                            co_std = 0
                            nox_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 1.83
                                nox_std = 1800
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.47
                                nox_std = 1440
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.21
                                nox_std = 1190
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.08
                                nox_std = 1050
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.94
                                nox_std = 920
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.87
                                nox_std = 840
                            elif 2000 <= car_weight:
                                co_std = 0.79
                                nox_std = 760
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1
                                nox_std = 1400
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                nox_std = 1120
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.67
                                nox_std = 930
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                nox_std = 820
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                nox_std = 720
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                nox_std = 660
                            elif 2000 <= car_weight:
                                co_std = 0.45
                                nox_std = 600
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                co_std = 0.77
                                nox_std = 1100
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                                nox_std = 870
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                                nox_std = 720
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                                nox_std = 630
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                                nox_std = 550
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                                nox_std = 500
                            elif 2000 <= car_weight:
                                co_std = 0.35
                                nox_std = 450
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                co_std = 0.77
                                nox_std = 1080
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                                nox_std = 850
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                                nox_std = 690
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                                nox_std = 610
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                                nox_std = 530
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                                nox_std = 490
                            elif 2000 <= car_weight:
                                co_std = 0.35
                                nox_std = 440

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 2.38
                                nox_std = 2160
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.91
                                nox_std = 1728
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.57
                                nox_std = 1428
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.4
                                nox_std = 1260
                            elif 1600 <= car_weight < 1800:
                                co_std = 1.22
                                nox_std = 1104
                            elif 1800 <= car_weight < 2000:
                                co_std = 1.13
                                nox_std = 1008
                            elif 2000 <= car_weight:
                                co_std = 1.03
                                nox_std = 912
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1.3
                                nox_std = 1680
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.05
                                nox_std = 1344
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.87
                                nox_std = 1116
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.78
                                nox_std = 984
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.69
                                nox_std = 864
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.64
                                nox_std = 792
                            elif 2000 <= car_weight:
                                co_std = 0.59
                                nox_std = 720
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                co_std = 1
                                nox_std = 1320
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                nox_std = 1040
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                nox_std = 860
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                nox_std = 580
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                nox_std = 530
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                co_std = 1
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                nox_std = 590
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                nox_std = 530
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                co_std = 1
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                                nox_std = 590
                            elif 2000 <= car_weight:
                                co_std = 0.46
                                nox_std = 530

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.2
                                nox_std = 1260
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.96
                                nox_std = 1120
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.68
                                nox_std = 960
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.52
                                nox_std = 860
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.39
                                nox_std = 780
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.24
                                nox_std = 690
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.14
                                nox_std = 630
                            elif 3000 <= car_weight:
                                co_std = 1.05
                                nox_std = 580
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 1.84
                                nox_std = 1120
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.64
                                nox_std = 1000
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.41
                                nox_std = 860
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.28
                                nox_std = 770
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.17
                                nox_std = 700
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.05
                                nox_std = 620
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.96
                                nox_std = 570
                            elif 3000 <= car_weight:
                                co_std = 0.89
                                nox_std = 520
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.48
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                                nox_std = 570
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                                nox_std = 420
                            elif 3000 <= car_weight:
                                co_std = 0.34
                                nox_std = 380
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.48
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                                nox_std = 560
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                                nox_std = 420
                            elif 3000 <= car_weight:
                                co_std = 0.34
                                nox_std = 380
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.6
                                nox_std = 1510
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.5
                                nox_std = 1350
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.8
                                nox_std = 1150
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.8
                                nox_std = 1030
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.1
                                nox_std = 940
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.1
                                nox_std = 830
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.8
                                nox_std = 750
                            elif 3000 <= car_weight:
                                co_std = 1.7
                                nox_std = 700
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 2.9
                                nox_std = 1350
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.3
                                nox_std = 1200
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.3
                                nox_std = 1030
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.6
                                nox_std = 920
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.2
                                nox_std = 840
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.7
                                nox_std = 740
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.5
                                nox_std = 680
                            elif 3000 <= car_weight:
                                co_std = 1.6
                                nox_std = 630
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.2
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                                nox_std = 500
                            elif 3000 <= car_weight:
                                co_std = 0.4
                                nox_std = 460
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.2
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                                nox_std = 500
                            elif 3000 <= car_weight:
                                co_std = 0.4
                                nox_std = 460

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                co_std = 2.51
                                nox_std = 1950
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.22
                                nox_std = 1720
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.01
                                nox_std = 1550
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.82
                                nox_std = 1400
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.62
                                nox_std = 1230
                            elif 2750 <= car_weight < 3000:
                                co_std = 1.49
                                nox_std = 1120
                            elif 3000 <= car_weight <= 3250:
                                co_std = 1.37
                                nox_std = 1040
                            elif 3250 <= car_weight:
                                co_std = 1.36
                                nox_std = 1040
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 1.89
                                nox_std = 1720
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                                nox_std = 1530
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                                nox_std = 1370
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                                nox_std = 1240
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                                nox_std = 1090
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                                nox_std = 990
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                                nox_std = 920
                            elif 3000 <= car_weight:
                                co_std = 1.04
                                nox_std = 920
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 1.89
                                nox_std = 1500
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                                nox_std = 1330
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                                nox_std = 1190
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                                nox_std = 1080
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                                nox_std = 950
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                                nox_std = 870
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                                nox_std = 810
                            elif 3000 <= car_weight:
                                co_std = 1.04
                                nox_std = 810
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 3.26
                                nox_std = 2340
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.89
                                nox_std = 2070
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.61
                                nox_std = 1860
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.37
                                nox_std = 1680
                            elif 2250 <= car_weight < 2500:
                                co_std = 2.11
                                nox_std = 1480
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.94
                                nox_std = 1340
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.78
                                nox_std = 1250
                            elif 3000 <= car_weight:
                                co_std = 1.77
                                nox_std = 1240
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 2.46
                                nox_std = 2070
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                                nox_std = 1830
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                                nox_std = 1640
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                                nox_std = 1480
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                                nox_std = 1310
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                                nox_std = 1190
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                                nox_std = 1110
                            elif 3000 <= car_weight:
                                co_std = 1.35
                                nox_std = 1110
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 2.46
                                nox_std = 1800
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                                nox_std = 1590
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                                nox_std = 1430
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                                nox_std = 1290
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                                nox_std = 1140
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                                nox_std = 1040
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                                nox_std = 970
                            elif 3000 <= car_weight:
                                co_std = 1.35
                                nox_std = 970

            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox1) != 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # N Y Y
                co_std = 1
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            hc_std = 1090
                            nox_std = 2110
                        elif '19980101' <= product_date <= '20001231':
                            hc_std = 390
                            nox_std = 1800
                        elif '20010101' <= product_date <= '20031231':
                            hc_std = 210
                            nox_std = 1640
                        elif '20040101' <= product_date <= '20051231':
                            hc_std = 180
                            nox_std = 1240
                        elif '20060101' <= product_date:
                            hc_std = 180
                            nox_std = 1240
                        else:
                            hc_std = 0
                            nox_std = 0
                    else:
                        if product_date < '19971231':
                            hc_std = 1410
                            nox_std = 2530
                        elif '19980101' <= product_date <= '20001231':
                            hc_std = 500
                            nox_std = 2160
                        elif '20010101' <= product_date <= '20031231':
                            hc_std = 270
                            nox_std = 1960
                        elif '20040101' <= product_date <= '20071231':
                            hc_std = 230
                            nox_std = 1490
                        elif '20080101' <= product_date:
                            hc_std = 180
                            nox_std = 1490
                        else:
                            hc_std = 0
                            nox_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                hc_std = 210
                                nox_std = 1800
                            elif 1000 <= car_weight < 1200:
                                hc_std = 170
                                nox_std = 1440
                            elif 1200 <= car_weight < 1400:
                                hc_std = 140
                                nox_std = 1190
                            elif 1400 <= car_weight < 1600:
                                hc_std = 130
                                nox_std = 1050
                            elif 1600 <= car_weight < 1800:
                                hc_std = 120
                                nox_std = 920
                            elif 1800 <= car_weight < 2000:
                                hc_std = 110
                                nox_std = 840
                            elif 2000 <= car_weight:
                                hc_std = 100
                                nox_std = 760
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                hc_std = 180
                                nox_std = 1400
                            elif 1000 <= car_weight < 1200:
                                hc_std = 150
                                nox_std = 1120
                            elif 1200 <= car_weight < 1400:
                                hc_std = 120
                                nox_std = 930
                            elif 1400 <= car_weight < 1600:
                                hc_std = 110
                                nox_std = 820
                            elif 1600 <= car_weight < 1800:
                                hc_std = 100
                                nox_std = 720
                            elif 1800 <= car_weight < 2000:
                                hc_std = 90
                                nox_std = 660
                            elif 2000 <= car_weight:
                                hc_std = 80
                                nox_std = 600
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                hc_std = 140
                                nox_std = 1100
                            elif 1000 <= car_weight < 1200:
                                hc_std = 110
                                nox_std = 870
                            elif 1200 <= car_weight < 1400:
                                hc_std = 100
                                nox_std = 720
                            elif 1400 <= car_weight < 1600:
                                hc_std = 90
                                nox_std = 630
                            elif 1600 <= car_weight < 1800:
                                hc_std = 80
                                nox_std = 550
                            elif 1800 <= car_weight < 2000:
                                hc_std = 70
                                nox_std = 500
                            elif 2000 <= car_weight:
                                hc_std = 60
                                nox_std = 450
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                hc_std = 59
                                nox_std = 1080
                            elif 1000 <= car_weight < 1200:
                                hc_std = 49
                                nox_std = 850
                            elif 1200 <= car_weight < 1400:
                                hc_std = 42
                                nox_std = 690
                            elif 1400 <= car_weight < 1600:
                                hc_std = 37
                                nox_std = 610
                            elif 1600 <= car_weight < 1800:
                                hc_std = 33
                                nox_std = 530
                            elif 1800 <= car_weight < 2000:
                                hc_std = 30
                                nox_std = 490
                            elif 2000 <= car_weight:
                                hc_std = 30
                                nox_std = 440

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                hc_std = 273
                                nox_std = 2160
                            elif 1000 <= car_weight < 1200:
                                hc_std = 221
                                nox_std = 1728
                            elif 1200 <= car_weight < 1400:
                                hc_std = 182
                                nox_std = 1428
                            elif 1400 <= car_weight < 1600:
                                hc_std = 169
                                nox_std = 1260
                            elif 1600 <= car_weight < 1800:
                                hc_std = 156
                                nox_std = 1104
                            elif 1800 <= car_weight < 2000:
                                hc_std = 143
                                nox_std = 1008
                            elif 2000 <= car_weight:
                                hc_std = 130
                                nox_std = 912
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                hc_std = 234
                                nox_std = 1680
                            elif 1000 <= car_weight < 1200:
                                hc_std = 195
                                nox_std = 1344
                            elif 1200 <= car_weight < 1400:
                                hc_std = 156
                                nox_std = 1116
                            elif 1400 <= car_weight < 1600:
                                hc_std = 143
                                nox_std = 984
                            elif 1600 <= car_weight < 1800:
                                hc_std = 130
                                nox_std = 864
                            elif 1800 <= car_weight < 2000:
                                hc_std = 117
                                nox_std = 792
                            elif 2000 <= car_weight:
                                hc_std = 104
                                nox_std = 720
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                hc_std = 180
                                nox_std = 1320
                            elif 1000 <= car_weight < 1200:
                                hc_std = 140
                                nox_std = 1040
                            elif 1200 <= car_weight < 1400:
                                hc_std = 120
                                nox_std = 860
                            elif 1400 <= car_weight < 1600:
                                hc_std = 110
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                hc_std = 100
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                hc_std = 90
                                nox_std = 580
                            elif 2000 <= car_weight:
                                hc_std = 80
                                nox_std = 530
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                hc_std = 180
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                hc_std = 140
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                hc_std = 130
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                hc_std = 120
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                hc_std = 100
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                hc_std = 90
                                nox_std = 590
                            elif 2000 <= car_weight:
                                hc_std = 80
                                nox_std = 530
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                hc_std = 59
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                hc_std = 49
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                hc_std = 42
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                hc_std = 37
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                hc_std = 33
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                hc_std = 30
                                nox_std = 590
                            elif 2000 <= car_weight:
                                hc_std = 30
                                nox_std = 530

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                hc_std = 200
                                nox_std = 1260
                            elif 1500 <= car_weight < 1750:
                                hc_std = 180
                                nox_std = 1120
                            elif 1750 <= car_weight < 2000:
                                hc_std = 150
                                nox_std = 960
                            elif 2000 <= car_weight < 2250:
                                hc_std = 140
                                nox_std = 860
                            elif 2250 <= car_weight < 2500:
                                hc_std = 130
                                nox_std = 780
                            elif 2500 <= car_weight < 2750:
                                hc_std = 120
                                nox_std = 690
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 110
                                nox_std = 630
                            elif 3000 <= car_weight:
                                hc_std = 100
                                nox_std = 580
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                hc_std = 180
                                nox_std = 1120
                            elif 1500 <= car_weight < 1750:
                                hc_std = 160
                                nox_std = 1000
                            elif 1750 <= car_weight < 2000:
                                hc_std = 140
                                nox_std = 860
                            elif 2000 <= car_weight < 2250:
                                hc_std = 130
                                nox_std = 770
                            elif 2250 <= car_weight < 2500:
                                hc_std = 120
                                nox_std = 700
                            elif 2500 <= car_weight < 2750:
                                hc_std = 110
                                nox_std = 620
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 100
                                nox_std = 570
                            elif 3000 <= car_weight:
                                hc_std = 90
                                nox_std = 520
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 100
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                hc_std = 90
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                hc_std = 90
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                hc_std = 80
                                nox_std = 570
                            elif 2250 <= car_weight < 2500:
                                hc_std = 80
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                hc_std = 70
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 70
                                nox_std = 420
                            elif 3000 <= car_weight:
                                hc_std = 60
                                nox_std = 380
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 100
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                hc_std = 90
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                hc_std = 90
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                hc_std = 80
                                nox_std = 560
                            elif 2250 <= car_weight < 2500:
                                hc_std = 80
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                hc_std = 70
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 70
                                nox_std = 420
                            elif 3000 <= car_weight:
                                hc_std = 60
                                nox_std = 380
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                hc_std = 260
                                nox_std = 1510
                            elif 1500 <= car_weight < 1750:
                                hc_std = 240
                                nox_std = 1350
                            elif 1750 <= car_weight < 2000:
                                hc_std = 200
                                nox_std = 1150
                            elif 2000 <= car_weight < 2250:
                                hc_std = 180
                                nox_std = 1030
                            elif 2250 <= car_weight < 2500:
                                hc_std = 170
                                nox_std = 940
                            elif 2500 <= car_weight < 2750:
                                hc_std = 160
                                nox_std = 830
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 140
                                nox_std = 750
                            elif 3000 <= car_weight:
                                hc_std = 130
                                nox_std = 700
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                hc_std = 240
                                nox_std = 1350
                            elif 1500 <= car_weight < 1750:
                                hc_std = 210
                                nox_std = 1200
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                                nox_std = 1030
                            elif 2000 <= car_weight < 2250:
                                hc_std = 170
                                nox_std = 920
                            elif 2250 <= car_weight < 2500:
                                hc_std = 160
                                nox_std = 840
                            elif 2500 <= car_weight < 2750:
                                hc_std = 140
                                nox_std = 740
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 130
                                nox_std = 680
                            elif 3000 <= car_weight:
                                hc_std = 120
                                nox_std = 630
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 130
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                hc_std = 120
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                hc_std = 120
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                hc_std = 110
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                hc_std = 100
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                hc_std = 90
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 90
                                nox_std = 500
                            elif 3000 <= car_weight:
                                hc_std = 80
                                nox_std = 460
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 130
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                hc_std = 120
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                hc_std = 120
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                hc_std = 110
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                hc_std = 100
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                hc_std = 90
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 90
                                nox_std = 500
                            elif 3000 <= car_weight:
                                hc_std = 80
                                nox_std = 460

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                hc_std = 200
                                nox_std = 1950
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                                nox_std = 1720
                            elif 2000 <= car_weight < 2250:
                                hc_std = 160
                                nox_std = 1550
                            elif 2250 <= car_weight < 2500:
                                hc_std = 150
                                nox_std = 1400
                            elif 2500 <= car_weight < 2750:
                                hc_std = 130
                                nox_std = 1230
                            elif 2750 <= car_weight < 3000:
                                hc_std = 120
                                nox_std = 1120
                            elif 3000 <= car_weight <= 3250:
                                hc_std = 110
                                nox_std = 1040
                            elif 3250 <= car_weight:
                                hc_std = 110
                                nox_std = 1040
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 170
                                nox_std = 1720
                            elif 1500 <= car_weight < 1750:
                                hc_std = 150
                                nox_std = 1530
                            elif 1750 <= car_weight < 2000:
                                hc_std = 140
                                nox_std = 1370
                            elif 2000 <= car_weight < 2250:
                                hc_std = 130
                                nox_std = 1240
                            elif 2250 <= car_weight < 2500:
                                hc_std = 110
                                nox_std = 1090
                            elif 2500 <= car_weight < 2750:
                                hc_std = 100
                                nox_std = 990
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 100
                                nox_std = 920
                            elif 3000 <= car_weight:
                                hc_std = 100
                                nox_std = 920
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 170
                                nox_std = 1500
                            elif 1500 <= car_weight < 1750:
                                hc_std = 150
                                nox_std = 1330
                            elif 1750 <= car_weight < 2000:
                                hc_std = 140
                                nox_std = 1190
                            elif 2000 <= car_weight < 2250:
                                hc_std = 130
                                nox_std = 1080
                            elif 2250 <= car_weight < 2500:
                                hc_std = 110
                                nox_std = 950
                            elif 2500 <= car_weight < 2750:
                                hc_std = 100
                                nox_std = 870
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 100
                                nox_std = 810
                            elif 3000 <= car_weight:
                                hc_std = 100
                                nox_std = 810
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                hc_std = 260
                                nox_std = 2340
                            elif 1500 <= car_weight < 1750:
                                hc_std = 230
                                nox_std = 2070
                            elif 1750 <= car_weight < 2000:
                                hc_std = 210
                                nox_std = 1860
                            elif 2000 <= car_weight < 2250:
                                hc_std = 190
                                nox_std = 1680
                            elif 2250 <= car_weight < 2500:
                                hc_std = 170
                                nox_std = 1480
                            elif 2500 <= car_weight < 2750:
                                hc_std = 160
                                nox_std = 1340
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 150
                                nox_std = 1250
                            elif 3000 <= car_weight:
                                hc_std = 150
                                nox_std = 1240
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 220
                                nox_std = 2070
                            elif 1500 <= car_weight < 1750:
                                hc_std = 200
                                nox_std = 1830
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                                nox_std = 1640
                            elif 2000 <= car_weight < 2250:
                                hc_std = 160
                                nox_std = 1480
                            elif 2250 <= car_weight < 2500:
                                hc_std = 150
                                nox_std = 1310
                            elif 2500 <= car_weight < 2750:
                                hc_std = 140
                                nox_std = 1190
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 130
                                nox_std = 1110
                            elif 3000 <= car_weight:
                                hc_std = 130
                                nox_std = 1110
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 220
                                nox_std = 1800
                            elif 1500 <= car_weight < 1750:
                                hc_std = 200
                                nox_std = 1590
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                                nox_std = 1430
                            elif 2000 <= car_weight < 2250:
                                hc_std = 160
                                nox_std = 1290
                            elif 2250 <= car_weight < 2500:
                                hc_std = 150
                                nox_std = 1140
                            elif 2500 <= car_weight < 2750:
                                hc_std = 140
                                nox_std = 1040
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 130
                                nox_std = 970
                            elif 3000 <= car_weight:
                                hc_std = 130
                                nox_std = 970 
            
            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox1) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox1)) ): # Y N N
                hc_std = 1
                nox_std = 1
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            co_std = 4.2
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 2.4
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.2
                        elif '20040101' <= product_date <= '20051231':
                            co_std = 1
                        elif '20060101' <= product_date:
                            co_std = 1
                        else:
                            co_std = 0
                    else:
                        if product_date < '19971231':
                            co_std = 5.46
                        elif '19980101' <= product_date <= '20001231':
                            co_std = 3.12
                        elif '20010101' <= product_date <= '20031231':
                            co_std = 1.56
                        elif '20040101' <= product_date <= '20071231':
                            co_std = 1.3
                        elif '20080101' <= product_date:
                            co_std = 1.3
                        else:
                            co_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 1.83
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.47
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.21
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.08
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.94
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.87
                            elif 2000 <= car_weight:
                                co_std = 0.79
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.67
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                            elif 2000 <= car_weight:
                                co_std = 0.45
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                co_std = 0.77
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                            elif 2000 <= car_weight:
                                co_std = 0.35
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                co_std = 0.77
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.62
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.52
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.46
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.41
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.38
                            elif 2000 <= car_weight:
                                co_std = 0.35

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                co_std = 2.38
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.91
                            elif 1200 <= car_weight < 1400:
                                co_std = 1.57
                            elif 1400 <= car_weight < 1600:
                                co_std = 1.4
                            elif 1600 <= car_weight < 1800:
                                co_std = 1.22
                            elif 1800 <= car_weight < 2000:
                                co_std = 1.13
                            elif 2000 <= car_weight:
                                co_std = 1.03
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                co_std = 1.3
                            elif 1000 <= car_weight < 1200:
                                co_std = 1.05
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.87
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.78
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.69
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.64
                            elif 2000 <= car_weight:
                                co_std = 0.59
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                co_std = 1
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                            elif 2000 <= car_weight:
                                co_std = 0.46
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                co_std = 1
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                            elif 2000 <= car_weight:
                                co_std = 0.46
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                co_std = 1
                            elif 1000 <= car_weight < 1200:
                                co_std = 0.81
                            elif 1200 <= car_weight < 1400:
                                co_std = 0.68
                            elif 1400 <= car_weight < 1600:
                                co_std = 0.6
                            elif 1600 <= car_weight < 1800:
                                co_std = 0.53
                            elif 1800 <= car_weight < 2000:
                                co_std = 0.49
                            elif 2000 <= car_weight:
                                co_std = 0.46

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.2
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.96
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.68
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.52
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.39
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.24
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.14
                            elif 3000 <= car_weight:
                                co_std = 1.05
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 1.84
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.64
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.41
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.28
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.17
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.05
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.96
                            elif 3000 <= car_weight:
                                co_std = 0.89
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.48
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                            elif 3000 <= car_weight:
                                co_std = 0.34
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.48
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.43
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.49
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.45
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.42
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.38
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.35
                            elif 3000 <= car_weight:
                                co_std = 0.34
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                co_std = 2.6
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.5
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.8
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.8
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.1
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.1
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.8
                            elif 3000 <= car_weight:
                                co_std = 1.7
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 2.9
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.3
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.3
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.6
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.2
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.7
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.5
                            elif 3000 <= car_weight:
                                co_std = 1.6
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 0.2
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                            elif 3000 <= car_weight:
                                co_std = 0.4
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 0.2
                            elif 1500 <= car_weight < 1750:
                                co_std = 0.6
                            elif 1750 <= car_weight < 2000:
                                co_std = 0.4
                            elif 2000 <= car_weight < 2250:
                                co_std = 0.9
                            elif 2250 <= car_weight < 2500:
                                co_std = 0.5
                            elif 2500 <= car_weight < 2750:
                                co_std = 0.9
                            elif 2750 <= car_weight <= 3000:
                                co_std = 0.6
                            elif 3000 <= car_weight:
                                co_std = 0.4

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                co_std = 2.51
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.22
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.01
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.82
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.62
                            elif 2750 <= car_weight < 3000:
                                co_std = 1.49
                            elif 3000 <= car_weight <= 3250:
                                co_std = 1.37
                            elif 3250 <= car_weight:
                                co_std = 1.36
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 1.89
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                            elif 3000 <= car_weight:
                                co_std = 1.04
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 1.89
                            elif 1500 <= car_weight < 1750:
                                co_std = 1.68
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.52
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.38
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.24
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.13
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.05
                            elif 3000 <= car_weight:
                                co_std = 1.04
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                co_std = 3.26
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.89
                            elif 1750 <= car_weight < 2000:
                                co_std = 2.61
                            elif 2000 <= car_weight < 2250:
                                co_std = 2.37
                            elif 2250 <= car_weight < 2500:
                                co_std = 2.11
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.94
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.78
                            elif 3000 <= car_weight:
                                co_std = 1.77
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                co_std = 2.46
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                            elif 3000 <= car_weight:
                                co_std = 1.35
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                co_std = 2.46
                            elif 1500 <= car_weight < 1750:
                                co_std = 2.18
                            elif 1750 <= car_weight < 2000:
                                co_std = 1.98
                            elif 2000 <= car_weight < 2250:
                                co_std = 1.79
                            elif 2250 <= car_weight < 2500:
                                co_std = 1.61
                            elif 2500 <= car_weight < 2750:
                                co_std = 1.47
                            elif 2750 <= car_weight <= 3000:
                                co_std = 1.37
                            elif 3000 <= car_weight:
                                co_std = 1.35
            
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox1) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox1)) ): # N Y N
                co_std = 1
                nox_std = 1
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            hc_std = 1090
                        elif '19980101' <= product_date <= '20001231':
                            hc_std = 390
                        elif '20010101' <= product_date <= '20031231':
                            hc_std = 210
                        elif '20040101' <= product_date <= '20051231':
                            hc_std = 180
                        elif '20060101' <= product_date:
                            hc_std = 180
                        else:
                            hc_std = 0
                    else:
                        if product_date < '19971231':
                            hc_std = 1410
                        elif '19980101' <= product_date <= '20001231':
                            hc_std = 500
                        elif '20010101' <= product_date <= '20031231':
                            hc_std = 270
                        elif '20040101' <= product_date <= '20071231':
                            hc_std = 230
                        elif '20080101' <= product_date:
                            hc_std = 180
                        else:
                            hc_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                hc_std = 210
                            elif 1000 <= car_weight < 1200:
                                hc_std = 170
                            elif 1200 <= car_weight < 1400:
                                hc_std = 140
                            elif 1400 <= car_weight < 1600:
                                hc_std = 130
                            elif 1600 <= car_weight < 1800:
                                hc_std = 120
                            elif 1800 <= car_weight < 2000:
                                hc_std = 110
                            elif 2000 <= car_weight:
                                hc_std = 100
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                hc_std = 180
                            elif 1000 <= car_weight < 1200:
                                hc_std = 150
                            elif 1200 <= car_weight < 1400:
                                hc_std = 120
                            elif 1400 <= car_weight < 1600:
                                hc_std = 110
                            elif 1600 <= car_weight < 1800:
                                hc_std = 100
                            elif 1800 <= car_weight < 2000:
                                hc_std = 90
                            elif 2000 <= car_weight:
                                hc_std = 80
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                hc_std = 140
                            elif 1000 <= car_weight < 1200:
                                hc_std = 110
                            elif 1200 <= car_weight < 1400:
                                hc_std = 100
                            elif 1400 <= car_weight < 1600:
                                hc_std = 90
                            elif 1600 <= car_weight < 1800:
                                hc_std = 80
                            elif 1800 <= car_weight < 2000:
                                hc_std = 70
                            elif 2000 <= car_weight:
                                hc_std = 60
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                hc_std = 59
                            elif 1000 <= car_weight < 1200:
                                hc_std = 49
                            elif 1200 <= car_weight < 1400:
                                hc_std = 42
                            elif 1400 <= car_weight < 1600:
                                hc_std = 37
                            elif 1600 <= car_weight < 1800:
                                hc_std = 33
                            elif 1800 <= car_weight < 2000:
                                hc_std = 30
                            elif 2000 <= car_weight:
                                hc_std = 30

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                hc_std = 273
                            elif 1000 <= car_weight < 1200:
                                hc_std = 221
                            elif 1200 <= car_weight < 1400:
                                hc_std = 182
                            elif 1400 <= car_weight < 1600:
                                hc_std = 169
                            elif 1600 <= car_weight < 1800:
                                hc_std = 156
                            elif 1800 <= car_weight < 2000:
                                hc_std = 143
                            elif 2000 <= car_weight:
                                hc_std = 130
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                hc_std = 234
                            elif 1000 <= car_weight < 1200:
                                hc_std = 195
                            elif 1200 <= car_weight < 1400:
                                hc_std = 156
                            elif 1400 <= car_weight < 1600:
                                hc_std = 143
                            elif 1600 <= car_weight < 1800:
                                hc_std = 130
                            elif 1800 <= car_weight < 2000:
                                hc_std = 117
                            elif 2000 <= car_weight:
                                hc_std = 104
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                hc_std = 180
                            elif 1000 <= car_weight < 1200:
                                hc_std = 140
                            elif 1200 <= car_weight < 1400:
                                hc_std = 120
                            elif 1400 <= car_weight < 1600:
                                hc_std = 110
                            elif 1600 <= car_weight < 1800:
                                hc_std = 100
                            elif 1800 <= car_weight < 2000:
                                hc_std = 90
                            elif 2000 <= car_weight:
                                hc_std = 80
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                hc_std = 180
                            elif 1000 <= car_weight < 1200:
                                hc_std = 140
                            elif 1200 <= car_weight < 1400:
                                hc_std = 130
                            elif 1400 <= car_weight < 1600:
                                hc_std = 120
                            elif 1600 <= car_weight < 1800:
                                hc_std = 100
                            elif 1800 <= car_weight < 2000:
                                hc_std = 90
                            elif 2000 <= car_weight:
                                hc_std = 80
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                hc_std = 59
                            elif 1000 <= car_weight < 1200:
                                hc_std = 49
                            elif 1200 <= car_weight < 1400:
                                hc_std = 42
                            elif 1400 <= car_weight < 1600:
                                hc_std = 37
                            elif 1600 <= car_weight < 1800:
                                hc_std = 33
                            elif 1800 <= car_weight < 2000:
                                hc_std = 30
                            elif 2000 <= car_weight:
                                hc_std = 30

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                hc_std = 200
                            elif 1500 <= car_weight < 1750:
                                hc_std = 180
                            elif 1750 <= car_weight < 2000:
                                hc_std = 150
                            elif 2000 <= car_weight < 2250:
                                hc_std = 140
                            elif 2250 <= car_weight < 2500:
                                hc_std = 130
                            elif 2500 <= car_weight < 2750:
                                hc_std = 120
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 110
                            elif 3000 <= car_weight:
                                hc_std = 100
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                hc_std = 180
                            elif 1500 <= car_weight < 1750:
                                hc_std = 160
                            elif 1750 <= car_weight < 2000:
                                hc_std = 140
                            elif 2000 <= car_weight < 2250:
                                hc_std = 130
                            elif 2250 <= car_weight < 2500:
                                hc_std = 120
                            elif 2500 <= car_weight < 2750:
                                hc_std = 110
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 100
                            elif 3000 <= car_weight:
                                hc_std = 90
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 100
                            elif 1500 <= car_weight < 1750:
                                hc_std = 90
                            elif 1750 <= car_weight < 2000:
                                hc_std = 90
                            elif 2000 <= car_weight < 2250:
                                hc_std = 80
                            elif 2250 <= car_weight < 2500:
                                hc_std = 80
                            elif 2500 <= car_weight < 2750:
                                hc_std = 70
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 70
                            elif 3000 <= car_weight:
                                hc_std = 60
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 100
                            elif 1500 <= car_weight < 1750:
                                hc_std = 90
                            elif 1750 <= car_weight < 2000:
                                hc_std = 90
                            elif 2000 <= car_weight < 2250:
                                hc_std = 80
                            elif 2250 <= car_weight < 2500:
                                hc_std = 80
                            elif 2500 <= car_weight < 2750:
                                hc_std = 70
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 70
                            elif 3000 <= car_weight:
                                hc_std = 60
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                hc_std = 260
                            elif 1500 <= car_weight < 1750:
                                hc_std = 240
                            elif 1750 <= car_weight < 2000:
                                hc_std = 200
                            elif 2000 <= car_weight < 2250:
                                hc_std = 180
                            elif 2250 <= car_weight < 2500:
                                hc_std = 170
                            elif 2500 <= car_weight < 2750:
                                hc_std = 160
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 140
                            elif 3000 <= car_weight:
                                hc_std = 130
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                hc_std = 240
                            elif 1500 <= car_weight < 1750:
                                hc_std = 210
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                hc_std = 170
                            elif 2250 <= car_weight < 2500:
                                hc_std = 160
                            elif 2500 <= car_weight < 2750:
                                hc_std = 140
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 130
                            elif 3000 <= car_weight:
                                hc_std = 120
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 130
                            elif 1500 <= car_weight < 1750:
                                hc_std = 120
                            elif 1750 <= car_weight < 2000:
                                hc_std = 120
                            elif 2000 <= car_weight < 2250:
                                hc_std = 110
                            elif 2250 <= car_weight < 2500:
                                hc_std = 100
                            elif 2500 <= car_weight < 2750:
                                hc_std = 90
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 90
                            elif 3000 <= car_weight:
                                hc_std = 80
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 130
                            elif 1500 <= car_weight < 1750:
                                hc_std = 120
                            elif 1750 <= car_weight < 2000:
                                hc_std = 120
                            elif 2000 <= car_weight < 2250:
                                hc_std = 110
                            elif 2250 <= car_weight < 2500:
                                hc_std = 100
                            elif 2500 <= car_weight < 2750:
                                hc_std = 90
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 90
                            elif 3000 <= car_weight:
                                hc_std = 80

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                hc_std = 200
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                hc_std = 160
                            elif 2250 <= car_weight < 2500:
                                hc_std = 150
                            elif 2500 <= car_weight < 2750:
                                hc_std = 130
                            elif 2750 <= car_weight < 3000:
                                hc_std = 120
                            elif 3000 <= car_weight <= 3250:
                                hc_std = 110
                            elif 3250 <= car_weight:
                                hc_std = 110
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 170
                            elif 1500 <= car_weight < 1750:
                                hc_std = 150
                            elif 1750 <= car_weight < 2000:
                                hc_std = 140
                            elif 2000 <= car_weight < 2250:
                                hc_std = 130
                            elif 2250 <= car_weight < 2500:
                                hc_std = 110
                            elif 2500 <= car_weight < 2750:
                                hc_std = 100
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 100
                            elif 3000 <= car_weight:
                                hc_std = 100
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 170
                            elif 1500 <= car_weight < 1750:
                                hc_std = 150
                            elif 1750 <= car_weight < 2000:
                                hc_std = 140
                            elif 2000 <= car_weight < 2250:
                                hc_std = 130
                            elif 2250 <= car_weight < 2500:
                                hc_std = 110
                            elif 2500 <= car_weight < 2750:
                                hc_std = 100
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 100
                            elif 3000 <= car_weight:
                                hc_std = 100
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                hc_std = 260
                            elif 1500 <= car_weight < 1750:
                                hc_std = 230
                            elif 1750 <= car_weight < 2000:
                                hc_std = 210
                            elif 2000 <= car_weight < 2250:
                                hc_std = 190
                            elif 2250 <= car_weight < 2500:
                                hc_std = 170
                            elif 2500 <= car_weight < 2750:
                                hc_std = 160
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 150
                            elif 3000 <= car_weight:
                                hc_std = 150
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                hc_std = 220
                            elif 1500 <= car_weight < 1750:
                                hc_std = 200
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                hc_std = 160
                            elif 2250 <= car_weight < 2500:
                                hc_std = 150
                            elif 2500 <= car_weight < 2750:
                                hc_std = 140
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 130
                            elif 3000 <= car_weight:
                                hc_std = 130
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                hc_std = 220
                            elif 1500 <= car_weight < 1750:
                                hc_std = 200
                            elif 1750 <= car_weight < 2000:
                                hc_std = 180
                            elif 2000 <= car_weight < 2250:
                                hc_std = 160
                            elif 2250 <= car_weight < 2500:
                                hc_std = 150
                            elif 2500 <= car_weight < 2750:
                                hc_std = 140
                            elif 2750 <= car_weight <= 3000:
                                hc_std = 130
                            elif 3000 <= car_weight:
                                hc_std = 130
            
            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox1) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox1)) ): # N N Y
                co_std = 1
                hc_std = 1
                if ci2 == '경형':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19971231':
                            nox_std = 2110
                        elif '19980101' <= product_date <= '20001231':
                            nox_std = 1800
                        elif '20010101' <= product_date <= '20031231':
                            nox_std = 1640
                        elif '20040101' <= product_date <= '20051231':
                            nox_std = 1240
                        elif '20060101' <= product_date:
                            nox_std = 1240
                        else:
                            nox_std = 0
                    else:
                        if product_date < '19971231':
                            nox_std = 2530
                        elif '19980101' <= product_date <= '20001231':
                            nox_std = 2160
                        elif '20010101' <= product_date <= '20031231':
                            nox_std = 1960
                        elif '20040101' <= product_date <= '20071231':
                            nox_std = 1490
                        elif '20080101' <= product_date:
                            nox_std = 1490
                        else:
                            nox_std = 0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                nox_std = 1800
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1440
                            elif 1200 <= car_weight < 1400:
                                nox_std = 1190
                            elif 1400 <= car_weight < 1600:
                                nox_std = 1050
                            elif 1600 <= car_weight < 1800:
                                nox_std = 920
                            elif 1800 <= car_weight < 2000:
                                nox_std = 840
                            elif 2000 <= car_weight:
                                nox_std = 760
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                nox_std = 1400
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1120
                            elif 1200 <= car_weight < 1400:
                                nox_std = 930
                            elif 1400 <= car_weight < 1600:
                                nox_std = 820
                            elif 1600 <= car_weight < 1800:
                                nox_std = 720
                            elif 1800 <= car_weight < 2000:
                                nox_std = 660
                            elif 2000 <= car_weight:
                                nox_std = 600
                        elif '20010101' <= product_date <= '20051231':
                            if car_weight < 1000:
                                nox_std = 1100
                            elif 1000 <= car_weight < 1200:
                                nox_std = 870
                            elif 1200 <= car_weight < 1400:
                                nox_std = 720
                            elif 1400 <= car_weight < 1600:
                                nox_std = 630
                            elif 1600 <= car_weight < 1800:
                                nox_std = 550
                            elif 1800 <= car_weight < 2000:
                                nox_std = 500
                            elif 2000 <= car_weight:
                                nox_std = 450
                        elif '20060101' <= product_date:
                            if car_weight < 1000:
                                nox_std = 1080
                            elif 1000 <= car_weight < 1200:
                                nox_std = 850
                            elif 1200 <= car_weight < 1400:
                                nox_std = 690
                            elif 1400 <= car_weight < 1600:
                                nox_std = 610
                            elif 1600 <= car_weight < 1800:
                                nox_std = 530
                            elif 1800 <= car_weight < 2000:
                                nox_std = 490
                            elif 2000 <= car_weight:
                                nox_std = 440

                    else: # 가스사용 승용차
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1000:
                                nox_std = 2160
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1728
                            elif 1200 <= car_weight < 1400:
                                nox_std = 1428
                            elif 1400 <= car_weight < 1600:
                                nox_std = 1260
                            elif 1600 <= car_weight < 1800:
                                nox_std = 1104
                            elif 1800 <= car_weight < 2000:
                                nox_std = 1008
                            elif 2000 <= car_weight:
                                nox_std = 912
                        elif '19880101' <= product_date <= '20001231':
                            if car_weight < 1000:
                                nox_std = 1680
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1344
                            elif 1200 <= car_weight < 1400:
                                nox_std = 1116
                            elif 1400 <= car_weight < 1600:
                                nox_std = 984
                            elif 1600 <= car_weight < 1800:
                                nox_std = 864
                            elif 1800 <= car_weight < 2000:
                                nox_std = 792
                            elif 2000 <= car_weight:
                                nox_std = 720
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1000:
                                nox_std = 1320
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1040
                            elif 1200 <= car_weight < 1400:
                                nox_std = 860
                            elif 1400 <= car_weight < 1600:
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                nox_std = 580
                            elif 2000 <= car_weight:
                                nox_std = 530
                        elif '20040101' <= product_date <= '20071231':
                            if car_weight < 1000:
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                nox_std = 590
                            elif 2000 <= car_weight:
                                nox_std = 530
                        elif '20080101' <= product_date:
                            if car_weight < 1000:
                                nox_std = 1240
                            elif 1000 <= car_weight < 1200:
                                nox_std = 1000
                            elif 1200 <= car_weight < 1400:
                                nox_std = 830
                            elif 1400 <= car_weight < 1600:
                                nox_std = 730
                            elif 1600 <= car_weight < 1800:
                                nox_std = 640
                            elif 1800 <= car_weight < 2000:
                                nox_std = 590
                            elif 2000 <= car_weight:
                                nox_std = 530

                elif ci2 == '소형' and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                nox_std = 1260
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1120
                            elif 1750 <= car_weight < 2000:
                                nox_std = 960
                            elif 2000 <= car_weight < 2250:
                                nox_std = 860
                            elif 2250 <= car_weight < 2500:
                                nox_std = 780
                            elif 2500 <= car_weight < 2750:
                                nox_std = 690
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 630
                            elif 3000 <= car_weight:
                                nox_std = 580
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                nox_std = 1120
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1000
                            elif 1750 <= car_weight < 2000:
                                nox_std = 860
                            elif 2000 <= car_weight < 2250:
                                nox_std = 770
                            elif 2250 <= car_weight < 2500:
                                nox_std = 700
                            elif 2500 <= car_weight < 2750:
                                nox_std = 620
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 570
                            elif 3000 <= car_weight:
                                nox_std = 520
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                nox_std = 570
                            elif 2250 <= car_weight < 2500:
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 420
                            elif 3000 <= car_weight:
                                nox_std = 380
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                nox_std = 660
                            elif 1500 <= car_weight < 1750:
                                nox_std = 590
                            elif 1750 <= car_weight < 2000:
                                nox_std = 630
                            elif 2000 <= car_weight < 2250:
                                nox_std = 560
                            elif 2250 <= car_weight < 2500:
                                nox_std = 510
                            elif 2500 <= car_weight < 2750:
                                nox_std = 460
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 420
                            elif 3000 <= car_weight:
                                nox_std = 380
                    else: # 가스 사용
                        if '19880101' <= product_date <= '19971231':
                            if car_weight < 1500:
                                nox_std = 1510
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1350
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1150
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1030
                            elif 2250 <= car_weight < 2500:
                                nox_std = 940
                            elif 2500 <= car_weight < 2750:
                                nox_std = 830
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 750
                            elif 3000 <= car_weight:
                                nox_std = 700
                        elif '19980101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                nox_std = 1350
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1200
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1030
                            elif 2000 <= car_weight < 2250:
                                nox_std = 920
                            elif 2250 <= car_weight < 2500:
                                nox_std = 840
                            elif 2500 <= car_weight < 2750:
                                nox_std = 740
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 680
                            elif 3000 <= car_weight:
                                nox_std = 630
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 500
                            elif 3000 <= car_weight:
                                nox_std = 460
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                nox_std = 790
                            elif 1500 <= car_weight < 1750:
                                nox_std = 700
                            elif 1750 <= car_weight < 2000:
                                nox_std = 750
                            elif 2000 <= car_weight < 2250:
                                nox_std = 680
                            elif 2250 <= car_weight < 2500:
                                nox_std = 620
                            elif 2500 <= car_weight < 2750:
                                nox_std = 550
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 500
                            elif 3000 <= car_weight:
                                nox_std = 460

                elif ((ci2 == '중형') or (ci2 == '대형')) and ((ci == '승합') or (ci == '화물') or (ci == '특수')):
                    if fuel == '휘발유' or fuel == '알코올':
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1750:
                                nox_std = 1950
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1720
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1550
                            elif 2250 <= car_weight < 2500:
                                nox_std = 1400
                            elif 2500 <= car_weight < 2750:
                                nox_std = 1230
                            elif 2750 <= car_weight < 3000:
                                nox_std = 1120
                            elif 3000 <= car_weight <= 3250:
                                nox_std = 1040
                            elif 3250 <= car_weight:
                                nox_std = 1040
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                nox_std = 1720
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1530
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1370
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1240
                            elif 2250 <= car_weight < 2500:
                                nox_std = 1090
                            elif 2500 <= car_weight < 2750:
                                nox_std = 990
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 920
                            elif 3000 <= car_weight:
                                nox_std = 920
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                nox_std = 1500
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1330
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1190
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1080
                            elif 2250 <= car_weight < 2500:
                                nox_std = 950
                            elif 2500 <= car_weight < 2750:
                                nox_std = 870
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 810
                            elif 3000 <= car_weight:
                                nox_std = 810
                    else: # 가스 사용
                        if '19880101' <= product_date <= '20001231':
                            if car_weight < 1500:
                                nox_std = 2340
                            elif 1500 <= car_weight < 1750:
                                nox_std = 2070
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1860
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1680
                            elif 2250 <= car_weight < 2500:
                                nox_std = 1480
                            elif 2500 <= car_weight < 2750:
                                nox_std = 1340
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 1250
                            elif 3000 <= car_weight:
                                nox_std = 1240
                        elif '20010101' <= product_date <= '20031231':
                            if car_weight < 1500:
                                nox_std = 2070
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1830
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1640
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1480
                            elif 2250 <= car_weight < 2500:
                                nox_std = 1310
                            elif 2500 <= car_weight < 2750:
                                nox_std = 1190
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 1110
                            elif 3000 <= car_weight:
                                nox_std = 1110
                        elif '20040101' <= product_date:
                            if car_weight < 1500:
                                nox_std = 1800
                            elif 1500 <= car_weight < 1750:
                                nox_std = 1590
                            elif 1750 <= car_weight < 2000:
                                nox_std = 1430
                            elif 2000 <= car_weight < 2250:
                                nox_std = 1290
                            elif 2250 <= car_weight < 2500:
                                nox_std = 1140
                            elif 2500 <= car_weight < 2750:
                                nox_std = 1040
                            elif 2750 <= car_weight <= 3000:
                                nox_std = 970
                            elif 3000 <= car_weight:
                                nox_std = 970

            else: # N N N
                co_std = 1
                hc_std = 1
                nox_std = 1

        else: # '무부하검사(TSI)', '무부하검사(정지가동)', '무부하검사(급가속)' # CO(무부하매연측정치1), HC(무부하매연측정치2), 공기과잉률(무부하매연측정치6) 사용
            nox_std = 1
            if ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox2) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # Y Y Y
                if ci2 == '경형':
                    if product_date < '19971231':
                        co_std = 4.5
                        hc_std = 1200
                    elif '19980101' <= product_date < '20001231':
                        co_std = 2.5
                        hc_std = 400
                    elif '20010101' <= product_date < '20031231':
                        co_std = 1.2
                        hc_std = 220
                    elif '20040101' <= product_date:
                        co_std = 1.0
                        hc_std = 150
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19871231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                            hc_std = 220
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                            hc_std = 220
                        elif '20060101' <= product_date:
                            co_std = 1.0
                            hc_std = 120
                    else:
                        if product_date < '19871231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                            hc_std = 400
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                            hc_std = 220
                        elif '20060101' <= product_date:
                            co_std = 1.0
                            hc_std = 120
                elif (ci == '승합') or (ci == '화물') or (ci == '특수'):
                    if ci2 == '소형':
                        if product_date < '19891231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '19900101' <= product_date < '20031231':
                            co_std = 2.5
                            hc_std = 400
                        elif '20040101' <= product_date:
                            co_std = 1.2
                            hc_std = 220
                    elif ci2 == '중형' or ci2 == '대형':
                        if product_date < '20031231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '20040101' <= product_date:
                            co_std = 2.5
                            hc_std = 400

            elif ( str(co1) != 'nan' and str(hc1) != 'nan' and str(nox2) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox2)) ): # Y Y N
                if ci2 == '경형':
                    if product_date < '19971231':
                        co_std = 4.5
                        hc_std = 1200
                    elif '19980101' <= product_date < '20001231':
                        co_std = 2.5
                        hc_std = 400
                    elif '20010101' <= product_date < '20031231':
                        co_std = 1.2
                        hc_std = 220
                    elif '20040101' <= product_date:
                        co_std = 1.0
                        hc_std = 150
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19871231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                            hc_std = 220
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                            hc_std = 220
                        elif '20060101' <= product_date:
                            co_std = 1.0
                            hc_std = 120
                    else:
                        if product_date < '19871231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                            hc_std = 400
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                            hc_std = 220
                        elif '20060101' <= product_date:
                            co_std = 1.0
                            hc_std = 120
                elif (ci == '승합') or (ci == '화물') or (ci == '특수'):
                    if ci2 == '소형':
                        if product_date < '19891231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '19900101' <= product_date < '20031231':
                            co_std = 2.5
                            hc_std = 400
                        elif '20040101' <= product_date:
                            co_std = 1.2
                            hc_std = 220
                    elif ci2 == '중형' or ci2 == '대형':
                        if product_date < '20031231':
                            co_std = 4.5
                            hc_std = 1200
                        elif '20040101' <= product_date:
                            co_std = 2.5
                            hc_std = 400

            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox2) != 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # Y N Y
                hc_std = 1
                if ci2 == '경형':
                    if product_date < '19971231':
                        co_std = 4.5
                    elif '19980101' <= product_date < '20001231':
                        co_std = 2.5
                    elif '20010101' <= product_date < '20031231':
                        co_std = 1.2
                    elif '20040101' <= product_date:
                        co_std = 1.0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19871231':
                            co_std = 4.5
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                        elif '20060101' <= product_date:
                            co_std = 1.0
                    else:
                        if product_date < '19871231':
                            co_std = 4.5
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                        elif '20060101' <= product_date:
                            co_std = 1.0
                elif (ci == '승합') or (ci == '화물') or (ci == '특수'):
                    if ci2 == '소형':
                        if product_date < '19891231':
                            co_std = 4.5
                        elif '19900101' <= product_date < '20031231':
                            co_std = 2.5
                        elif '20040101' <= product_date:
                            co_std = 1.2
                    elif ci2 == '중형' or ci2 == '대형':
                        if product_date < '20031231':
                            co_std = 4.5
                        elif '20040101' <= product_date:
                            co_std = 2.5

            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox2) != 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # N Y Y
                co_std = 1
                if ci2 == '경형':
                    if product_date < '19971231':
                        hc_std = 1200
                    elif '19980101' <= product_date < '20001231':
                        hc_std = 400
                    elif '20010101' <= product_date < '20031231':
                        hc_std = 220
                    elif '20040101' <= product_date:
                        hc_std = 150
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19871231':
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            hc_std = 220
                        elif '20010101' <= product_date < '20051231':
                            hc_std = 220
                        elif '20060101' <= product_date:
                            hc_std = 120
                    else:
                        if product_date < '19871231':
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            hc_std = 400
                        elif '20010101' <= product_date < '20051231':
                            hc_std = 220
                        elif '20060101' <= product_date:
                            hc_std = 120
                elif (ci == '승합') or (ci == '화물') or (ci == '특수'):
                    if ci2 == '소형':
                        if product_date < '19891231':
                            hc_std = 1200
                        elif '19900101' <= product_date < '20031231':
                            hc_std = 400
                        elif '20040101' <= product_date:
                            hc_std = 220
                    elif ci2 == '중형' or ci2 == '대형':
                        if product_date < '20031231':
                            hc_std = 1200
                        elif '20040101' <= product_date:
                            hc_std = 400

            elif ( str(co1) != 'nan' and str(hc1) == 'nan' and str(nox2) == 'nan' ) or ( gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox2)) ): # Y N N
                hc_std = 1
                if ci2 == '경형':
                    if product_date < '19971231':
                        co_std = 4.5
                    elif '19980101' <= product_date < '20001231':
                        co_std = 2.5
                    elif '20010101' <= product_date < '20031231':
                        co_std = 1.2
                    elif '20040101' <= product_date:
                        co_std = 1.0
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19871231':
                            co_std = 4.5
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                        elif '20060101' <= product_date:
                            co_std = 1.0
                    else:
                        if product_date < '19871231':
                            co_std = 4.5
                        elif '19880101' <= product_date < '20001231':
                            co_std = 1.2
                        elif '20010101' <= product_date < '20051231':
                            co_std = 1.2
                        elif '20060101' <= product_date:
                            co_std = 1.0
                elif (ci == '승합') or (ci == '화물') or (ci == '특수'):
                    if ci2 == '소형':
                        if product_date < '19891231':
                            co_std = 4.5
                        elif '19900101' <= product_date < '20031231':
                            co_std = 2.5
                        elif '20040101' <= product_date:
                            co_std = 1.2
                    elif ci2 == '중형' or ci2 == '대형':
                        if product_date < '20031231':
                            co_std = 4.5
                        elif '20040101' <= product_date:
                            co_std = 2.5

            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox2) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and gas_mea_value_p.search(str(hc1)) and not gas_mea_value_p.search(str(nox2)) ): # N Y N
                co_std = 1
                if ci2 == '경형':
                    if product_date < '19971231':
                        hc_std = 1200
                    elif '19980101' <= product_date < '20001231':
                        hc_std = 400
                    elif '20010101' <= product_date < '20031231':
                        hc_std = 220
                    elif '20040101' <= product_date:
                        hc_std = 150
                elif ci == '승용':
                    if fuel == '휘발유' or fuel == '알코올':
                        if product_date < '19871231':
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            hc_std = 220
                        elif '20010101' <= product_date < '20051231':
                            hc_std = 220
                        elif '20060101' <= product_date:
                            hc_std = 120
                    else:
                        if product_date < '19871231':
                            hc_std = 1200
                        elif '19880101' <= product_date < '20001231':
                            hc_std = 400
                        elif '20010101' <= product_date < '20051231':
                            hc_std = 220
                        elif '20060101' <= product_date:
                            hc_std = 120
                elif (ci == '승합') or (ci == '화물') or (ci == '특수'):
                    if ci2 == '소형':
                        if product_date < '19891231':
                            hc_std = 1200
                        elif '19900101' <= product_date < '20031231':
                            hc_std = 400
                        elif '20040101' <= product_date:
                            hc_std = 220
                    elif ci2 == '중형' or ci2 == '대형':
                        if product_date < '20031231':
                            hc_std = 1200
                        elif '20040101' <= product_date:
                            hc_std = 400

            elif ( str(co1) == 'nan' and str(hc1) != 'nan' and str(nox2) == 'nan' ) or ( not gas_mea_value_p.search(str(co1)) and not gas_mea_value_p.search(str(hc1)) and gas_mea_value_p.search(str(nox2)) ): # N N Y
                co_std = 1
                hc_std = 1

            else: # N N N
                co_std = 1
                hc_std = 1


        # Total 값 계산
        fail_point = 100 - p_f_rate * 100
        gas_point = (100 - mco*100/co_std)*0.25 + (100 - mhc*100/hc_std)*0.25 + (100 - mnox*100/nox_std)*0.25 + fail_point*0.25

        # 리스트에 저장
        mean_co_list.append(mco)
        mean_hc_list.append(mhc)
        mean_nox_list.append(mnox)
        fail_rate_list.append(p_f_rate)
        emno_list.append(one)
        car_num_list.append(car_num)
        total_point_list.append(gas_point)

  0%|          | 0/85 [00:00<?, ?it/s]


KeyError: '적재중량'

# 코드 마지막